# 📄 AskYourDocuments: Chat with Your Documents using AI

Welcome to **AskYourDocuments**! This notebook creates an intelligent document analysis system that lets you upload various document types (PDF, DOCX, XLSX, PPTX, images) and ask questions about their content using powerful AI models.

## 🔍 What This Notebook Does

1. Sets up a complete document processing pipeline
2. Extracts text, tables, and image content from your documents
3. Creates a searchable knowledge base from your documents
4. Deploys a web interface to upload documents and ask questions
5. Processes your queries using AI and retrieves targeted information

## 📋 How To Use This Notebook (in Google Colab)

1. **Run each cell in sequence** from top to bottom
2. **Set up your API keys** in the second cell:
   - Azure API key - for embeddings and LLM (required)
   - Hugging Face API token - for vision features (optional)
   - Ngrok authtoken - for better web access (recommended)
3. **Upload HTML and JS files** when prompted (or use the auto-generated basic templates)
4. The **final cell launches the web interface** via ngrok link

## 🔑 Required API Keys

- **Azure AI Inference API Key**: Required for document processing and query answering. Get one from your [Azure AI account](https://github.com/settings/tokens).
- **Hugging Face API Token**: Optional for enhanced vision features. Get a token from your [Hugging Face account settings](https://huggingface.co/settings/tokens).
- **Ngrok Authtoken**: Recommended for stable public URLs. Sign up at [ngrok.com](https://ngrok.com/) and get your token.

## 🚀 Getting Started

Run the first cell below to install all required dependencies, then proceed through each cell sequentially. Make sure to configure your API keys in the second cell!

In [1]:
# --- 0. Installations ---
!pip install Flask Flask-CORS pyngrok -q
!pip install --upgrade pymupdf Flask Flask-CORS pyngrok loguru -q
!pip install langchain tiktoken sentence-transformers accelerate chromadb torch numpy pdfplumber pdf2image pytesseract opencv-python-headless Pillow huggingface_hub python-dotenv -q
!pip install azure-ai-inference openai langchain-openai -q
!pip install python-docx openpyxl python-pptx -q
!sudo apt-get update -qq
!sudo apt-get install -y tesseract-ocr tesseract-ocr-eng poppler-utils libreoffice unoconv -qq
print("--- Dependencies installed/updated ---")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 

In [ ]:
AZURE_EMBEDDING_API_KEY_SECRET = 'KEY' # add your own key
HF_API_TOKEN_SECRET = 'KEY' # add your own key
!ngrok config add-authtoken KEY # add your own key

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Frontend for colab

# 1. Create directories for templates and static files
import os
import shutil

# These directories will be created in your Colab's current working directory
# Usually /content/
TEMPLATES_DIR = 'templates'
STATIC_DIR = 'static'
STATIC_JS_DIR = os.path.join(STATIC_DIR, 'js')

os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(STATIC_DIR, exist_ok=True)
os.makedirs(STATIC_JS_DIR, exist_ok=True)

print(f"Directory '{TEMPLATES_DIR}' created (or already exists).")
print(f"Directory '{STATIC_JS_DIR}' created (or already exists).")

# 2. Write basic HTML files instead of uploading
hero_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hero Geometric Section</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        /* PASTE THE FULL CSS FROM THE PREVIOUS RESPONSE HERE */
        /* For brevity, I'm not repeating the full CSS block here.
           Ensure the CSS provided in the last message is inserted here.
           It starts with: / * Custom CSS for animations and specific styles * /
           and ends with:   .files-page.active { ... }
        */

        /* Custom CSS for animations and specific styles */
        body {
            font-family: sans-serif; /* Default Tailwind font stack */
        }

        #ask-docs-title:hover {
            cursor: pointer;
        }

        #hover-indicator {
            position: fixed; /* Use fixed to position relative to viewport */
            display: none; /* Hidden by default */
            padding: 6px 10px;
            background-color: rgba(30, 30, 30, 0.85); /* Darker, more opaque background */
            border: 1px solid rgba(255, 255, 255, 0.15); /* Subtler border */
            border-radius: 6px;
            color: white;
            font-size: 0.75rem; /* 12px */
            font-weight: 500; /* Semi-bold text */
            pointer-events: none; /* So it doesn't interfere with mouse events on other elements */
            z-index: 50; /* Ensure it's above other elements */
            backdrop-filter: blur(4px); /* Keep a slight blur for modern feel */
            box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            transform: translate(10px, 10px); /* Initial offset from cursor */
        }

        /* Hover indicator style - Note: This duplicates a previous #hover-indicator. Consolidate if desired. */
        #hover-indicator { /* This definition seems to be a more specific one, keeping it */
            position: absolute;
            display: none;
            background-color: rgba(29, 23, 23, 0.5);
            backdrop-filter: blur(5px);
            color: rgba(255, 255, 255, 0.9);
            font-size: 0.9rem;
            padding: 0.5rem 1rem;
            border-radius: 9999px;
            border: 1px solid rgba(255, 255, 255, 0.2);
            z-index: 50;
            pointer-events: none;
            box-shadow: 0 0 20px rgba(99, 102, 241, 0.3);
            font-weight: 500;
        }

        /* Animation for the outer part of ElegantShape */
        @keyframes elegantShapeAppear {
            0% {
                opacity: 0;
                transform: translateY(-150px) rotate(calc(var(--initial-rotate, 0deg) - 15deg));
            }
            50% { /* Opacity transition duration is 1.2s, total is 2.4s */
                opacity: 1;
            }
            100% {
                opacity: 1;
                transform: translateY(0) rotate(var(--final-rotate, 0deg));
            }
        }

        /* Animation for the inner part (oscillation) of ElegantShape */
        @keyframes elegantShapeOscillate {
            0%, 100% {
                transform: translateY(0);
            }
            50% {
                transform: translateY(15px);
            }
        }

        /* Animation for fade-up text elements */
        @keyframes fadeUp {
            0% {
                opacity: 0;
                transform: translateY(30px);
            }
            100% {
                opacity: 1;
                transform: translateY(0);
            }
        }

        /* Base class for elegant shapes' outer container to apply animation */
        .elegant-shape-outer-animated {
            animation-name: elegantShapeAppear;
            animation-duration: 2.4s;
            animation-timing-function: cubic-bezier(0.23, 0.86, 0.39, 0.96);
            animation-fill-mode: forwards; /* Keep the state of the last keyframe */
        }

        /* Base class for elegant shapes' inner container to apply oscillation */
        .elegant-shape-inner-animated {
            animation-name: elegantShapeOscillate;
            animation-duration: 12s;
            animation-timing-function: easeInOut;
            animation-iteration-count: infinite;
        }

        /* Base class for fade-up animated text elements */
        .fade-up-animated {
            opacity: 0; /* Start hidden, animation will make it visible */
            animation-name: fadeUp;
            animation-duration: 1s;
            animation-timing-function: cubic-bezier(0.25, 0.4, 0.25, 1);
            animation-fill-mode: forwards; /* Keep the state of the last keyframe */
        }

        /* Pixel Sparks Animation */
        @keyframes sparkle {
            0%, 100% {
                opacity: 0.1;
                transform: scale(0.8);
            }
            50% {
                opacity: 0.8;
                transform: scale(1.2);
            }
        }

        @keyframes slowFloat {
            0%, 100% {
                transform: translateY(0) translateX(0);
            }
            50% {
                transform: translateY(-10px) translateX(5px);
            }
        }

        .spark {
            position: absolute;
            width: 1px;
            height: 1px;
            background-color: white;
            border-radius: 50%;
            animation: sparkle var(--spark-duration, 5s) infinite cubic-bezier(0.45, 0.05, 0.55, 0.95),
                       slowFloat 15s infinite ease-in-out;
            animation-delay: var(--spark-delay, 0s);
            opacity: var(--spark-opacity, 0.5);
        }

        .spark-medium {
            width: 2px;
            height: 2px;
        }

        .spark-large {
            width: 3px;
            height: 3px;
            box-shadow: 0 0 4px 1px rgba(255, 255, 255, 0.3);
        }

        .spark-blue {
            background-color: rgba(99, 102, 241, 0.8);
            box-shadow: 0 0 4px 1px rgba(99, 102, 241, 0.4);
        }

        .spark-purple {
            background-color: rgba(139, 92, 246, 0.8);
            box-shadow: 0 0 4px 1px rgba(139, 92, 246, 0.4);
        }

        .spark-rose {
            background-color: rgba(244, 63, 94, 0.8);
            box-shadow: 0 0 4px 1px rgba(244, 63, 94, 0.4);
        }

        /* Shape Animations */
        @keyframes shapeMoveLeft {
            0% { transform: translateX(0) rotate(var(--final-rotate, 0deg)); opacity: 1; }
            100% { transform: translateX(-120vw) rotate(calc(var(--final-rotate, 0deg) - 30deg)); opacity: 0; }
        }
        @keyframes shapeMoveRight {
            0% { transform: translateX(0) rotate(var(--final-rotate, 0deg)); opacity: 1; }
            100% { transform: translateX(120vw) rotate(calc(var(--final-rotate, 0deg) + 30deg)); opacity: 0; }
        }
        .animate-shape-move-left {
            animation: shapeMoveLeft 1.5s ease-out forwards !important;
        }
        .animate-shape-move-right {
            animation: shapeMoveRight 1.5s ease-out forwards !important;
        }

        /* Content Split Animations */
        @keyframes contentSplitLeft {
            0% {
                opacity: 1;
                transform: translateX(0) rotate(0deg);
                filter: blur(0px);
            }
            100% {
                opacity: 0;
                transform: translateX(-60vw) rotate(-10deg);
                filter: blur(8px);
            }
        }

        @keyframes contentSplitRight {
            0% {
                opacity: 1;
                transform: translateX(0) rotate(0deg);
                filter: blur(0px);
            }
            100% {
                opacity: 0;
                transform: translateX(60vw) rotate(10deg);
                filter: blur(8px);
            }
        }

        @keyframes contentMiddleFade {
            0% {
                opacity: 1;
                transform: scale(1);
                filter: blur(0px);
            }
            100% {
                opacity: 0;
                transform: scale(0.85);
                filter: blur(15px);
            }
        }

        .animate-content-left {
            animation: contentSplitLeft 1.5s cubic-bezier(0.65, 0, 0.35, 1) forwards;
        }

        .animate-content-right {
            animation: contentSplitRight 1.5s cubic-bezier(0.65, 0, 0.35, 1) forwards;
        }

        .animate-content-middle {
            animation: contentMiddleFade 1.2s cubic-bezier(0.65, 0, 0.35, 1) forwards;
        }

        /* Interactive Hover Button Styles */
        .interactive-hover-btn {
            position: relative;
            overflow: hidden;
            border-radius: 9999px;
            width: 128px;
            padding: 0.5rem 1.25rem;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s;
            border: 1px solid rgba(255, 255, 255, 0.15);
        }

        .interactive-hover-btn .btn-text {
            display: inline-block;
            transform: translateX(0);
            transition: all 0.3s;
            z-index: 2;
            position: relative;
            font-size: 1rem;
        }

        .interactive-hover-btn:hover .btn-text {
            font-size: 1.1rem; /* Increased font size on hover for Ask button */
            transform: translateX(12px);
            opacity: 0;
        }

        .interactive-hover-btn .hover-content {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 6px;
            transform: translateX(12px);
            opacity: 0;
            transition: all 0.3s;
            z-index: 2;
        }

        .interactive-hover-btn:hover .hover-content {
            transform: translateX(0);
            opacity: 1;
        }

        .interactive-hover-btn .btn-bg-dot {
            position: absolute;
            left: 15%; /* Moved further left to avoid text overlap */
            top: 40%;
            height: 8px;
            width: 8px;
            border-radius: 8px;
            background-color: rgb(79, 70, 229); /* indigo-600 */
            transition: all 0.3s;
            z-index: 1;
        }

        .interactive-hover-btn:hover .btn-bg-dot {
            left: 0;
            top: 0;
            height: 100%;
            width: 100%;
            border-radius: 9999px;
            transform: scale(1.2);
        }

        /* Special styling for Process mode button */
        .interactive-hover-btn.process-mode {
            padding-left: 1.25rem; /* 5 in Tailwind */
            padding-right: 1.25rem;
            min-width: 110px; /* Ensure minimum width for the button */
        }

        .interactive-hover-btn.process-mode .btn-bg-dot {
            left: 10%; /* Move further left for process button to avoid text overlap */
            top: 42%;
        }

        .interactive-hover-btn:hover {
            color: white;
            box-shadow: 0 0 20px rgba(79, 70, 229, 0.4);
        }

        /* Arrow icon for the button */
        .arrow-right {
            height: 16px;
            width: 16px;
            stroke: currentColor;
            stroke-width: 2;
            fill: none;
        }

        /* Upload Component Styles */
        /* Custom scrollbar */
        .scrollbar-thin {
            scrollbar-width: thin;
            scrollbar-color: #52525b transparent; /* zinc-600 transparent */
        }
        .scrollbar-thin::-webkit-scrollbar {
            width: 8px;
            height: 8px;
        }
        .scrollbar-thin::-webkit-scrollbar-track {
            background: transparent;
        }
        .scrollbar-thin::-webkit-scrollbar-thumb {
            background-color: #52525b; /* zinc-600 */
            border-radius: 4px;
            border: 2px solid transparent;
            background-clip: content-box;
        }

        /* Settings Panel Styles */
        .settings-panel {
            background-color: rgba(24, 24, 27, 0.85); /* zinc-900 with opacity */
            border: 1px solid rgba(255, 255, 255, 0.08);
            border-radius: 1rem;
            backdrop-filter: blur(10px);
            transform: translateY(20px);
            opacity: 0;
            transition: all 0.5s cubic-bezier(0.16, 1, 0.3, 1);
            box-shadow: 0 10px 25px -5px rgba(0, 0, 0, 0.3), 0 8px 10px -6px rgba(0, 0, 0, 0.2);
            overflow: hidden;
            max-width: 100%;
            margin: 0 auto;
        }

        .settings-panel.show {
            transform: translateY(0);
            opacity: 1;
        }

        .settings-panel::before {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            height: 1px;
            background: linear-gradient(90deg, transparent, rgba(99, 102, 241, 0.3), transparent);
        }

        /* Settings Card Styles (kept from original hero-geometric for cards holding the toggles) */
        .settings-card {
            background-color: rgba(39, 39, 42, 0.5); /* zinc-800 with opacity */
            border: 1px solid rgba(63, 63, 70, 0.5); /* zinc-700 with opacity */
            transition: all 0.3s;
            position: relative;
            overflow: hidden;
            margin-bottom: 8px; /* Add spacing between vertical cards */
            width: 100%; /* Ensure full width in vertical layout */
        }

        .settings-card:hover {
            border-color: rgba(99, 102, 241, 0.4); /* indigo-500 with opacity */
            box-shadow: 0 0 15px rgba(99, 102, 241, 0.2);
            transform: translateY(-2px);
        }

        .settings-card:after {
            content: '';
            position: absolute;
            inset: 0;
            z-index: -1;
            background: radial-gradient(circle at center, rgba(99, 102, 241, 0.15), transparent 70%);
            opacity: 0;
            transition: opacity 0.5s;
        }

        .settings-card:hover:after {
            opacity: 1;
        }

        /* START: LIQUID TOGGLE CSS - INTEGRATED */
        .gooey-filter-definition {
            position: fixed;
            width: 0;
            height: 0;
            overflow: hidden; /* Ensure it doesn't take up space */
        }

        .liquid-toggle-switch {
            position: relative;
            display: block;
            cursor: pointer;
            height: 32px;
            width: 52px;

            --c-active: #4F46E5; /* indigo-600 (Matches hero-geometric theme) */
            --c-success: #10B981;
            --c-warning: #F59E0B;
            --c-danger: #EF4444;
            --c-active-inner: #FFFFFF;
            --c-default: rgba(82, 82, 91, 0.4); /* Matches hero-geometric settings panel toggle bg */
            --c-default-dark: rgba(82, 82, 91, 0.6); /* Matches hero-geometric settings panel toggle bg hover */
            --c-black: #1B1B22;

            transform: translateZ(0);
            -webkit-transform: translateZ(0);
            backface-visibility: hidden;
            -webkit-backface-visibility: hidden;
            perspective: 1000px;
            -webkit-perspective: 1000px;
        }

        .liquid-toggle-switch.default { --c-background: var(--c-active); }
        .liquid-toggle-switch.success { --c-background: var(--c-success); }
        .liquid-toggle-switch.warning { --c-background: var(--c-warning); }
        .liquid-toggle-switch.danger { --c-background: var(--c-danger); }

        .liquid-toggle-input {
            height: 100%;
            width: 100%;
            cursor: pointer;
            appearance: none;
            -webkit-appearance: none;
            border-radius: 9999px;
            background-color: var(--c-default);
            outline: none;
            transition: background-color 0.5s ease;
            transform: translate3d(0,0,0);
            -webkit-transform: translate3d(0,0,0);
        }

        .liquid-toggle-input:hover {
            background-color: var(--c-default-dark);
        }

        .liquid-toggle-input:checked {
            background-color: var(--c-background);
        }
        .liquid-toggle-input:checked:hover {
            background-color: var(--c-background);
        }

        .liquid-toggle-svg {
            position: absolute;
            inset: 0;
            pointer-events: none;
            fill: var(--c-active-inner);
            transform: translate3d(0,0,0);
            -webkit-transform: translate3d(0,0,0);
        }

        .liquid-toggle-circle {
            transform-gpu: translate3d(0,0,0);
            -webkit-transform: translate3d(0,0,0);
            transition: transform 0.5s cubic-bezier(0.34, 1.56, 0.64, 1);
            backface-visibility: hidden;
            -webkit-backface-visibility: hidden;
            will-change: transform;
        }

        .liquid-toggle-circle-1 {
            transform-origin: 16px 16px;
            transform: translateX(0px) scale(1);
        }

        .liquid-toggle-circle-2 {
            transform-origin: 36px 16px;
            transform: translateX(-12px) scale(0);
        }

        .liquid-toggle-drop-circle {
            transform-gpu: translate3d(0,0,0);
            -webkit-transform: translate3d(0,0,0);
            transition: transform 0.7s cubic-bezier(0.34, 1.56, 0.64, 1), opacity 0.7s ease;
            opacity: 0;
            transform: translateY(10px) scale(0);
            will-change: transform, opacity;
            filter: drop-shadow(0 0 2px rgba(255, 255, 255, 0.5));
        }

        .liquid-toggle-switch.is-checked .liquid-toggle-circle-1 {
            transform: translateX(12px) scale(0);
        }

        .liquid-toggle-switch.is-checked .liquid-toggle-circle-2 {
            transform: translateX(0px) scale(1);
        }

        .liquid-toggle-switch.is-checked .liquid-toggle-drop-circle {
            opacity: 1;
            transform: translateY(0) scale(1);
        }
        /* END: LIQUID TOGGLE CSS - INTEGRATED */


        /* Upload Component Animations */
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }

        @keyframes glow {
            0%, 100% { box-shadow: 0 0 8px rgba(59, 130, 246, 0.6); }
            50% { box-shadow: 0 0 16px rgba(59, 130, 246, 0.8); }
        }
        @keyframes bounce-y {
            0%, 100% { transform: translateY(0); }
            50% { transform: translateY(-5px); }
        }
        @keyframes pulse-glow-effect {
            0%, 100% { opacity: 0.5; transform: scale(0.95); }
            50% { opacity: 1; transform: scale(1.05); }
        }
        .animate-spin-custom {
            animation: spin 1s linear infinite;
        }
        @keyframes spin {
            from { transform: rotate(0deg); }
            to { transform: rotate(360deg); }
        }

        /* Pulse animation for the Ask button */
        @keyframes button-pulse {
            0% { box-shadow: 0 0 0 0 rgba(99, 102, 241, 0.4); }
            70% { box-shadow: 0 0 0 10px rgba(99, 102, 241, 0); }
            100% { box-shadow: 0 0 0 0 rgba(99, 102, 241, 0); }
        }

        .animate-pulse { /* Tailwind class name, ensure it doesn't conflict if you also use Tailwind's pulse */
            animation: button-pulse 1.5s cubic-bezier(0.4, 0, 0.6, 1) infinite;
        }

        /* File item entry/exit animations */
        .file-item {
            transition: opacity 0.3s ease-out, transform 0.3s ease-out;
        }
        .file-item-enter {
            opacity: 0;
            transform: translateY(20px) scale(0.97);
        }
        .file-item-enter-active {
            opacity: 1;
            transform: translateY(0) scale(1);
        }
        .file-item-exit-active {
            opacity: 0;
            transform: translateY(-20px) scale(0.95);
        }

        #upload-component {
            display: none; /* Initially hidden by JS */
            opacity: 0;    /* Start transparent for fade-in */
            animation: fadeIn 0.8s ease-out forwards; /* Custom fadeIn animation */
        }

        /* Animation for checkmark appearing on completed upload */
        @keyframes fadeInScale {
            to {
                opacity: 1;
                transform: scale(1);
            }
        }

        /* Pagination styles */
        .pagination {
            display: flex;
            justify-content: center;
            gap: 0.5rem;
            margin-top: 1rem;
        }

        .pagination-btn {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 2rem;
            height: 2rem;
            border-radius: 0.5rem;
            background-color: rgba(63, 63, 70, 0.5);
            color: rgba(255, 255, 255, 0.7);
            border: 1px solid rgba(255, 255, 255, 0.1);
            cursor: pointer;
            transition: all 0.2s ease;
        }

        .pagination-btn:hover {
            background-color: rgba(99, 102, 241, 0.3);
            color: rgba(255, 255, 255, 0.9);
        }

        .pagination-btn:disabled {
            opacity: 0.5;
            cursor: not-allowed;
        }

        .pagination-btn.active {
            background-color: rgba(99, 102, 241, 0.5);
            color: white;
        }

        /* Upload component file transitions */
        .files-page {
            opacity: 0;
            transform: translateX(20px);
            position: absolute;
            width: 100%;
            transition: opacity 0.3s ease, transform 0.3s ease;
            pointer-events: none;
        }

        .files-page.active {
            opacity: 1;
            transform: translateX(0);
            position: relative;
            pointer-events: all;
        }
    </style>
</head>
<body class="bg-[#030303]">

    <!-- SVG Gooey Filter Definition -->
    <svg class="gooey-filter-definition">
        <defs>
            <filter id="goo">
                <feGaussianBlur in="SourceGraphic" stdDeviation="2" result="blur" />
                <feColorMatrix in="blur" mode="matrix" values="1 0 0 0 0  0 1 0 0 0  0 0 1 0 0  0 0 0 18 -7" result="goo" />
                <feComposite in="SourceGraphic" in2="goo" operator="atop" />
            </filter>
        </defs>
    </svg>

    <div class="relative min-h-screen w-full flex items-center justify-center overflow-hidden bg-[#030303]">
        <canvas id="animated-background-canvas" class="fixed inset-0 w-full h-full" style="z-index: 0; display: none;"></canvas>
        <div class="absolute inset-0 bg-gradient-to-br from-indigo-500/[0.05] via-transparent to-rose-500/[0.05] blur-3xl"></div>

        <!-- Pixel Sparks Layer -->
        <div class="absolute inset-0 overflow-hidden pointer-events-none">
            <!-- Spark elements... (kept for brevity, same as your original) -->
            <div class="spark" style="top: 5%; left: 10%; --spark-delay: 0s; --spark-duration: 4s;"></div>
            <div class="spark spark-medium spark-blue" style="top: 15%; left: 25%; --spark-delay: 2s; --spark-duration: 6s;"></div>
            <div class="spark" style="top: 8%; left: 40%; --spark-delay: 1s; --spark-duration: 5s;"></div>
            <div class="spark spark-large spark-rose" style="top: 12%; left: 65%; --spark-delay: 0.5s; --spark-duration: 7s;"></div>
            <div class="spark" style="top: 3%; left: 80%; --spark-delay: 3s; --spark-duration: 4.5s;"></div>
            <div class="spark spark-medium" style="top: 20%; left: 90%; --spark-delay: 1.5s; --spark-duration: 5.5s;"></div>
            <div class="spark spark-medium spark-purple" style="top: 30%; left: 5%; --spark-delay: 2.5s; --spark-duration: 6.5s;"></div>
            <div class="spark" style="top: 35%; left: 20%; --spark-delay: 0.7s; --spark-duration: 4.2s;"></div>
            <div class="spark spark-large" style="top: 28%; left: 45%; --spark-delay: 3.5s; --spark-duration: 7.5s;"></div>
            <div class="spark spark-medium spark-blue" style="top: 32%; left: 70%; --spark-delay: 1.2s; --spark-duration: 5.2s;"></div>
            <div class="spark" style="top: 25%; left: 85%; --spark-delay: 2.2s; --spark-duration: 4.8s;"></div>
            <div class="spark" style="top: 50%; left: 8%; --spark-delay: 1.8s; --spark-duration: 5.8s;"></div>
            <div class="spark spark-medium spark-rose" style="top: 45%; left: 30%; --spark-delay: 0.3s; --spark-duration: 6.3s;"></div>
            <div class="spark" style="top: 55%; left: 50%; --spark-delay: 2.8s; --spark-duration: 4.1s;"></div>
            <div class="spark spark-large spark-blue" style="top: 48%; left: 75%; --spark-delay: 1.1s; --spark-duration: 7.1s;"></div>
            <div class="spark spark-medium" style="top: 52%; left: 95%; --spark-delay: 3.3s; --spark-duration: 5.3s;"></div>
            <div class="spark spark-large spark-purple" style="top: 70%; left: 15%; --spark-delay: 0.9s; --spark-duration: 6.9s;"></div>
            <div class="spark" style="top: 65%; left: 35%; --spark-delay: 3.7s; --spark-duration: 4.7s;"></div>
            <div class="spark spark-medium" style="top: 75%; left: 55%; --spark-delay: 1.4s; --spark-duration: 5.4s;"></div>
            <div class="spark" style="top: 68%; left: 80%; --spark-delay: 2.6s; --spark-duration: 4.6s;"></div>
            <div class="spark" style="top: 85%; left: 10%; --spark-delay: 3.2s; --spark-duration: 4.2s;"></div>
            <div class="spark spark-medium spark-rose" style="top: 90%; left: 30%; --spark-delay: 0.6s; --spark-duration: 6.6s;"></div>
            <div class="spark spark-large" style="top: 80%; left: 60%; --spark-delay: 2.1s; --spark-duration: 7.2s;"></div>
            <div class="spark spark-medium spark-blue" style="top: 88%; left: 80%; --spark-delay: 1.7s; --spark-duration: 5.7s;"></div>
            <div class="spark" style="top: 92%; left: 95%; --spark-delay: 2.9s; --spark-duration: 4.9s;"></div>
        </div>

        <div class="absolute inset-0 overflow-hidden">
            <!-- Elegant shape elements... (kept for brevity, same as your original) -->
            <div id="elegant-shape-1" class="elegant-shape-outer-animated absolute left-[-10%] md:left-[-5%] top-[15%] md:top-[20%]"
                 style="--initial-rotate: 12deg; --final-rotate: 12deg; animation-delay: 0.3s;">
                <div class="elegant-shape-inner-animated relative" style="width: 600px; height: 140px;">
                    <div class="absolute inset-0 rounded-full bg-gradient-to-r to-transparent from-indigo-500/[0.15] backdrop-blur-[2px] border-2 border-white/[0.15] shadow-[0_8px_32px_0_rgba(255,255,255,0.1),0_0_40px_3px_rgba(99,102,241,0.4)] after:absolute after:inset-0 after:rounded-full after:bg-[radial-gradient(circle_at_50%_50%,rgba(255,255,255,0.2),transparent_70%)]">
                    </div>
                </div>
            </div>
            <div id="elegant-shape-2" class="elegant-shape-outer-animated absolute right-[-5%] md:right-[0%] top-[70%] md:top-[75%]"
                 style="--initial-rotate: -15deg; --final-rotate: -15deg; animation-delay: 0.5s;">
                <div class="elegant-shape-inner-animated relative" style="width: 500px; height: 120px;">
                    <div class="absolute inset-0 rounded-full bg-gradient-to-r to-transparent from-rose-500/[0.15] backdrop-blur-[2px] border-2 border-white/[0.15] shadow-[0_8px_32px_0_rgba(255,255,255,0.1),0_0_40px_3px_rgba(244,63,94,0.4)] after:absolute after:inset-0 after:rounded-full after:bg-[radial-gradient(circle_at_50%_50%,rgba(255,255,255,0.2),transparent_70%)]">
                    </div>
                </div>
            </div>
            <div id="elegant-shape-3" class="elegant-shape-outer-animated absolute left-[5%] md:left-[10%] bottom-[5%] md:bottom-[10%]"
                 style="--initial-rotate: -8deg; --final-rotate: -8deg; animation-delay: 0.4s;">
                <div class="elegant-shape-inner-animated relative" style="width: 300px; height: 80px;">
                    <div class="absolute inset-0 rounded-full bg-gradient-to-r to-transparent from-violet-500/[0.15] backdrop-blur-[2px] border-2 border-white/[0.15] shadow-[0_8px_32px_0_rgba(255,255,255,0.1),0_0_40px_3px_rgba(139,92,246,0.4)] after:absolute after:inset-0 after:rounded-full after:bg-[radial-gradient(circle_at_50%_50%,rgba(255,255,255,0.2),transparent_70%)]">
                    </div>
                </div>
            </div>
            <div id="elegant-shape-4" class="elegant-shape-outer-animated absolute right-[15%] md:right-[20%] top-[10%] md:top-[15%]"
                 style="--initial-rotate: 20deg; --final-rotate: 20deg; animation-delay: 0.6s;">
                <div class="elegant-shape-inner-animated relative" style="width: 200px; height: 60px;">
                    <div class="absolute inset-0 rounded-full bg-gradient-to-r to-transparent from-amber-500/[0.15] backdrop-blur-[2px] border-2 border-white/[0.15] shadow-[0_8px_32px_0_rgba(255,255,255,0.1),0_0_40px_3px_rgba(245,158,11,0.4)] after:absolute after:inset-0 after:rounded-full after:bg-[radial-gradient(circle_at_50%_50%,rgba(255,255,255,0.2),transparent_70%)]">
                    </div>
                </div>
            </div>
            <div id="elegant-shape-5" class="elegant-shape-outer-animated absolute left-[20%] md:left-[25%] top-[5%] md:top-[10%]"
                 style="--initial-rotate: -25deg; --final-rotate: -25deg; animation-delay: 0.7s;">
                <div class="elegant-shape-inner-animated relative" style="width: 150px; height: 40px;">
                    <div class="absolute inset-0 rounded-full bg-gradient-to-r to-transparent from-cyan-500/[0.15] backdrop-blur-[2px] border-2 border-white/[0.15] shadow-[0_8px_32px_0_rgba(255,255,255,0.1),0_0_40px_3px_rgba(6,182,212,0.4)] after:absolute after:inset-0 after:rounded-full after:bg-[radial-gradient(circle_at_50%_50%,rgba(255,255,255,0.2),transparent_70%)]">
                    </div>
                </div>
            </div>
        </div>

        <div class="relative z-10 container mx-auto px-4 md:px-6">
            <div id="main-text-content" class="max-w-3xl mx-auto text-center">
                <!-- Left split content container -->
                <div id="content-left" class="inline-flex items-center gap-2 px-3 py-1 rounded-full bg-white/[0.03] border border-white/[0.08] mb-8 md:mb-12">
                    <div class="h-2 w-2 rounded-full bg-rose-500/80"></div>
                    <span class="text-sm text-white/60 tracking-wide">
                        AI-Powered Document Analysis
                    </span>
                </div>

                <!-- Middle split content container -->
                <div id="content-middle">
                    <h1 id="ask-docs-title" class="text-4xl sm:text-6xl md:text-8xl font-bold mb-6 md:mb-8 tracking-tight">
                        <span class="bg-clip-text text-transparent bg-gradient-to-b from-white to-white/80">
                            Ask Your
                        </span>
                        <br />
                        <span class="bg-clip-text text-transparent bg-gradient-to-r from-indigo-300 via-white/90 to-rose-300">
                            Documents
                        </span>
                    </h1>
                </div>

                <!-- Right split content container -->
                <div id="content-right">
                    <p class="text-base sm:text-lg md:text-xl text-white/40 mb-8 leading-relaxed font-light tracking-wide max-w-xl mx-auto px-4">
                        Simply upload your files, ask any question, and let our intelligent AI provide you with precise answers and summaries in seconds.
                    </p>
                </div>

                <!-- Upload Component (initially hidden) -->
                <div id="upload-component" class="mt-10 max-w-md mx-auto"> <!-- style="display: none; opacity: 0;" is handled by CSS now -->
                    <div id="file-upload-container">
                        <div id="drop-zone"
                            class="relative rounded-2xl p-8 text-center cursor-pointer bg-zinc-900 border border-zinc-700 shadow-sm hover:shadow-md backdrop-blur group"
                            style="transition: border-color 0.2s ease, transform 0.2s ease;">
                            <div class="flex flex-col items-center gap-5">
                                <div id="upload-icon-container" class="relative" style="transition: transform 1.5s ease-in-out;">
                                    <div id="upload-icon-glow"
                                        class="absolute -inset-4 bg-blue-400/10 rounded-full blur-md"
                                        style="display: none; transition: opacity 2s ease-in-out, transform 2s ease-in-out;"></div>
                                    <svg id="upload-cloud-icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"
                                        class="w-16 h-16 md:w-20 md:h-20 drop-shadow-sm text-zinc-300 group-hover:text-blue-500 transition-colors duration-300">
                                        <path d="M4 14.899A7 7 0 1 1 15.71 8h1.79a4.5 4.5 0 0 1 2.5 8.242"/>
                                        <path d="M12 12v9"/><path d="m16 16-4-4-4 4"/>
                                    </svg>
                                </div>

                                <div class="space-y-2">
                                    <h3 id="drop-zone-title" class="text-xl md:text-2xl font-semibold text-zinc-100">
                                        Upload your files
                                    </h3>
                                    <p id="drop-zone-subtitle" class="text-zinc-300 md:text-lg max-w-md mx-auto">
                                        Drag & drop files here, or <span class="text-blue-500 font-medium">browse</span>
                                    </p>
                                    <p class="text-sm text-zinc-400">
                                        Supports images, doc, pdf, and more
                                    </p>
                                </div>

                                <input id="file-input-element" type="file" multiple hidden
                                    accept="image/*,application/pdf,video/*,audio/*,text/*,application/zip" />
                            </div>
                        </div>

                        <div id="uploaded-files-section" class="mt-8" style="opacity: 0; transition: opacity 0.3s ease;">
                            <!-- Document contains heading - hidden until Ask button is clicked -->
                            <div id="document-contains-heading" class="mb-12 w-full flex justify-center relative" style="display: none;">
                                <h4 class="text-4xl sm:text-5xl md:text-6xl font-bold tracking-tight whitespace-nowrap relative z-10">
                                    <span class="bg-clip-text text-transparent bg-gradient-to-r from-indigo-300 via-white/90 to-rose-300">Document contains</span>
                                </h4>
                            </div>
                            <div id="files-header" class="flex justify-between items-center mb-3 px-2" style="display: none;">
                                <h3 id="uploaded-files-title" class="font-semibold text-lg md:text-xl text-zinc-200">
                                    Uploaded files (0)
                                </h3>
                                <div class="flex items-center space-x-3">
                                    <button id="ask-button"
                                        class="interactive-hover-btn text-sm font-medium text-white bg-black/80 border border-white/10 hover:bg-indigo-600 hover:border-indigo-500 transition-all duration-300 rounded-full px-4 py-2"
                                        style="display: none;">
                                        <span class="btn-text">Ask</span>
                                        <div class="hover-content text-white">
                                            <span>Ask</span>
                                            <svg class="arrow-right" viewBox="0 0 24 24">
                                                <path d="M5 12h14"></path>
                                                <path d="M12 5l7 7-7 7"></path>
                                            </svg>
                                        </div>
                                        <div class="btn-bg-dot"></div>
                                    </button>
                                </div>
                            </div>

                            <div id="files-list-container" class="relative min-h-[190px]">
                                <!-- Files will be paginated here -->
                            </div>

                            <div id="files-pagination" class="pagination mt-4" style="display: none;">
                                <!-- Pagination buttons will be generated here -->
                            </div>

                            
                            
                            <!-- Settings Panel - Initially hidden, shown after clicking Ask -->
                            <div id="settings-panel" class="settings-panel p-4 w-full" style="display: none;">
                                <div class="flex flex-col gap-3 mt-2">
                                    <!-- Text Only Option -->
                                    <div class="settings-card rounded-lg p-4 flex items-center justify-between">
                                        <span class="text-white text-base font-medium">Text</span>
                                        <label class="liquid-toggle-switch default">
                                            <input type="checkbox" class="liquid-toggle-input" checked id="toggle-text-only">
                                            <svg viewBox="0 0 52 32" filter="url(#goo)" class="liquid-toggle-svg">
                                                <circle class="liquid-toggle-circle liquid-toggle-circle-1" cx="16" cy="16" r="10" />
                                                <circle class="liquid-toggle-circle liquid-toggle-circle-2" cx="36" cy="16" r="10" />
                                                <circle class="liquid-toggle-drop-circle" cx="35" cy="-1" r="2.5" />
                                            </svg>
                                        </label>
                                    </div>

                                    <!-- OCR Option -->
                                    <div class="settings-card rounded-lg p-4 flex items-center justify-between">
                                        <span class="text-white text-base font-medium">OCR</span>
                                        <label class="liquid-toggle-switch default">
                                            <input type="checkbox" class="liquid-toggle-input" id="toggle-ocr">
                                            <svg viewBox="0 0 52 32" filter="url(#goo)" class="liquid-toggle-svg">
                                                <circle class="liquid-toggle-circle liquid-toggle-circle-1" cx="16" cy="16" r="10" />
                                                <circle class="liquid-toggle-circle liquid-toggle-circle-2" cx="36" cy="16" r="10" />
                                                <circle class="liquid-toggle-drop-circle" cx="35" cy="-1" r="2.5" />
                                            </svg>
                                        </label>
                                    </div>

                                    <!-- Handwritten Text Option -->
                                    <div class="settings-card rounded-lg p-4 flex items-center justify-between">
                                        <span class="text-white text-base font-medium">Handwritten</span>
                                        <label class="liquid-toggle-switch default">
                                            <input type="checkbox" class="liquid-toggle-input" id="toggle-handwritten">
                                            <svg viewBox="0 0 52 32" filter="url(#goo)" class="liquid-toggle-svg">
                                                <circle class="liquid-toggle-circle liquid-toggle-circle-1" cx="16" cy="16" r="10" />
                                                <circle class="liquid-toggle-circle liquid-toggle-circle-2" cx="36" cy="16" r="10" />
                                                <circle class="liquid-toggle-drop-circle" cx="35" cy="-1" r="2.5" />
                                            </svg>
                                        </label>
                                    </div>

                                    <!-- Images Option -->
                                    <div class="settings-card rounded-lg p-4 flex items-center justify-between">
                                        <span class="text-white text-base font-medium">Images</span>
                                        <label class="liquid-toggle-switch default">
                                            <input type="checkbox" class="liquid-toggle-input" id="toggle-images">
                                            <svg viewBox="0 0 52 32" filter="url(#goo)" class="liquid-toggle-svg">
                                                <circle class="liquid-toggle-circle liquid-toggle-circle-1" cx="16" cy="16" r="10" />
                                                <circle class="liquid-toggle-circle liquid-toggle-circle-2" cx="36" cy="16" r="10" />
                                                <circle class="liquid-toggle-drop-circle" cx="35" cy="-1" r="2.5" />
                                            </svg>
                                        </label>
                                    </div>
                                </div>
                                <!-- Process button removed, functionality moved to Ask button -->
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>

        <div class="absolute inset-0 bg-gradient-to-t from-[#030303] via-transparent to-[#030303]/80 pointer-events-none"></div>
    </div>

    <div id="hover-indicator">Let's ask</div>

    <script>
        // === START: Animated Background for 'Document Contains' Heading ===
        let bgAnimCanvas, bgAnimCtx, bgAnimAnimationFrameId;
        let bgAnimNumSquaresX, bgAnimNumSquaresY;
        let bgAnimGridOffset = { x: 0, y: 0 };
        const bgAnimConfig = { direction: "diagonal", speed: 0.2, borderColor: "rgba(100, 100, 100, 0.15)", squareSize: 25, canvasBackgroundColor: "rgba(0, 0, 0, 0)", canvasInitialWidth: null, canvasInitialHeight: null, };
        function bgAnimResizeCanvas() { if (!bgAnimCanvas || !document.body.contains(bgAnimCanvas)) return; const dpr = window.devicePixelRatio || 1; const newWidth = window.innerWidth; const newHeight = window.innerHeight; if (newWidth === 0 || newHeight === 0) return; bgAnimCanvas.width = newWidth * dpr; bgAnimCanvas.height = newHeight * dpr; bgAnimCanvas.style.width = `${newWidth}px`; bgAnimCanvas.style.height = `${newHeight}px`; if (bgAnimCtx) { bgAnimCtx.scale(dpr, dpr); } bgAnimNumSquaresX = Math.ceil(newWidth / bgAnimConfig.squareSize) + 2; bgAnimNumSquaresY = Math.ceil(newHeight / bgAnimConfig.squareSize) + 2; if (bgAnimCtx && bgAnimCanvas.style.display !== 'none') { bgAnimDrawGrid(); } }
        function bgAnimDrawGrid() { if (!bgAnimCtx || !bgAnimCanvas) return; const currentCanvasWidth = bgAnimCanvas.width / (window.devicePixelRatio || 1); const currentCanvasHeight = bgAnimCanvas.height / (window.devicePixelRatio || 1); bgAnimCtx.clearRect(0, 0, currentCanvasWidth, currentCanvasHeight); bgAnimCtx.lineWidth = 0.5; bgAnimCtx.strokeStyle = bgAnimConfig.borderColor; const offsetX = bgAnimGridOffset.x % bgAnimConfig.squareSize; const offsetY = bgAnimGridOffset.y % bgAnimConfig.squareSize; for (let i = -1; i < bgAnimNumSquaresX; i++) { for (let j = -1; j < bgAnimNumSquaresY; j++) { const squareX = (i * bgAnimConfig.squareSize) - offsetX; const squareY = (j * bgAnimConfig.squareSize) - offsetY; bgAnimCtx.strokeRect(squareX, squareY, bgAnimConfig.squareSize, bgAnimConfig.squareSize); } } }
        function bgAnimUpdateAnimation() { if (!bgAnimCanvas || !document.body.contains(bgAnimCanvas) || bgAnimCanvas.style.display === 'none') { if (bgAnimAnimationFrameId) { cancelAnimationFrame(bgAnimAnimationFrameId); bgAnimAnimationFrameId = null; } return; } const effectiveSpeed = bgAnimConfig.speed; switch (bgAnimConfig.direction) { case "right": bgAnimGridOffset.x -= effectiveSpeed; break; case "left": bgAnimGridOffset.x += effectiveSpeed; break; case "up": bgAnimGridOffset.y += effectiveSpeed; break; case "down": bgAnimGridOffset.y -= effectiveSpeed; break; case "diagonal": bgAnimGridOffset.x -= effectiveSpeed; bgAnimGridOffset.y -= effectiveSpeed; break; } bgAnimGridOffset.x = (bgAnimGridOffset.x % bgAnimConfig.squareSize + bgAnimConfig.squareSize) % bgAnimConfig.squareSize; bgAnimGridOffset.y = (bgAnimGridOffset.y % bgAnimConfig.squareSize + bgAnimConfig.squareSize) % bgAnimConfig.squareSize; bgAnimDrawGrid(); bgAnimAnimationFrameId = requestAnimationFrame(bgAnimUpdateAnimation); }
        function initAnimatedBackground() { bgAnimCanvas = document.getElementById('animated-background-canvas'); if (!bgAnimCanvas) { console.error("Animated background canvas not found!"); return; } bgAnimCtx = bgAnimCanvas.getContext('2d'); if (!bgAnimCtx) { console.error("Failed to get 2D context for animated background canvas!"); return; } bgAnimCanvas.style.display = 'block'; bgAnimResizeCanvas(); if (bgAnimAnimationFrameId) { cancelAnimationFrame(bgAnimAnimationFrameId); } bgAnimAnimationFrameId = requestAnimationFrame(bgAnimUpdateAnimation); window.addEventListener('resize', bgAnimResizeCanvas); }
        function stopAnimatedBackground() { if (bgAnimAnimationFrameId) { cancelAnimationFrame(bgAnimAnimationFrameId); bgAnimAnimationFrameId = null; } if (bgAnimCanvas) { bgAnimCanvas.style.display = 'none'; if(bgAnimCtx) { const dpr = window.devicePixelRatio || 1; bgAnimCtx.clearRect(0, 0, bgAnimCanvas.width / dpr, bgAnimCanvas.height / dpr);}} window.removeEventListener('resize', bgAnimResizeCanvas); }
        // === END: Animated Background ===

        document.addEventListener('DOMContentLoaded', () => {
            const askDocsTitle = document.getElementById('ask-docs-title');
            const hoverIndicator = document.getElementById('hover-indicator');
            const uploadComponent = document.getElementById('upload-component');
            const mainTextContent = document.getElementById('main-text-content');
            const contentLeft = document.getElementById('content-left');
            const contentMiddle = document.getElementById('content-middle');
            const contentRight = document.getElementById('content-right');
            const shape1 = document.getElementById('elegant-shape-1');
            const shape2 = document.getElementById('elegant-shape-2');
            const shape3 = document.getElementById('elegant-shape-3');
            const shape4 = document.getElementById('elegant-shape-4');
            const shape5 = document.getElementById('elegant-shape-5');

            const innerShape1 = shape1 ? shape1.querySelector('.elegant-shape-inner-animated') : null;
            const innerShape2 = shape2 ? shape2.querySelector('.elegant-shape-inner-animated') : null;
            const innerShape3 = shape3 ? shape3.querySelector('.elegant-shape-inner-animated') : null;
            const innerShape4 = shape4 ? shape4.querySelector('.elegant-shape-inner-animated') : null;
            const innerShape5 = shape5 ? shape5.querySelector('.elegant-shape-inner-animated') : null;

            if (askDocsTitle && hoverIndicator) {
                 askDocsTitle.addEventListener('mouseenter', () => { if (!contentMiddle || !contentMiddle.classList.contains('animate-content-middle')) { hoverIndicator.style.display = 'block'; } });
                 askDocsTitle.addEventListener('mousemove', (event) => { if (!contentMiddle || !contentMiddle.classList.contains('animate-content-middle')) { hoverIndicator.style.left = event.clientX + 'px'; hoverIndicator.style.top = event.clientY + 'px'; } });
                 askDocsTitle.addEventListener('mouseleave', () => { hoverIndicator.style.display = 'none'; });
            }

            if (askDocsTitle) {
                askDocsTitle.addEventListener('click', () => {
                    if (hoverIndicator) hoverIndicator.style.display = 'none';
                    if (shape1) shape1.classList.add('animate-shape-move-left'); 
                    if (shape3) shape3.classList.add('animate-shape-move-left');
                    if (shape5) shape5.classList.add('animate-shape-move-left');
                    if (shape2) shape2.classList.add('animate-shape-move-right');
                    if (shape4) shape4.classList.add('animate-shape-move-right');
                    if (innerShape1) innerShape1.classList.remove('elegant-shape-inner-animated'); 
                    if (innerShape2) innerShape2.classList.remove('elegant-shape-inner-animated');
                    if (innerShape3) innerShape3.classList.remove('elegant-shape-inner-animated');
                    if (innerShape4) innerShape4.classList.remove('elegant-shape-inner-animated');
                    if (innerShape5) innerShape5.classList.remove('elegant-shape-inner-animated');
                    if (contentLeft) contentLeft.classList.add('animate-content-left');
                    if (contentMiddle) contentMiddle.classList.add('animate-content-middle');
                    if (contentRight) contentRight.classList.add('animate-content-right');
                    setTimeout(() => {
                        if (uploadComponent) {
                            if (contentLeft) contentLeft.style.display = 'none';
                            if (contentMiddle) contentMiddle.style.display = 'none';
                            if (contentRight) contentRight.style.display = 'none';
                            uploadComponent.style.display = 'block';
                            void uploadComponent.offsetWidth; 
                            uploadComponent.style.opacity = '1';
                            initUploadFunctionality();
                        }
                    }, 1500);
                    askDocsTitle.style.pointerEvents = 'none';
                }, { once: true });
            }

            function initUploadFunctionality() {
                const ICONS = { 
                    FileIcon: `<svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" class="w-16 h-16 md:w-20 md:h-20 text-zinc-500"><path d="M14.5 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V7.5L14.5 2z"/><polyline points="14 2 14 8 20 8"/></svg>`,
                    FileIconSmall: `<svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" class="w-5 h-5 flex-shrink-0 text-blue-400"><path d="M14.5 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V7.5L14.5 2z"/><polyline points="14 2 14 8 20 8"/></svg>`,
                    Trash2Icon: `<svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" class="w-4 h-4 cursor-pointer text-zinc-500 hover:text-red-400 transition-colors duration-200"><path d="M3 6h18"/><path d="M19 6v14c0 1-1 2-2 2H7c-1 0-2-1-2-2V6"/><path d="M8 6V4c0-1 1-2 2-2h4c1 0 2 1 2 2v2"/><line x1="10" y1="11" x2="10" y2="17"/><line x1="14" y1="11" x2="14" y2="17"/></svg>`,
                    LoaderIcon: `<svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" class="w-4 h-4 animate-spin-custom text-blue-500"><line x1="12" y1="2" x2="12" y2="6"/><line x1="12" y1="18" x2="12" y2="22"/><line x1="4.93" y1="4.93" x2="7.76" y2="7.76"/><line x1="16.24" y1="16.24" x2="19.07" y2="19.07"/><line x1="2" y1="12" x2="6" y2="12"/><line x1="18" y1="12" x2="22" y2="12"/><line x1="4.93" y1="19.07" x2="7.76" y2="16.24"/><line x1="16.24" y1="7.76" x2="19.07" y2="4.93"/></svg>`,
                    CheckCircleIcon: `<svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" class="w-5 h-5 text-emerald-500"><path d="M22 11.08V12a10 10 0 1 1-5.93-9.14"/><polyline points="22 4 12 14.01 9 11.01"/></svg>`
                };
                const dropZone = document.getElementById('drop-zone');
                const fileInputElement = document.getElementById('file-input-element');
                const uploadIconContainer = document.getElementById('upload-icon-container');
                const uploadIconGlow = document.getElementById('upload-icon-glow');
                const uploadCloudIcon = document.getElementById('upload-cloud-icon');
                const dropZoneTitle = document.getElementById('drop-zone-title');
                const dropZoneSubtitle = document.getElementById('drop-zone-subtitle');
                const uploadedFilesSection = document.getElementById('uploaded-files-section');
                const filesHeader = document.getElementById('files-header');
                const uploadedFilesTitle = document.getElementById('uploaded-files-title');
                const askButton = document.getElementById('ask-button');
                const filesListContainer = document.getElementById('files-list-container');
                const paginationContainer = document.getElementById('files-pagination');
                const settingsPanel = document.getElementById('settings-panel');
                const documentContainsHeading = document.getElementById('document-contains-heading');

                let files = []; 
                let isDragging = false;
                let currentPage = 1;
                const filesPerPage = 2; 
                let inSettingsMode = false;
                let isCurrentlyProcessingFull = false; 

                function updateAskButtonState() {
                    if (!askButton) return;
                    const successfullyStagedFiles = files.filter(f => f.statusMessage === "Ready" && f.stagingId);
                    
                    const btnTextSpan = askButton.querySelector('.btn-text');
                    const hoverTextSpan = askButton.querySelector('.hover-content span');

                    if (isCurrentlyProcessingFull) {
                        askButton.disabled = true;
                        if (btnTextSpan) btnTextSpan.textContent = "Processing...";
                        if (hoverTextSpan) hoverTextSpan.textContent = "Processing...";
                        askButton.classList.add('animate-pulse');
                        askButton.style.display = 'block'; 
                    } else if (inSettingsMode) { 
                        askButton.disabled = successfullyStagedFiles.length === 0;
                        if (btnTextSpan) btnTextSpan.textContent = "Process";
                        if (hoverTextSpan) hoverTextSpan.textContent = "Process";
                        askButton.classList.remove('animate-pulse');
                        askButton.style.display = 'block'; 
                    } else { 
                        askButton.disabled = successfullyStagedFiles.length === 0;
                        if (btnTextSpan) btnTextSpan.textContent = "Ask";
                        if (hoverTextSpan) hoverTextSpan.textContent = "Ask";
                        askButton.classList.remove('animate-pulse');
                        askButton.style.display = successfullyStagedFiles.length > 0 ? 'block' : 'none';
                    }
                }
                
                const formatFileSize = (bytes) => {
                    if (!bytes) return "0 Bytes"; const k = 1024; const sizes = ["Bytes", "KB", "MB", "GB"]; const i = Math.floor(Math.log(bytes) / Math.log(k)); return `${(bytes / Math.pow(k, i)).toFixed(2)} ${sizes[i]}`;
                };

                async function stageFileOnServer(fileWrapper) {
                    fileWrapper.progress = 10; 
                    fileWrapper.statusMessage = "Uploading...";
                    renderFiles(); 

                    const formData = new FormData();
                    formData.append('file', fileWrapper.file); 
                    formData.append('originalFilename', fileWrapper.name);

                    try {
                        const response = await fetch('/stage_file', { method: 'POST', body: formData });
                        const result = await response.json();

                        if (!response.ok) {
                            console.error(`Staging failed for ${fileWrapper.name}:`, result.error);
                            fileWrapper.progress = 0;
                            fileWrapper.statusMessage = "Upload Failed";
                        } else {
                            console.log(`Staging success for ${fileWrapper.name}:`, result);
                            fileWrapper.progress = 100;
                            fileWrapper.statusMessage = "Ready"; 
                            fileWrapper.stagingId = result.staging_id; 
                        }
                    } catch (error) {
                        console.error(`Error staging file ${fileWrapper.name}:`, error);
                        fileWrapper.progress = 0;
                        fileWrapper.statusMessage = "Network Error";
                    }
                    renderFiles(); 
                }

                const handleFiles = (fileListFromInput) => {
                    const newFileWrappers = Array.from(fileListFromInput).map((rawFile) => ({
                        id: `file-${Date.now()}-${Math.random().toString(16).slice(2)}`,
                        name: rawFile.name,
                        size: rawFile.size,
                        type: rawFile.type,
                        file: rawFile, 
                        progress: 0,
                        statusMessage: "Pending Stage",
                        stagingId: null 
                    }));
                    files = [...files, ...newFileWrappers];
                    renderFiles(); 
                    newFileWrappers.forEach(fw => stageFileOnServer(fw));
                };

                const removeFile = (fileId) => {
                    // Future: If file is already staged, might want to send a request to backend to delete staged file
                    // For now, just removes from frontend `files` array
                    const fileToRemove = files.find(f => f.id === fileId);
                    if (fileToRemove && fileToRemove.stagingId) {
                        console.log(`File ${fileToRemove.name} (stagingId: ${fileToRemove.stagingId}) removed from UI. Consider backend cleanup for staged file.`);
                        // Example:
                        // fetch(`/cleanup_staged_file?staging_id=${fileToRemove.stagingId}`, {method: 'POST'})
                        //   .then(res => console.log(`Cleanup request for ${fileToRemove.stagingId} sent.`))
                        //   .catch(err => console.error(`Cleanup error for ${fileToRemove.stagingId}:`, err));
                    }
                    files = files.filter((f) => f.id !== fileId);
                    renderFiles();
                };
                
                const updateDropZoneAppearance = () => {
                    if (!dropZone || !uploadIconContainer || !uploadIconGlow || !uploadCloudIcon || !dropZoneTitle || !dropZoneSubtitle) return;
                    dropZone.style.borderColor = isDragging ? '#3b82f6' : '#3f3f46';
                    dropZone.style.transform = isDragging ? 'scale(1.02)' : 'scale(1)';
                    dropZone.classList.toggle('ring-4', isDragging);
                    dropZone.classList.toggle('ring-blue-400/30', isDragging);
                    if(uploadIconContainer) uploadIconContainer.style.animation = isDragging ? 'bounce-y 1.5s ease-in-out infinite' : 'none';
                    if(uploadIconGlow) uploadIconGlow.style.display = isDragging ? 'block' : 'none';
                    if(uploadIconGlow) uploadIconGlow.style.animation = isDragging ? 'pulse-glow-effect 2s ease-in-out infinite' : 'none';
                    if(uploadCloudIcon) uploadCloudIcon.classList.toggle('text-blue-500', isDragging);
                    if(uploadCloudIcon) uploadCloudIcon.classList.toggle('text-zinc-300', !isDragging);
                    if (isDragging) {
                        if(dropZoneTitle) dropZoneTitle.textContent = "Drop files here";
                        if(dropZoneSubtitle) dropZoneSubtitle.innerHTML = `<span class="font-medium text-blue-500">Release to upload</span>`;
                    } else {
                        if(dropZoneTitle) dropZoneTitle.textContent = files.length ? "Add more files" : "Upload your files";
                        if(dropZoneSubtitle) dropZoneSubtitle.innerHTML = `Drag & drop files here, or <span class="text-blue-500 font-medium">browse</span>`;
                    }
                };

                const renderFiles = () => {
                    if (files.length > 0) {
                        if(uploadedFilesSection) uploadedFilesSection.style.opacity = '1';
                        if(filesHeader) filesHeader.style.display = 'flex';
                        if(uploadedFilesTitle) uploadedFilesTitle.textContent = `Uploaded files (${files.length})`;
                    } else {
                        if(uploadedFilesSection) uploadedFilesSection.style.opacity = '0';
                        setTimeout(() => { if(files.length === 0 && filesHeader && uploadedFilesTitle) {
                            filesHeader.style.display = 'none';
                            uploadedFilesTitle.textContent = `Uploaded files (0)`;
                        } }, 300); // Delay to allow opacity transition
                    }

                    if (!filesListContainer) return; 
                    filesListContainer.innerHTML = ''; // Clear previous items
                    const totalPages = Math.ceil(files.length / filesPerPage);
                    if (currentPage > totalPages && totalPages > 0) currentPage = totalPages;
                    else if (totalPages === 0) currentPage = 1;


                    for (let pageNum = 1; pageNum <= totalPages; pageNum++) {
                        const startIdx = (pageNum - 1) * filesPerPage;
                        const endIdx = Math.min(startIdx + filesPerPage, files.length);
                        const pageFiles = files.slice(startIdx, endIdx);
                        const pageContainer = document.createElement('div');
                        pageContainer.id = `files-page-${pageNum}`;
                        pageContainer.className = `files-page ${pageNum === currentPage ? 'active' : ''}`;
                        
                        pageFiles.forEach(file => {
                            const isReadyForProcessing = file.statusMessage === "Ready";
                            const isFullyProcessed = file.statusMessage === "Processed";
                            const isUploading = file.statusMessage === "Uploading..." || file.statusMessage === "Pending Stage";
                            const hasFailed = file.statusMessage === "Upload Failed" || file.statusMessage === "Network Error" || file.statusMessage === "Processing Failed";

                            const displayProgress = (isReadyForProcessing || isFullyProcessed) ? 100 : file.progress;
                            let progressBarColorClass = 'bg-blue-500';
                            if (isReadyForProcessing || isFullyProcessed) progressBarColorClass = 'bg-emerald-500';
                            else if (hasFailed) progressBarColorClass = 'bg-red-500';

                            let actionIconHtml = ICONS.LoaderIcon;
                            if (isUploading) actionIconHtml = ICONS.LoaderIcon; // Explicitly
                            else if (isFullyProcessed) actionIconHtml = ICONS.CheckCircleIcon; 
                            else if (isReadyForProcessing || hasFailed) { 
                                actionIconHtml = `<span class="remove-file-button" data-fileid="${file.id}" aria-label="Remove file">${ICONS.Trash2Icon}</span>`;
                            }
                            
                            let thumbnailHtml = ICONS.FileIcon; 
                            if (file.file && file.type.startsWith("image/")) {
                                thumbnailHtml = `<img src="${URL.createObjectURL(file.file)}" alt="${file.name}" class="w-16 h-16 md:w-20 md:h-20 rounded-lg object-cover border border-zinc-700 shadow-sm">`;
                            } else if (file.file && file.type.startsWith("video/")) {
                                thumbnailHtml = `<video src="${URL.createObjectURL(file.file)}" class="w-16 h-16 md:w-20 md:h-20 rounded-lg object-cover border border-zinc-700 shadow-sm" controls="false" muted loop playsinline preload="metadata"></video>`;
                            }

                            const fileItem = document.createElement('div'); 
                            fileItem.id = `file-item-${file.id}`;
                            fileItem.className = "file-item px-4 py-4 flex items-start gap-4 rounded-xl bg-zinc-800 shadow hover:shadow-md transition-all duration-200 mb-3";
                            fileItem.innerHTML = `
                                <div class="relative flex-shrink-0">${thumbnailHtml}
                                    ${isFullyProcessed ? `<div class="absolute -right-2 -bottom-2 bg-zinc-800 rounded-full shadow-sm" style="opacity:1; transform: scale(1); animation: fadeInScale 0.3s 0.1s forwards;">${ICONS.CheckCircleIcon}</div>` : ''}
                                </div>
                                <div class="flex-1 min-w-0">
                                    <div class="flex flex-col gap-1 w-full">
                                        <div class="flex items-center gap-2 min-w-0">${ICONS.FileIconSmall}<h4 class="font-medium text-base md:text-lg truncate text-zinc-200" title="${file.name}">${file.name}</h4></div>
                                        <div class="flex items-center justify-between gap-3 text-sm text-zinc-400">
                                            <span class="text-xs md:text-sm">${formatFileSize(file.size)}</span>
                                            <span class="flex items-center gap-1.5">
                                                <span class="font-medium">${file.statusMessage}</span>
                                                ${actionIconHtml}
                                            </span>
                                        </div>
                                    </div>
                                    <div class="w-full h-2 bg-zinc-700 rounded-full overflow-hidden mt-3">
                                        <div class="h-full rounded-full shadow-inner ${progressBarColorClass}" style="width: ${displayProgress}%; transition: width 0.2s ease-out, background-color 0.2s ease-out;"></div>
                                    </div>
                                </div>`;
                            pageContainer.appendChild(fileItem);
                        });
                        filesListContainer.appendChild(pageContainer);
                    }

                    if (paginationContainer) {
                        if (totalPages > 1) {
                            paginationContainer.style.display = 'flex'; paginationContainer.innerHTML = '';
                            const prevButton = document.createElement('button'); prevButton.className = 'pagination-btn'; prevButton.innerHTML = `<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><polyline points="15 18 9 12 15 6"></polyline></svg>`; prevButton.disabled = currentPage === 1; prevButton.addEventListener('click', () => { if (currentPage > 1) { currentPage--; renderFiles(); }}); paginationContainer.appendChild(prevButton);
                            for (let i = 1; i <= totalPages; i++) { const pageButton = document.createElement('button'); pageButton.className = `pagination-btn ${i === currentPage ? 'active' : ''}`; pageButton.textContent = i; pageButton.addEventListener('click', () => { currentPage = i; renderFiles(); }); paginationContainer.appendChild(pageButton); }
                            const nextButton = document.createElement('button'); nextButton.className = 'pagination-btn'; nextButton.innerHTML = `<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><polyline points="9 18 15 12 9 6"></polyline></svg>`; nextButton.disabled = currentPage === totalPages; nextButton.addEventListener('click', () => { if (currentPage < totalPages) { currentPage++; renderFiles(); }}); paginationContainer.appendChild(nextButton);
                        } else { paginationContainer.style.display = 'none'; }
                    }
                    document.querySelectorAll('.remove-file-button').forEach(button => button.addEventListener('click', (e) => { e.stopPropagation(); removeFile(button.dataset.fileid); }));
                    updateDropZoneAppearance();
                    updateAskButtonState();
                };

                if (dropZone) { 
                    dropZone.addEventListener('dragover', (e) => { e.preventDefault(); isDragging = true; updateDropZoneAppearance(); });
                    dropZone.addEventListener('dragleave', () => { isDragging = false; updateDropZoneAppearance(); });
                    dropZone.addEventListener('drop', (e) => { e.preventDefault(); isDragging = false; updateDropZoneAppearance(); handleFiles(e.dataTransfer.files); });
                    if (fileInputElement) dropZone.addEventListener('click', () => fileInputElement.click());
                    dropZone.addEventListener('mouseenter', () => { if (!isDragging && dropZone) dropZone.style.transform = 'scale(1.01)'; });
                    dropZone.addEventListener('mouseleave', () => { if (!isDragging && dropZone) dropZone.style.transform = 'scale(1)'; });
                }
                if (fileInputElement) fileInputElement.addEventListener('change', (e) => { if (e.target.files) handleFiles(e.target.files); e.target.value = null; });

                async function triggerFullProcessing(stagedFileWrappers, processingOptions) {
                    if (isCurrentlyProcessingFull) { console.warn("Full processing already in progress."); return; }
                    isCurrentlyProcessingFull = true;
                    stagedFileWrappers.forEach(fw => { if(fw.stagingId) fw.statusMessage = "Queued..."; });
                    renderFiles(); 

                    const stagingIds = stagedFileWrappers.map(fw => fw.stagingId).filter(id => id != null);
                    if (stagingIds.length === 0) {
                        alert("No files are ready (staged) for processing.");
                        isCurrentlyProcessingFull = false;
                        renderFiles(); 
                        return;
                    }

                    try {
                        const response = await fetch('/process_staged_files', { method: 'POST', headers: {'Content-Type': 'application/json'}, body: JSON.stringify({ staging_ids: stagingIds, options: processingOptions }) });
                        const result = await response.json();
                        isCurrentlyProcessingFull = false; 

                        if (!response.ok) {
                            console.error('Full backend processing failed:', result);
                            alert(`Error during full processing: ${result.error || response.statusText}`);
                            stagedFileWrappers.forEach(fw => { if(fw.stagingId) fw.statusMessage = "Processing Failed"; });
                        } else {
                            console.log('Full backend processing successful:', result);
                            const successfullyProcessedDocsForChat = [];
                            if (result.processed_documents && Array.isArray(result.processed_documents)) {
                                result.processed_documents.forEach(processedDoc => {
                                    const frontendFile = files.find(f => f.stagingId === processedDoc.staging_id || (f.name === processedDoc.filename && f.stagingId) ); // Match by stagingId first, then filename if stagingId was somehow lost but name matches
                                    if (frontendFile) {
                                        frontendFile.statusMessage = "Processed";
                                        frontendFile.collectionName = processedDoc.collection_name; 
                                    }
                                    if (processedDoc.filename) { 
                                       successfullyProcessedDocsForChat.push(processedDoc.filename);
                                    }
                                });
                            }
                            if (successfullyProcessedDocsForChat.length > 0) {
                                localStorage.setItem('activeDocs', JSON.stringify(successfullyProcessedDocsForChat));
                                // Redirect immediately after setting localStorage
                                window.location.href = '/chatui';
                            } else {
                                alert("Backend processing finished, but no documents were confirmed as fully processed.");
                            }
                        }
                    } catch (error) {
                        console.error('Network error during full processing:', error);
                        alert('A network error occurred during full processing.');
                        isCurrentlyProcessingFull = false;
                        stagedFileWrappers.forEach(fw => { if(fw.stagingId) fw.statusMessage = "Network Error (Proc)"; });
                    }
                    renderFiles(); // This will update statuses, including "Processed"
                }

                if (askButton) {
                    updateAskButtonState(); 
                    askButton.addEventListener('click', () => {
                        const btnBgDot = askButton.querySelector('.btn-bg-dot'); if (btnBgDot) { /* ... */ }
                        if (inSettingsMode) { 
                            const readyFiles = files.filter(f => f.statusMessage === "Ready" && f.stagingId);
                            if (readyFiles.length === 0) { alert("Please upload and wait for files to be ready for processing."); return; }
                            const opts = { 
                                textOnly: document.getElementById('toggle-text-only')?.checked ?? true,
                                ocr: document.getElementById('toggle-ocr')?.checked ?? false,
                                handwritten: document.getElementById('toggle-handwritten')?.checked ?? false,
                                images: document.getElementById('toggle-images')?.checked ?? false
                            };
                            triggerFullProcessing(readyFiles, opts);
                        } else { 
                            showSettingsPanel(); 
                        }
                    });
                }

                function showSettingsPanel() { 
                    inSettingsMode = true;
                    if (dropZone) dropZone.style.transition = 'opacity 0.3s ease'; 
                    if (filesListContainer) filesListContainer.style.transition = 'opacity 0.3s ease';
                    if (paginationContainer) paginationContainer.style.transition = 'opacity 0.3s ease';
                    if (documentContainsHeading) { documentContainsHeading.style.display = 'flex'; initAnimatedBackground(); }
                    if (dropZone) dropZone.style.opacity = '0'; if (filesListContainer) filesListContainer.style.opacity = '0'; if (paginationContainer) paginationContainer.style.opacity = '0';
                    
                    const btnTextSpan = askButton?.querySelector('.btn-text');
                    const hoverTextSpan = askButton?.querySelector('.hover-content span');
                    if (askButton) {
                        if (btnTextSpan) btnTextSpan.textContent = "Process"; 
                        if (hoverTextSpan) hoverTextSpan.textContent = "Process"; 
                        askButton.classList.add('px-5', 'process-mode'); 
                        askButton.classList.remove('px-4'); 
                    }
                    
                    setTimeout(() => {
                        if (dropZone) dropZone.style.display = 'none'; if (filesListContainer) filesListContainer.style.display = 'none'; if (paginationContainer) paginationContainer.style.display = 'none';
                        if (settingsPanel) { settingsPanel.style.display = 'block'; void settingsPanel.offsetWidth; settingsPanel.classList.add('show'); setupSettingsPanelListeners(); }
                        updateAskButtonState(); 
                    }, 400);
                }
                
                function setupSettingsPanelListeners() { 
                    const liquidToggleSwitchesInPanel = document.querySelectorAll('#settings-panel .liquid-toggle-switch');
                    liquidToggleSwitchesInPanel.forEach(toggleSwitch => {
                        const input = toggleSwitch.querySelector('.liquid-toggle-input');
                        if (input) { const newInput = input.cloneNode(true); input.parentNode.replaceChild(newInput, input); if (newInput.checked) toggleSwitch.classList.add('is-checked'); else toggleSwitch.classList.remove('is-checked'); newInput.addEventListener('change', e => { if (e.target.checked) toggleSwitch.classList.add('is-checked'); else toggleSwitch.classList.remove('is-checked'); }); }
                    });
                }
                renderFiles(); 
                updateDropZoneAppearance();
            } 
        });
    </script>
</body>
</html>
"""
chat_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chat Application - Dark Theme</title>
    <script src="https://cdn.tailwindcss.com"></script> <!-- Added Tailwind CSS -->
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
    <style>
        /* Dark theme tooltip styling from upload.html */
        [data-tooltip]:hover::after {
            content: attr(data-tooltip);
            position: absolute;
            bottom: 100%;
            left: 50%;
            transform: translateX(-50%) translateY(-4px);
            background-color: #333333; /* Dark gray background for tooltip */
            color: #E0E0E0;       /* Light text for tooltip */
            padding: 4px 8px;
            border-radius: 4px;
            font-size: 0.75rem;
            white-space: nowrap;
            z-index: 10;
        }
        [data-tooltip] {
            position: relative;
        }

        /* Icon sizes from upload.html */
        .icon-sm { width: 1rem; height: 1rem; } /* size-4 */
        .icon-md { width: 1.25rem; height: 1.25rem; } /* size-5 */

        /* Dark Theme Color Definitions from upload.html (redefining shadcn/ui like classes for dark mode) */
        .bg-primary { background-color: #F0F0F0; } /* Send button background (light) */
        .text-primary-foreground { color: #1C1C1C; } /* Send button icon color (dark) */
        .hover\:bg-primary\/90:hover { background-color: #DCDCDC; } /* Send button hover - Fixed selector */
        
        .bg-secondary { background-color: #2A2A2A; } /* File tag background */
        .text-secondary-foreground { color: #B0B0B0; } /* File tag text color */
        
        /* Attach icon button hover background from upload.html */
        .hover\:bg-secondary-foreground\/10:hover { background-color: #2A2A2A; } /* Fixed selector */

        .border-input { border-color: rgba(255, 255, 255, 0.2); } /* Input container border - more white */
        .bg-background { background-color: #1C1C1C; } /* Input container background */
        .text-primary { color: #E0E0E0 !important; } /* Main text color for new input elements */

        /* Placeholder text color for textarea from upload.html */
        textarea::placeholder {
            color: #808080; /* Muted gray for placeholder */
            opacity: 1; /* Ensure visibility */
        }
        textarea:-ms-input-placeholder { /* IE 10-11 */
           color: #808080;
        }
        textarea::-ms-input-placeholder { /* Edge */
           color: #808080;
        }

        /* Custom styling for file input from upload.html */
        input[type="file"].hidden-file-input {
            position: absolute;
            width: 1px;
            height: 1px;
            padding: 0;
            margin: -1px;
            overflow: hidden;
            clip: rect(0, 0, 0, 0);
            white-space: nowrap;
            border-width: 0;
        }

        /* Original chat.html styles below */        /* Dark Theme CSS Variables */
        :root {
            --font-family-sans: "Inter", -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif, "Apple Color Emoji", "Segoe UI Emoji", "Segoe UI Symbol";            /* Overall Dark Theme */
            --color-page-bg: #121212; /* Very dark page background - image implies this */
            --color-container-bg: #121212; /* Chat container background - main chat area bg, pitch black */
            --color-text-primary: #E0E0E0; /* Primary text (light) */
            --color-text-secondary: #A0A0A0; /* Secondary text (icons, placeholders) */
            --color-text-on-light-bg: #1E1E1E; /* Text for light backgrounds (e.g., user bubble, send button) */

            /* Message Bubbles - Enhanced with more vibrant gradients */
            --color-ai-bubble-bg: #2D2D2D;    
            --color-ai-bubble-gradient: linear-gradient(135deg, #2D2D2D 0%, #383838 100%);
            --color-user-bubble-bg: #FFFFFF;  
            --color-user-bubble-gradient: linear-gradient(135deg, #4776E6 0%, #8E54E9 100%); /* New vibrant gradient */
            
            /* Enhanced Colors for Accents - More vibrant palette */
            --color-accent-blue: #4776E6;
            --color-accent-purple: #8E54E9;
            --color-accent-teal: #38b2ac;
            --color-accent-pink: #e95494;
            --color-accent-orange: #FF8B13;
            --color-accent-green: #2ecc71;
            
            /* New modern colors for cards and UI elements */
            --color-card-gradient-1: linear-gradient(135deg, rgba(71, 118, 230, 0.1) 0%, rgba(142, 84, 233, 0.1) 100%);
            --color-card-gradient-2: linear-gradient(135deg, rgba(56, 178, 172, 0.1) 0%, rgba(71, 118, 230, 0.1) 100%);
            --color-card-border-glow: rgba(142, 84, 233, 0.3);
            
            /* Glass effect colors */
            --color-glass-bg: rgba(30, 30, 30, 0.5);
            --color-glass-border: rgba(255, 255, 255, 0.1);
            --color-glass-highlight: rgba(255, 255, 255, 0.05);
            --border-radius-lg: 0.5rem;
            --border-radius-md: 0.375rem;
            --border-radius-xs: 0.2rem;
            --border-radius-full: 9999px;

            --shared-container-height: 580px; /* Added shared height */
        }body {
            font-family: var(--font-family-sans);
            background-color: var(--color-page-bg); /* Page background */
            margin: 0;
            padding: 20px;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            box-sizing: border-box;
            color: var(--color-text-primary); /* Default text color for the page */
        }

        .main-container {
            display: flex;
            gap: 20px;
            width: 100%;
            max-width: 1200px;
            justify-content: center;
            align-items: flex-start; /* Align items to the start for consistent height behavior */
        }        .chat-container {
            width: 100%;
            max-width: 750px; /* Smaller box width */
            height: var(--shared-container-height); /* USE VARIABLE */
            border: 1px solid var(--color-glass-border);
            background-color: var(--color-glass-bg) !important; 
            border-radius: var(--border-radius-lg);
            display: flex;
            flex-direction: column;
            overflow: hidden;
            padding-bottom: 0;
            position: relative; /* For absolute positioning of canvas */
            margin-left: 32px; /* Move chat background to the right */
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
            backdrop-filter: blur(10px);
            -webkit-backdrop-filter: blur(10px);
            animation: container-fade-in 0.8s ease-out forwards;
        }
        
        @keyframes container-fade-in {
            0% {
                opacity: 0;
                transform: translateY(10px);
            }
            100% {
                opacity: 1;
                transform: translateY(0);
            }
        }

        /* Canvas styles for the flickering grid */
        .chat-container canvas {
            display: block;
            pointer-events: none; /* Make canvas non-interactive */
            position: absolute;
            inset: 0;
            width: 100%;
            height: 100%;
            z-index: 0; /* Behind content */
            border-radius: var(--border-radius-lg); /* Match container border radius */
        }

        /* Message List Area */        .message-list-area {
            flex: 1 1 0%;
            position: relative;
            overflow: hidden;
            bottom: 0px;
            z-index: 1; /* Ensure above canvas */ 
            width: 100%; /* Ensure full width */
            box-sizing: border-box; /* Consistent sizing model */
        }

        .message-list {
            height: 100%;
            padding: 1rem; /* p-4 */
            padding-bottom: 0px; /* Adjust as needed to match input area height */
            overflow-y: auto;
            display: flex;
            flex-direction: column;
            position: relative; /* Ensure proper stacking context */
            scrollbar-width: none; /* Firefox - hide scrollbar */
            -ms-overflow-style: none;  /* IE and Edge - hide scrollbar */
            width: 100%; /* Ensure full width */
            box-sizing: border-box; /* Consistent sizing model */
            scrollbar-gutter: stable; /* Reserve space for the gutter */
        }
        
        /* Webkit Scrollbar Styles */
        .message-list::-webkit-scrollbar {
            display: none; /* Webkit - hide scrollbar */
        }

        .message-items-container {
            display: flex;
            flex-direction: column;
            gap: 1rem;
            margin-top: auto;
        }        /* Individual Chat Bubble */
        .chat-bubble {
            display: flex;
            align-items: flex-start;
            gap: 0.75rem;
            margin-bottom: 1rem;
            opacity: 0;
            transform: translateY(20px);
            animation: fadeInUp var(--transition-normal) forwards;
            position: relative;
        }

        @keyframes fadeInUp {
            from {
                opacity: 0;
                transform: translateY(20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        .chat-bubble.sent {
            flex-direction: row-reverse;
        }
        
        /* Add animation delay to sequence multiple messages */
        .chat-bubble:nth-child(1) { animation-delay: 0.05s; }
        .chat-bubble:nth-child(2) { animation-delay: 0.1s; }
        .chat-bubble:nth-child(3) { animation-delay: 0.15s; }
        .chat-bubble:nth-child(4) { animation-delay: 0.2s; }
        .chat-bubble:nth-child(5) { animation-delay: 0.25s; }        .chat-bubble-avatar {
            height: 38px;
            width: 38px;
            flex-shrink: 0;
            border-radius: var(--border-radius-full);
            overflow: hidden;
            background-color: var(--color-ai-bubble-bg); /* Fallback BG */
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 0.75rem;
            color: var(--color-text-primary); /* Fallback text color */
            position: relative;
            box-shadow: 0 2px 6px rgba(0, 0, 0, 0.2);
            border: 2px solid rgba(255, 255, 255, 0.1);
            transition: transform var(--transition-normal), box-shadow var(--transition-normal);
        }

        .chat-bubble.received .chat-bubble-avatar {
            background: linear-gradient(135deg, #2D2D2D 0%, #383838 100%);
        }
        
        .chat-bubble.sent .chat-bubble-avatar {
            background: linear-gradient(135deg, #4776E6 0%, #8E54E9 100%);
        }
        
        .chat-bubble:hover .chat-bubble-avatar {
            transform: scale(1.05);
            box-shadow: 0 3px 10px rgba(0, 0, 0, 0.3);
        }
        
        /* AI avatar pulse animation */
        .chat-bubble.received .chat-bubble-avatar svg {
            animation: subtle-pulse 3s infinite ease-in-out;
        }
        
        @keyframes subtle-pulse {
            0% { transform: scale(1); }
            50% { transform: scale(1.05); }
            100% { transform: scale(1); }
        }
        
        .chat-bubble-avatar img {
            width: 100%;
            height: 100%;
            object-fit: cover;
        }
        
        .chat-bubble-avatar .fallback-text {
            position: absolute;
            width: 100%;
            height: 100%;
            display: flex;
            align-items: center;
            justify-content: center;
            background: var(--color-ai-bubble-bg); /* Ensure fallback has background */
        }

        .chat-bubble.sent .chat-bubble-avatar .fallback-text {
            background: linear-gradient(135deg, #4776E6 0%, #8E54E9 100%);
        }        .chat-bubble-message {
            border-radius: var(--border-radius-lg);
            padding: 1rem;
            max-width: 75%;
            word-wrap: break-word;
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.12);
            transition: transform var(--transition-normal), box-shadow var(--transition-normal);
            position: relative;
            overflow: hidden;
        }
        
        /* Add subtle grid pattern to chat bubbles */
        .chat-bubble-message::before {
            content: "";
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background-image: 
                linear-gradient(rgba(255, 255, 255, 0.025) 1px, transparent 1px),
                linear-gradient(90deg, rgba(255, 255, 255, 0.025) 1px, transparent 1px);
            background-size: 12px 12px;
            pointer-events: none;
            opacity: 0.5;
        }

        .chat-bubble:hover .chat-bubble-message {
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
        }
        
        /* Apply ripple effect on click */
        .chat-bubble-message.ripple::after {
            content: "";
            position: absolute;
            top: 50%;
            left: 50%;
            width: 5px;
            height: 5px;
            background: rgba(255, 255, 255, 0.4);
            opacity: 0;
            border-radius: 100%;
            transform: scale(1, 1) translate(-50%, -50%);
            transform-origin: 0 0;
            animation: ripple 0.6s ease-out;
        }
        
        @keyframes ripple {
            0% {
                transform: scale(0, 0) translate(-50%, -50%);
                opacity: 0.5;
            }
            100% {
                transform: scale(100, 100) translate(-50%, -50%);
                opacity: 0;
            }
        }

        .chat-bubble.received .chat-bubble-message {
            background-image: var(--color-ai-bubble-gradient);
            color: var(--color-text-primary);
            border-top-left-radius: 4px;
            border-left: 2px solid var(--color-accent-blue);
        }

        .chat-bubble.sent .chat-bubble-message {
            background-image: var(--color-user-bubble-gradient);
            color: white;
            border-top-right-radius: 4px;
            text-shadow: 0px 1px 1px rgba(0, 0, 0, 0.1);
        }        /* Loading Indicator */
        .loading-indicator {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 0.5rem 0;
        }
        
        .loading-indicator svg {
            width: 36px;
            height: 36px;
            fill: currentColor; /* Will inherit from parent bubble's text color */
            filter: drop-shadow(0 2px 3px rgba(0, 0, 0, 0.2));
        }
        
        /* Enhanced typing indicator */
        .typing-indicator {
            display: flex;
            align-items: center;
            padding: 0.5rem;
        }
        
        .typing-indicator span {
            height: 8px;
            width: 8px;
            background: var(--color-accent-blue);
            display: block;
            border-radius: 50%;
            opacity: 0.4;
            margin: 0 2px;
            animation: typing 1.5s infinite;
        }
        
        .typing-indicator span:nth-child(1) {
            animation-delay: 0s;
        }
        
        .typing-indicator span:nth-child(2) {
            animation-delay: 0.3s;
        }
        
        .typing-indicator span:nth-child(3) {
            animation-delay: 0.6s;
        }
        
        @keyframes typing {
            0%, 100% {
                transform: translateY(0);
                opacity: 0.4;
            }
            50% {
                transform: translateY(-10px);
                opacity: 1;
            }
        }
        
        .chat-bubble-message.loading .loading-indicator {
            display: flex;
            align-items: center;
            justify-content: center;
        }        /* Scroll to Bottom Button */
        .scroll-to-bottom-button {
            position: absolute;
            bottom: 0.75rem;
            left: 50%;
            transform: translateX(-50%);
            display: inline-flex;
            align-items: center;
            justify-content: center;
            width: 42px;
            height: 42px;
            border-radius: var(--border-radius-full);
            background-image: linear-gradient(135deg, #4776E6 0%, #8E54E9 100%);
            border: 2px solid rgba(255, 255, 255, 0.15);
            box-shadow: 0 3px 8px rgba(0, 0, 0, 0.3);
            cursor: pointer;
            color: white;
            transition: all var(--transition-normal);
            opacity: 0;
            visibility: hidden;
            z-index: 10;
        }
        
        .scroll-to-bottom-button.visible {
            opacity: 1;
            visibility: visible;
            animation: bounce 2s infinite;
        }
        
        @keyframes bounce {
            0%, 20%, 50%, 80%, 100% {
                transform: translateX(-50%) translateY(0);
            }
            40% {
                transform: translateX(-50%) translateY(-8px);
            }
            60% {
                transform: translateX(-50%) translateY(-4px);
            }
        }

        .scroll-to-bottom-button:hover {
            background-image: linear-gradient(135deg, #8E54E9 0%, #4776E6 100%);
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.4);
            transform: translateX(-50%) translateY(-2px);
        }
        
        .scroll-to-bottom-button:active {
            transform: translateX(-50%) scale(0.95);
            box-shadow: 0 2px 6px rgba(0, 0, 0, 0.3);
        }
        
        .scroll-to-bottom-button svg {
            width: 18px;
            height: 18px;
            filter: drop-shadow(0 1px 1px rgba(0, 0, 0, 0.2));
        }        /* Input Area */        .input-area {
            padding: 1rem;
            border-top: 1px solid rgba(255, 255, 255, 0.1); /* More white border */
            background-color: rgba(18, 18, 18, 0.4); /* Semi-transparent background that matches container */
            position: sticky;
            bottom: 0;
            z-index: 2;
            background-image: linear-gradient(to top, rgba(18, 18, 18, 0.6) 0%, rgba(18, 18, 18, 0) 100%);
            backdrop-filter: blur(5px);
        }        .input-form {
            position: relative;
            border-radius: var(--border-radius-lg);
            border: 1px solid rgba(255, 255, 255, 0.15); /* More white border */
            background-color: rgba(25, 25, 25, 0.4); /* Lighter opacity */
            padding: 0.5rem;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.15);
            transition: all var(--transition-normal);
        }
        
        .input-form:focus-within {
            border-color: var(--color-border-input-focus);
            box-shadow: 0 5px 16px rgba(58, 143, 255, 0.15);
            transform: translateY(-2px);
        }        .chat-textarea {
            display: block;
            width: 100%;
            min-height: 48px;
            resize: none;
            border-radius: var(--border-radius-md);
            background-color: rgba(25, 25, 25, 0.6); /* Slightly lighter than container bg */
            border: 0;
            padding: 0.75rem;
            box-shadow: none;
            font-size: 0.875rem;
            color: var(--color-text-primary); /* Light text for input */
            box-sizing: border-box;
        }
        .chat-textarea::placeholder {
            color: var(--color-text-secondary); /* Muted placeholder */
        }
        .chat-textarea:focus {
            outline: none;
        }

        .input-actions {
            display: flex;
            align-items: center;
            justify-content: space-between;
            padding: 0.75rem;
            padding-top: 0;
        }

        .input-actions-left {
            display: flex;
        }

        /* Button Styles */
        .btn {
            display: inline-flex;
            align-items: center;
            justify-content: center;
            white-space: nowrap;
            border-radius: var(--border-radius-md);
            font-size: 0.875rem;
            font-weight: 500;
            transition: background-color 0.2s, color 0.2s, border-color 0.2s, box-shadow 0.2s;
            cursor: pointer;
            border: 1px solid transparent;
            padding: 0.5rem 0.75rem;
        }
        .btn:focus-visible {
            outline: 2px solid var(--color-border-input-focus);
            outline-offset: 2px;
        }
        .btn:disabled {
            pointer-events: none;
            opacity: 0.5;
        }

        .btn-sm {
            height: 36px;
            padding: 0 0.75rem;
        }

        .btn-icon {
            height: 40px;
            width: 40px;
            padding: 0;
        }
         .btn-icon.sm { /* Custom small icon button */
            height: 32px;
            width: 32px;
        }        /* Send Message Button - now primary */
        .btn-primary {
            background-image: linear-gradient(135deg, var(--color-button-send-bg) 0%, var(--color-button-send-hover-bg) 100%);
            color: var(--color-button-send-text);
            box-shadow: 0 2px 6px rgba(0, 99, 230, 0.4);
            transform-origin: center;
            transition: all var(--transition-normal);
        }
        
        .btn-primary:hover {
            background-image: linear-gradient(135deg, #0063e6 0%, #004bb1 100%);
            box-shadow: 0 4px 12px rgba(0, 99, 230, 0.5);
            transform: translateY(-2px) scale(1.05);
        }
        
        .btn-primary:active {
            transform: translateY(0) scale(0.98);
        }
        
        .btn-primary svg { /* Ensure icon color matches text */
           stroke: var(--color-button-send-text);
           filter: drop-shadow(0 1px 2px rgba(0, 0, 0, 0.2));
        }


        /* Ghost Buttons (Attach, Mic) */
        .btn-ghost {
            background-color: transparent;
            color: var(--color-button-icon-idle); /* Icon color */
        }
        .btn-ghost:hover {
            background-color: var(--color-button-icon-hover-bg);
            color: var(--color-button-icon-hover-fg);
        }
        .btn-ghost svg { /* Ensure icon color is set by parent 'color' */
            stroke: currentColor;
        }


        .btn-send {
            margin-left: auto;
            gap: 0.375rem;
        }
        .btn-send svg {
            width: 14px;
            height: 14px;
        }
        .input-actions-left .btn-icon svg {
            width: 16px;
            height: 16px;
        }        /* File Preview Styles */
        .file-preview-container {
            display: flex;
            flex-wrap: wrap;
            gap: 0.75rem;
            padding-bottom: 0.75rem;
            width: 100%;
        }

        .file-preview-item {
            background-image: linear-gradient(135deg, rgba(58, 143, 255, 0.15) 0%, rgba(38, 123, 235, 0.25) 100%);
            display: flex;
            align-items: center;
            gap: 0.6rem;
            border-radius: var(--border-radius-lg);
            padding: 0.6rem 0.9rem;
            font-size: 0.875rem;
            color: var(--color-text-primary);
            box-shadow: 0 2px 6px rgba(0, 0, 0, 0.15);
            border: 1px solid rgba(58, 143, 255, 0.2);
            transition: all var(--transition-normal);
            animation: fadeIn 0.5s ease forwards;
        }
        
        @keyframes fadeIn {
            from {
                opacity: 0;
                transform: translateY(10px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }
        
        .file-preview-item:hover {
            border-color: rgba(58, 143, 255, 0.5);
            box-shadow: 0 3px 10px rgba(0, 0, 0, 0.2);
            transform: translateY(-2px);
        }

        .file-preview-item .icon {
            width: 1rem; /* Equivalent to size-4 (16px) */
            height: 1rem;
            stroke: currentColor;
        }

        .file-preview-name {
            max-width: 120px;
            white-space: nowrap;
            overflow: hidden;
            text-overflow: ellipsis;
        }        .file-remove-button {
            background-color: rgba(255, 255, 255, 0.05);
            border: 1px solid rgba(255, 255, 255, 0.1);
            color: var(--color-text-primary);
            cursor: pointer;
            padding: 0.3rem;
            border-radius: var(--border-radius-full);
            display: flex;
            align-items: center;
            justify-content: center;
            transition: all var(--transition-normal);
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.15);
        }

        .file-remove-button:hover {
            background-color: rgba(237, 100, 166, 0.2);
            border-color: rgba(237, 100, 166, 0.3);
            color: white;
            transform: rotate(90deg);
        }

        .file-remove-button svg {
            width: 1rem;
            height: 1rem;
            transition: transform var(--transition-normal);
        }
        
        .file-remove-button:hover svg {
            transform: scale(1.2);
        }
        /* End File Preview Styles */

        /* Utility for hiding elements accessibly */
        .sr-only {
            position: absolute;
            width: 1px;
            height: 1px;
            padding: 0;
            margin: -1px;
            overflow: hidden;
            clip: rect(0, 0, 0, 0);
            white-space: nowrap;
            border-width: 0;
        }    /* Document section styles - Enhanced with modern design */
        :root {
            /* Additional color variables for document section */
            --color-accent-blue: #4776E6;
            --color-accent-purple: #8E54E9;
            --color-accent-teal: #38b2ac;
            --color-accent-pink: #e95494;
            --color-accent-orange: #FF8B13;
            --color-document-gradient: linear-gradient(135deg, rgba(71, 118, 230, 0.1) 0%, rgba(142, 84, 233, 0.1) 100%);
            --color-document-card-bg: rgba(45, 45, 45, 0.7);
            --color-document-header-bg: rgba(35, 35, 35, 0.8);
        }

        .documents-container {
            width: 330px;
            height: var(--shared-container-height); /* USE VARIABLE */
            border: 1px solid rgba(80, 80, 80, 0.3);
            background-color: var(--color-container-bg);
            background-image: var(--color-document-gradient);
            background-blend-mode: overlay;
            border-radius: var(--border-radius-lg);
            display: flex;
            flex-direction: column;
            overflow-y: auto; /* Handle overflow */
            scrollbar-width: thin; /* For Firefox */
            scrollbar-color: var(--color-scrollbar-thumb) var(--color-scrollbar-track); /* For Firefox */
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
            position: relative;
        }

        .documents-container::-webkit-scrollbar {
            width: 8px;
        }
        .documents-container::-webkit-scrollbar-track {
            background: var(--color-scrollbar-track);
            border-radius: 4px;
        }
        .documents-container::-webkit-scrollbar-thumb {
            background: var(--color-scrollbar-thumb);
            border-radius: 4px;
        }
        .documents-container::-webkit-scrollbar-thumb:hover {
            background: var(--color-scrollbar-thumb-hover);
        }

        /* Fancy background pattern - subtle grid */
        .documents-container::before {
            content: "";
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background-image: 
                linear-gradient(rgba(255, 255, 255, 0.03) 1px, transparent 1px),
                linear-gradient(90deg, rgba(255, 255, 255, 0.03) 1px, transparent 1px);
            background-size: 20px 20px;
            pointer-events: none;
            z-index: 0;
        }

        .documents-header {
            padding: 1rem 1.25rem;
            border-bottom: 1px solid rgba(80, 80, 80, 0.5);
            font-weight: 600;
            color: var(--color-text-primary);
            display: flex;
            justify-content: space-between;
            align-items: center;
            background-color: var(--color-document-header-bg);
            position: relative;
            z-index: 1;
        }

        .documents-header span {
            position: relative;
            padding-left: 0.5rem;
        }
        
        .documents-header span::before {
            content: "";
            position: absolute;
            left: 0;
            top: 50%;
            transform: translateY(-50%);
            width: 3px;
            height: 1rem;
            background: var(--color-accent-blue);
            border-radius: 4px;
        }

        .documents-list {
            flex: 1;
            overflow-y: auto;
            padding: 1rem;
            position: relative;
            z-index: 1;
        }

        .documents-list::-webkit-scrollbar {
            width: 6px;
        }
        .documents-list::-webkit-scrollbar-track {
            background: var(--color-scrollbar-track);
            border-radius: 3px;
        }
        .documents-list::-webkit-scrollbar-thumb {
            background: var(--color-scrollbar-thumb);
            border-radius: 3px;
        }
        .documents-list::-webkit-scrollbar-thumb:hover {
            background: var(--color-scrollbar-thumb-hover);
        }
        .documents-list {
            scrollbar-width: thin;
            scrollbar-color: var(--color-scrollbar-thumb) var(--color-scrollbar-track);
        }

        /* Modern card styling for document items */
        .document-item {
            padding: 0.9rem;
            border-radius: var(--border-radius-lg);
            margin-bottom: 0.75rem;
            background-color: var(--color-document-card-bg);
            cursor: pointer;
            display: flex;
            align-items: center;
            gap: 0.75rem;
            transition: all 0.3s ease;
            border: 1px solid transparent;
            position: relative;
            overflow: hidden;
            box-shadow: 0 2px 6px rgba(0, 0, 0, 0.15);
        }

        .document-item::before {
            content: "";
            position: absolute;
            left: 0;
            top: 0;
            height: 100%;
            width: 3px;
            background-color: transparent;
            transition: all 0.3s ease;
        }
        
        .document-item:hover {
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            background-color: rgba(50, 50, 50, 0.8);
            border-color: rgba(80, 80, 80, 0.5);
        }
        
        .document-item:hover::before {
            background-color: var(--color-accent-blue);
        }

        .document-item.active {
            background-color: rgba(71, 118, 230, 0.15);
            border-color: rgba(71, 118, 230, 0.3);
        }
        
        .document-item.active::before {
            background-color: var(--color-accent-blue);
        }

        /* Document icon styling with colored backgrounds */
        .document-icon {
            color: var(--color-text-primary);
            background: rgba(71, 118, 230, 0.2);
            width: 36px;
            height: 36px;
            border-radius: 8px;
            display: flex;
            align-items: center;
            justify-content: center;
            flex-shrink: 0;
            transition: all 0.3s ease;
        }

        .document-item:hover .document-icon {
            transform: scale(1.05);
        }

        .document-item[data-doctype="pdf"] .document-icon {
            background: rgba(233, 84, 148, 0.2);  /* Pink */
        }
        
        .document-item[data-doctype="docx"] .document-icon {
            background: rgba(71, 118, 230, 0.2);  /* Blue */
        }
        
        .document-item[data-doctype="xlsx"] .document-icon {
            background: rgba(56, 178, 172, 0.2);  /* Teal */
        }
        
        .document-item[data-doctype="pptx"] .document-icon {
            background: rgba(255, 139, 19, 0.2);  /* Orange */
        }
        
        .document-item[data-doctype="txt"] .document-icon {
            background: rgba(142, 84, 233, 0.2);  /* Purple */
        }

        .document-info {
            flex: 1;
            overflow: hidden;
        }

        .document-name {
            font-size: 0.9rem;
            font-weight: 500;
            white-space: nowrap;
            overflow: hidden;
            text-overflow: ellipsis;
            margin-bottom: 0.35rem;
            transition: color 0.2s ease;
        }

        .document-item:hover .document-name {
            color: var(--color-accent-blue);
        }

        .document-meta {
            font-size: 0.75rem;
            color: var(--color-text-secondary);
            display: flex;
            align-items: center;
            gap: 0.5rem;
        }
        
        .document-meta-dot {
            display: inline-block;
            width: 3px;
            height: 3px;
            border-radius: 50%;
            background-color: var(--color-text-secondary);
        }

        .empty-documents {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100%;
            color: var(--color-text-secondary);
            padding: 1.5rem;
            text-align: center;
            position: relative;
            z-index: 1;
        }

        .empty-documents svg {
            width: 56px;
            height: 56px;
            margin-bottom: 1.25rem;
            stroke: var(--color-text-secondary);
            opacity: 0.6;
            animation: float 3s ease-in-out infinite;
        }

        @keyframes float {
            0% { transform: translateY(0px); }
            50% { transform: translateY(-8px); }
            100% { transform: translateY(0px); }
        }

        .empty-documents h3 {
            margin-bottom: 0.5rem;
            font-weight: 500;
        }

        .empty-documents p {
            margin: 0.5rem 0;
            opacity: 0.8;
        }

        .upload-btn {
            background-color: rgba(71, 118, 230, 0.1);
            color: var(--color-accent-blue);
            border: 1px solid var(--color-accent-blue);
            padding: 0.6rem 1rem;
            border-radius: var(--border-radius-md);
            font-size: 0.875rem;
            font-weight: 500;
            cursor: pointer;
            margin-top: 1.25rem;
            transition: all 0.3s ease;
            position: relative;
            overflow: hidden;
            display: flex;
            align-items: center;
            gap: 0.5rem;
        }

        .upload-btn svg {
            width: 18px;
            height: 18px;
        }

        .upload-btn:hover {
            background-color: rgba(71, 118, 230, 0.2);
            transform: translateY(-1px);
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.15);
        }
          .upload-btn:active {
            transform: translateY(0);
        }
        
        /* Ripple effect for document items */
        .document-item {
            position: relative;
            overflow: hidden;
        }
        
        .ripple-effect {
            position: absolute;
            border-radius: 50%;
            background-color: rgba(255, 255, 255, 0.15);
            transform: scale(0);
            animation: ripple 0.6s linear;
            pointer-events: none;
            z-index: 0;
        }
        
        @keyframes ripple {
            to {
                transform: scale(2);
                opacity: 0;
            }
        }

        /* Pixel Sparks Animation */
        @keyframes sparkle {
            0%, 100% {
                opacity: 0.1;
                transform: scale(0.8);
            }
            50% {
                opacity: 0.8;
                transform: scale(1.2);
            }
        }

        @keyframes slowFloat {
            0%, 100% {
                transform: translateY(0) translateX(0);
            }
            50% {
                transform: translateY(-20px) translateX(10px); /* Increased float distance */
            }
        }

        .spark {
            position: fixed; /* Use fixed to cover the whole page */
            width: 1px;
            height: 1px;
            background-color: white;
            border-radius: 50%;
            animation: sparkle var(--spark-duration, 5s) infinite cubic-bezier(0.45, 0.05, 0.55, 0.95),
                       slowFloat 15s infinite ease-in-out;
            animation-delay: var(--spark-delay, 0s);
            opacity: var(--spark-opacity, 0.5);
            pointer-events: none; /* So they don't interfere with interactions */
            z-index: 0; /* Behind other content but visible */
        }

        .spark-medium {
            width: 2px;
            height: 2px;
        }

        .spark-large {
            width: 3px;
            height: 3px;
            box-shadow: 0 0 4px 1px rgba(255, 255, 255, 0.3);
        }

        .spark-blue {
            background-color: rgba(99, 102, 241, 0.8); /* indigo-500 */
            box-shadow: 0 0 4px 1px rgba(99, 102, 241, 0.4);
        }

        .spark-purple {
            background-color: rgba(139, 92, 246, 0.8); /* violet-500 */
            box-shadow: 0 0 4px 1px rgba(139, 92, 246, 0.4);
        }

        .spark-rose {
            background-color: rgba(244, 63, 94, 0.8); /* rose-500 */
            box-shadow: 0 0 4px 1px rgba(244, 63, 94, 0.4);
        }
    </style>
</head>
<body>

    <!-- Pixel Sparks Layer -->
    <div id="pixel-sparks-container" class="fixed inset-0 overflow-hidden pointer-events-none" style="z-index: 0;">
    </div>

    <div class="main-container">        <div class="chat-container">
            <canvas id="chat-grid-canvas"></canvas>
            <div class="message-list-area">
                <div class="message-list" id="messageList">
                    <div class="message-items-container" id="messageItemsContainer">
                        <!-- Messages will be rendered here by JavaScript -->
                    </div>
                </div>
            <button class="scroll-to-bottom-button" id="scrollToBottomButton" aria-label="Scroll to bottom">
                <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><line x1="12" y1="5" x2="12" y2="19"></line><polyline points="19 12 12 19 5 12"></polyline></svg>
            </button>
        </div>        <div class="input-area">
            <form class="input-form" id="chatForm">
                <textarea id="chatInput"
                          placeholder="Ask me anything..."
                          class="text-primary min-h-[44px] w-full resize-none border-none bg-transparent shadow-none outline-none focus-visible:ring-0 focus-visible:ring-offset-0 p-2"
                          rows="1"></textarea>

                <div class="flex items-center justify-between gap-2 pt-2">
                    <div data-tooltip="Select AI model" title="Select AI model" style="visibility: hidden;">
                        <div class="relative">
                            <select id="modelSelect" class="hover:bg-secondary-foreground/10 text-primary bg-transparent cursor-pointer flex h-8 px-2 items-center justify-center rounded-2xl border border-zinc-700 text-sm appearance-none">
                                <option value="gpt-4">GPT-4</option>
                                <option value="gpt-3.5-turbo" selected>GPT-3.5</option>
                                <option value="claude-3">Claude 3</option>
                                <option value="gemini-pro">Gemini Pro</option>
                                <option value="llama-3">Llama 3</option>
                            </select>
                            <div class="pointer-events-none absolute inset-y-0 right-0 flex items-center px-2 text-primary">
                                <svg class="icon-sm" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><polyline points="6 9 12 15 18 9"></polyline></svg>
                            </div>
                        </div>
                    </div>

                    <div id="submitButtonTooltipWrapper" data-tooltip="Send message" title="Send message">
                        <button type="submit" id="sendButton"
                                class="bg-primary text-primary-foreground hover:bg-primary/90 h-8 w-8 rounded-full flex items-center justify-center">
                            <svg id="arrowUpIcon" class="icon-md" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><line x1="12" y1="19" x2="12" y2="5"></line><polyline points="5 12 12 5 19 12"></polyline></svg>
                            <svg id="squareIcon" class="icon-md fill-current hidden" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><rect x="3" y="3" width="18" height="18" rx="2" ry="2"></rect></svg>
                        </button>
                    </div>
                </div>
            </form>        </div>
    </div>
      <div class="documents-container">
        <div class="documents-header">
            <span>Uploaded Documents</span>
        </div>
        <div class="documents-list" id="documentsList">
            <div class="document-item" data-doctype="pdf" data-id="doc1" tabindex="0">
                <div class="document-icon">
                    <svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                        <path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path>
                        <polyline points="14 2 14 8 20 8"></polyline>
                        <line x1="16" y1="13" x2="8" y2="13"></line>
                        <line x1="16" y1="17" x2="8" y2="17"></line>
                        <polyline points="10 9 9 9 8 9"></polyline>
                    </svg>
                </div>
                <div class="document-info">
                    <div class="document-name">Project_Report_2025.pdf</div>
                    <div class="document-meta">
                        <span>PDF</span>
                        <span class="document-meta-dot"></span>
                        <span>2.4 MB</span>
                    </div>
                </div>
            </div>
            <div class="document-item" data-doctype="docx" data-id="doc2" tabindex="0">
                <div class="document-icon">
                    <svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                        <path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path>
                        <polyline points="14 2 14 8 20 8"></polyline>
                        <line x1="16" y1="13" x2="8" y2="13"></line>
                        <line x1="16" y1="17" x2="8" y2="17"></line>
                        <polyline points="10 9 9 9 8 9"></polyline>
                    </svg>
                </div>
                <div class="document-info">
                    <div class="document-name">Research_Analysis.docx</div>
                    <div class="document-meta">
                        <span>DOCX</span>
                        <span class="document-meta-dot"></span>
                        <span>1.8 MB</span>
                    </div>
                </div>
            </div>
            <div class="document-item" data-doctype="xlsx" data-id="doc3" tabindex="0">
                <div class="document-icon">
                    <svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                        <rect x="3" y="3" width="18" height="18" rx="2" ry="2"></rect>
                        <line x1="3" y1="9" x2="21" y2="9"></line>
                        <line x1="9" y1="21" x2="9" y2="9"></line>
                    </svg>
                </div>
                <div class="document-info">
                    <div class="document-name">Financial_Data_Q1.xlsx</div>
                    <div class="document-meta">
                        <span>XLSX</span>
                        <span class="document-meta-dot"></span>
                        <span>3.2 MB</span>
                    </div>
                </div>
            </div>
            <div class="document-item" data-doctype="pptx" data-id="doc4" tabindex="0">
                <div class="document-icon">
                    <svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                        <path d="M2 3h20v18H2z"></path>
                        <path d="M12 3v18"></path>
                    </svg>
                </div>
                <div class="document-info">
                    <div class="document-name">Presentation_May.pptx</div>
                    <div class="document-meta">
                        <span>PPTX</span>
                        <span class="document-meta-dot"></span>
                        <span>5.7 MB</span>
                    </div>
                </div>
            </div>
            
            <!-- Empty state - hidden by default -->
            <div class="empty-documents" style="display: none;" id="emptyDocumentsState">
                <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                    <path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path>
                    <polyline points="14 2 14 8 20 8"></polyline>
                </svg>
                <h3>No documents yet</h3>
                <p>Upload documents to chat with them</p>
                <button class="upload-btn" id="emptyStateUploadBtn">
                    <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                        <path d="M21 15v4a2 2 0 0 1-2 2H5a2 2 0 0 1-2-2v-4"></path>
                        <polyline points="17 8 12 3 7 8"></polyline>
                        <line x1="12" y1="3" x2="12" y2="15"></line>
                    </svg>
                    Upload Document
                </button>
            </div>
        </div>
    </div>
    </div>    <script>
        document.addEventListener('DOMContentLoaded', () => {            // DOM Elements
            const messageItemsContainer = document.getElementById('messageItemsContainer');
            const chatForm = document.getElementById('chatForm');
            const chatInput = document.getElementById('chatInput');
            const messageList = document.getElementById('messageList');
            const scrollToBottomButton = document.getElementById('scrollToBottomButton');
            const modelSelect = document.getElementById('modelSelect');
            const sendButton = document.getElementById('sendButton');
            const arrowUpIcon = document.getElementById('arrowUpIcon');
            const squareIcon = document.getElementById('squareIcon');
            const submitButtonTooltipWrapper = document.getElementById('submitButtonTooltipWrapper');
            
            // Document section elements
            const documentsList = document.getElementById('documentsList');
            const documentItems = document.querySelectorAll('.document-item');
            const documentUploadBtn = document.getElementById('documentUploadBtn');
            const documentFileInput = document.getElementById('documentFileInput');
            const emptyDocumentsState = document.getElementById('emptyDocumentsState');
            const emptyStateUploadBtn = document.getElementById('emptyStateUploadBtn');

            // Initial messages data
            let messages = [
                { id: 1, content: "Hello! How can I help you today?", sender: "ai" },
                { id: 2, content: "I have a question about the component library.", sender: "user" },
                { id: 3, content: "Sure! I'd be happy to help. What would you like to know?", sender: "ai" },
            ];            let isLoading = false;
            let autoScrollEnabled = true;
            let selectedModel = "gpt-3.5-turbo"; // Default selected model
            const MAX_TEXTAREA_HEIGHT = 240; // Maximum height for the textarea

            const updateSubmitButtonUI = () => {
                if (isLoading) {
                    arrowUpIcon.classList.add('hidden');
                    squareIcon.classList.remove('hidden');
                    sendButton.disabled = true;
                    if (submitButtonTooltipWrapper) {
                        submitButtonTooltipWrapper.setAttribute('data-tooltip', 'Stop generation');
                        submitButtonTooltipWrapper.setAttribute('title', 'Stop generation');
                    }
                } else {
                    arrowUpIcon.classList.remove('hidden');
                    squareIcon.classList.add('hidden');
                    sendButton.disabled = false;
                    if (submitButtonTooltipWrapper) {
                        submitButtonTooltipWrapper.setAttribute('data-tooltip', 'Send message');
                        submitButtonTooltipWrapper.setAttribute('title', 'Send message');
                    }
                }
            };            chatInput.addEventListener('input', () => {
                chatInput.style.height = 'auto';
                const newHeight = Math.min(chatInput.scrollHeight, MAX_TEXTAREA_HEIGHT);
                chatInput.style.height = `${newHeight}px`;
            });            
            
            // Model Selection Handling
            modelSelect.addEventListener('change', (event) => {
                selectedModel = event.target.value;
                console.log(`Model changed to: ${selectedModel}`);
            });
            
            // Set initial selected model
            selectedModel = modelSelect.value;

            const isScrolledToBottom = () => {
                return messageList.scrollHeight - messageList.clientHeight <= messageList.scrollTop + 20;
            };

            const updateScrollToBottomButtonVisibility = () => {
                if (!isScrolledToBottom() && messageList.scrollHeight > messageList.clientHeight) {
                    scrollToBottomButton.classList.add('visible');
                } else {
                    scrollToBottomButton.classList.remove('visible');
                }
            };
            
            const scrollToBottom = (smooth = true) => {
                messageList.scrollTo({
                    top: messageList.scrollHeight,
                    behavior: smooth ? 'smooth' : 'auto'
                });
                autoScrollEnabled = true;
                updateScrollToBottomButtonVisibility();
            };

            messageList.addEventListener('scroll', () => {
                if (!isScrolledToBottom() && messageList.scrollTop > 0) {
                    autoScrollEnabled = false;
                } else if (isScrolledToBottom()) {
                    autoScrollEnabled = true;
                }
                updateScrollToBottomButtonVisibility();
            });
            
            const disableAutoScrollOnInteraction = () => {
                if (!isScrolledToBottom()) {
                    autoScrollEnabled = false;
                    updateScrollToBottomButtonVisibility();
                }
            };
            messageList.addEventListener('wheel', disableAutoScrollOnInteraction);
            messageList.addEventListener('touchmove', disableAutoScrollOnInteraction);


            scrollToBottomButton.addEventListener('click', () => scrollToBottom(true));
            
            const createMessageBubbleElement = (message) => {
                const bubbleDiv = document.createElement('div');
                bubbleDiv.classList.add('chat-bubble', message.sender === 'user' ? 'sent' : 'received');
                bubbleDiv.dataset.messageId = message.id;

                const avatarDiv = document.createElement('div');
                avatarDiv.classList.add('chat-bubble-avatar');
                const fallbackTextContent = message.sender === 'user' ? 'US' : 'AI';

                const fallbackSpan = document.createElement('span');
                fallbackSpan.classList.add('fallback-text');
                fallbackSpan.textContent = fallbackTextContent;
                fallbackSpan.style.display = 'none'; // Initially hidden
                avatarDiv.appendChild(fallbackSpan); // Add it, but hidden

                if (message.sender === 'user') {
                    const userAvatarSrc = "https://images.unsplash.com/photo-1534528741775-53994a69daeb?w=64&h=64&q=80&crop=faces&fit=crop";
                    if (userAvatarSrc) {
                        const img = document.createElement('img');
                        img.src = userAvatarSrc;
                        img.alt = 'User avatar';
                        img.onerror = () => {
                            img.style.display = 'none';
                            fallbackSpan.style.display = 'flex';
                        };
                        // Prepend img so it's visually on top of the (hidden) fallback span
                        avatarDiv.insertBefore(img, fallbackSpan);
                    } else {
                        fallbackSpan.style.display = 'flex'; // No src, show fallback
                    }
                } else { // AI sender - use SVG
                    const aiSvgIconString = `
                    <svg version="1.1" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 512 512">
                    <path d="M0 0 C9.69242413 6.95288386 16.11995831 16.46391612 18.3125 28.3125 C19.20255131 42.86141557 16.76980464 53.93664943 7.56640625 65.5078125 C1.29219651 72.36900652 -6.61295146 76.49759029 -15.6875 78.3125 C-15.6875 95.1425 -15.6875 111.9725 -15.6875 129.3125 C-11.16675781 129.29155273 -6.64601563 129.27060547 -1.98828125 129.24902344 C12.98728159 129.18259503 27.96285042 129.13829701 42.93852234 129.10525131 C52.0137792 129.08469022 61.08889575 129.05669417 70.1640625 129.01074219 C78.08073434 128.97067559 85.99730447 128.94502039 93.9140749 128.93614602 C98.09985774 128.93096456 102.28534984 128.91892363 106.47103882 128.88957977 C110.42405159 128.86209595 114.37665844 128.85412022 118.32975769 128.86006927 C119.76791096 128.85898356 121.20608017 128.85122813 122.64414978 128.8356781 C141.27513734 128.64438222 157.31037554 133.04632072 171.09887695 146.0065918 C182.50888799 157.19497629 189.18496563 172.04000378 189.53955078 188.11962891 C189.53924116 189.7594742 189.52837038 191.39934604 189.5078125 193.0390625 C189.50356766 194.34960098 189.50356766 194.34960098 189.49923706 195.68661499 C189.48817303 198.43705814 189.46309581 201.18715774 189.4375 203.9375 C189.42745234 205.81900567 189.41832964 207.70051651 189.41015625 209.58203125 C189.38827912 214.15898357 189.35384719 218.73568385 189.3125 223.3125 C190.74468872 223.29292236 190.74468872 223.29292236 192.20581055 223.27294922 C193.46176025 223.26279785 194.71770996 223.25264648 196.01171875 223.2421875 C197.87703247 223.22260986 197.87703247 223.22260986 199.7800293 223.20263672 C208.92942354 223.4871918 216.38261053 227.84273355 223.0703125 233.90625 C231.40892456 242.78959551 231.91870649 253.21747817 231.84375 264.8125 C231.84645171 266.2988307 231.85033974 267.78515966 231.85534668 269.27148438 C231.86120996 272.37126664 231.85278259 275.47058466 231.83398438 278.5703125 C231.81119598 282.51758796 231.82433633 286.46377508 231.84827423 290.41101074 C231.86281899 293.47566738 231.8580869 296.54007154 231.84776306 299.60473633 C231.84511964 301.05900201 231.8482803 302.5132916 231.85768127 303.9675293 C231.92229808 316.76816484 229.79235563 325.67021825 220.828125 335.109375 C213.78465388 341.27788076 206.30850289 343.47926685 197.125 343.375 C196.37541016 343.37048828 195.62582031 343.36597656 194.85351562 343.36132812 C193.00647617 343.34956354 191.15947787 343.33162865 189.3125 343.3125 C189.3439209 345.04983398 189.3439209 345.04983398 189.37597656 346.82226562 C189.44942568 351.1982747 189.49467335 355.57423173 189.53222656 359.95068359 C189.55210682 361.83000914 189.57922087 363.70927483 189.61425781 365.58837891 C189.98675596 386.10864428 187.31595348 403.5897168 172.578125 419.03515625 C171.83046875 419.78667969 171.0828125 420.53820312 170.3125 421.3125 C169.70921875 421.95058594 169.1059375 422.58867187 168.484375 423.24609375 C164.42061523 427.21267666 159.95800211 429.70827385 154.875 432.125 C154.26640137 432.41528076 153.65780273 432.70556152 153.03076172 433.00463867 C145.44486995 436.41698102 137.89366341 437.6159969 129.62207031 437.55732727 C128.38274435 437.56072413 128.38274435 437.56072413 127.1183815 437.56418961 C124.37571546 437.56944711 121.6332795 437.56068588 118.890625 437.55200195 C116.91377694 437.55311589 114.93692921 437.55517534 112.96008301 437.55810547 C108.71551669 437.56281498 104.47100575 437.56172089 100.22644043 437.55632973 C93.51062045 437.54870193 86.79490651 437.55939525 80.07910156 437.5735321 C60.98632375 437.6112221 41.89359463 437.62945488 22.80078125 437.61791992 C12.24642294 437.61170072 1.69223644 437.6232956 -8.86207962 437.65436524 C-15.53416942 437.67300013 -22.20577178 437.66971995 -28.87785339 437.65019763 C-33.03236651 437.64330603 -37.18661641 437.65729205 -41.34107971 437.67714691 C-43.26269738 437.68218499 -45.1843596 437.67873313 -47.10594177 437.66601372 C-60.74764239 437.36410688 -60.74764239 437.36410688 -72.6875 443.3125 C-74.1062087 444.24315187 -75.52939193 445.16704146 -76.95825195 446.08203125 C-78.44993142 447.09266955 -79.93899327 448.10717932 -81.42578125 449.125 C-82.24884766 449.68345108 -83.07191406 450.24190216 -83.91992188 450.817276 C-85.69245915 452.02002931 -87.46396557 453.22430299 -89.23452759 454.42996216 C-93.99375952 457.66971033 -98.76193927 460.89620726 -103.53125 464.12109375 C-104.51958328 464.78942734 -105.50791656 465.45776093 -106.52619934 466.14634705 C-117.62781645 473.64119013 -128.80813969 481.01585282 -140 488.375 C-141.42706772 489.31339722 -141.42706772 489.31339722 -142.88296509 490.27075195 C-146.38709105 492.57440546 -149.89133395 494.8778749 -153.39746094 497.17848206 C-154.51229858 497.91009804 -154.51229858 497.91009804 -155.6496582 498.65649414 C-156.34173874 499.10993698 -157.03381927 499.56337982 -157.74687195 500.03056335 C-159.54131919 501.21593614 -161.31901311 502.42326369 -163.09619141 503.63427734 C-167.07771692 506.21285341 -169.01997804 505.97928885 -173.6875 505.3125 C-176.8547802 502.8140388 -178.48422626 501.32434978 -179.28877258 497.31950378 C-179.27766953 496.21141861 -179.26656647 495.10333344 -179.25512695 493.96166992 C-179.2547493 492.7007402 -179.25437164 491.43981049 -179.25398254 490.14067078 C-179.22845324 488.77475994 -179.20237333 487.40885931 -179.17578125 486.04296875 C-179.16644859 484.63765218 -179.15933461 483.23231928 -179.15434265 481.82698059 C-179.13530772 478.13911541 -179.08623971 474.45224449 -179.03082275 470.76477051 C-178.97955304 466.99761402 -178.9568123 463.23029589 -178.93164062 459.46289062 C-178.87811537 452.0789752 -178.79284782 444.69587724 -178.6875 437.3125 C-179.30183899 437.31725845 -179.91617798 437.32201691 -180.5491333 437.32691956 C-181.35695251 437.33062057 -182.16477173 437.33432159 -182.99707031 437.33813477 C-183.7970343 437.34289322 -184.59699829 437.34765167 -185.42120361 437.35255432 C-200.02114966 437.09451617 -212.80354025 428.32271471 -222.6875 418.3125 C-223.24953125 417.77238281 -223.8115625 417.23226563 -224.390625 416.67578125 C-238.55216241 401.88070138 -239.13931666 382.16545145 -238.875 363 C-238.85991845 361.08399495 -238.84623622 359.16797836 -238.83398438 357.25195312 C-238.80127091 352.60517831 -238.74968253 347.95897005 -238.6875 343.3125 C-239.7801416 343.32917725 -240.8727832 343.34585449 -241.99853516 343.36303711 C-243.44954072 343.37652665 -244.90055013 343.38960724 -246.3515625 343.40234375 C-247.42869507 343.42041077 -247.42869507 343.42041077 -248.52758789 343.43884277 C-258.70466417 343.50575894 -265.16965762 339.72749604 -272.484375 332.7578125 C-278.51872414 326.06011292 -280.78959696 319.66963039 -280.86450195 310.77490234 C-280.87107315 310.12713837 -280.87764435 309.47937439 -280.88441467 308.8119812 C-280.90407727 306.67413193 -280.91579725 304.53635284 -280.92578125 302.3984375 C-280.92985678 301.66437305 -280.9339323 300.93030861 -280.93813133 300.17399979 C-280.95896665 296.28511083 -280.97327675 292.39625685 -280.98266602 288.50732422 C-280.99369805 284.51291696 -281.02806739 280.51907173 -281.06783772 276.52486229 C-281.09413086 273.4345301 -281.10236923 270.34432898 -281.10594749 267.25389481 C-281.11078431 265.78284555 -281.12231432 264.31180099 -281.14098549 262.84086227 C-281.28605064 250.65039031 -279.62190064 241.42439506 -271.0390625 232.359375 C-263.22603024 225.55866253 -255.55821798 223.15482974 -245.375 223.25 C-244.12074219 223.25902344 -242.86648437 223.26804688 -241.57421875 223.27734375 C-240.14529297 223.29474609 -240.14529297 223.29474609 -238.6875 223.3125 C-238.70321045 222.17409668 -238.7189209 221.03569336 -238.73510742 219.86279297 C-238.79008475 215.57250256 -238.8240924 211.28224214 -238.85229492 206.99169922 C-238.86722913 205.14724463 -238.88758395 203.30282457 -238.91381836 201.45849609 C-239.1987301 180.88898562 -236.68806188 162.84723687 -221.94506836 147.37426758 C-208.70730082 134.14099733 -193.19268212 129.10278348 -174.79718018 129.06365967 C-173.39782618 129.07073472 -171.99847317 129.07800429 -170.59912109 129.08544922 C-169.10012825 129.0860144 -167.60113486 129.08569468 -166.10214233 129.08456421 C-162.06149167 129.08399387 -158.02096142 129.09569608 -153.98033857 129.10971212 C-149.74695933 129.12227309 -145.51357688 129.12339419 -141.28018188 129.12576294 C-133.27647988 129.13196201 -125.27283297 129.14835141 -117.26915514 129.16846192 C-108.15181305 129.19087005 -99.03446824 129.20182667 -89.91710544 129.21185279 C-71.17386611 129.23271914 -52.43070224 129.26937557 -33.6875 129.3125 C-33.6875 112.4825 -33.6875 95.6525 -33.6875 78.3125 C-36.9875 77.3225 -40.2875 76.3325 -43.6875 75.3125 C-46.97873068 73.65497717 -49.79189136 71.58568809 -52.6875 69.3125 C-53.30496094 68.83296875 -53.92242187 68.3534375 -54.55859375 67.859375 C-61.82763389 61.44837301 -67.26221897 50.50347625 -67.92578125 40.84375 C-68.30840151 26.98841543 -65.32576318 15.5475025 -55.6875 5.15625 C-40.32345499 -9.54933594 -17.82502146 -10.84607238 0 0 Z M-43.5625 19.0625 C-48.36226145 25.0983949 -50.37910454 30.99022683 -50.09375 38.73046875 C-48.90234047 46.3027928 -44.78354451 52.20910873 -38.84765625 56.90234375 C-32.59175535 60.98621396 -26.70462848 61.91373614 -19.328125 60.9453125 C-12.04105407 59.1990026 -6.87543848 54.35304678 -2.6875 48.3125 C0.95796947 41.8721706 1.3801024 35.81120194 -0.30859375 28.6953125 C-3.02429084 20.75340723 -7.87486716 16.58541573 -14.9375 12.375 C-25.6956105 9.32686869 -35.7676854 10.79576552 -43.5625 19.0625 Z M-211.8984375 163.890625 C-218.58798582 172.94634543 -221.00588344 182.63944449 -220.94819641 193.75495911 C-220.95202196 194.5545584 -220.95584751 195.3541577 -220.95978898 196.17798728 C-220.97002567 198.83072245 -220.96614473 201.48323738 -220.9621582 204.13598633 C-220.96650989 206.04885557 -220.97169455 207.96172308 -220.97764587 209.87458801 C-220.99099891 215.05240136 -220.99161859 220.23014155 -220.98908257 225.40796876 C-220.98801022 229.73922232 -220.99290306 234.07046123 -220.997688 238.40171164 C-221.0087707 248.62532223 -221.00922774 258.84889608 -221.00317383 269.07250977 C-220.99713624 279.59880892 -221.00944248 290.12497432 -221.0307439 300.65125066 C-221.04839206 309.70717969 -221.05435613 318.76306628 -221.05110615 327.81901187 C-221.04929807 333.21952019 -221.05188185 338.61993313 -221.06582069 344.0204258 C-221.07844656 349.10207583 -221.07641006 354.18351828 -221.06340981 359.26516533 C-221.06105471 361.12263689 -221.06374014 362.98012328 -221.07203293 364.83757782 C-221.13981382 381.20899075 -219.56771497 394.80387152 -208.1875 407.5 C-199.32893227 415.78704723 -189.76944985 418.72330915 -178.05078125 419.8671875 C-172.24095985 420.63643215 -168.33792177 422.57744122 -164.25 426.875 C-162.04516398 430.3145442 -161.55719212 432.97033412 -161.48217773 436.9675293 C-161.4522599 438.37865242 -161.4522599 438.37865242 -161.42173767 439.81828308 C-161.39666893 441.33715874 -161.39666893 441.33715874 -161.37109375 442.88671875 C-161.35031265 443.92568802 -161.32953156 444.96465729 -161.30812073 446.03511047 C-161.24269344 449.35667253 -161.18377179 452.67831416 -161.125 456 C-161.08180831 458.2506593 -161.03819381 460.50131052 -160.99414062 462.75195312 C-160.88701902 468.2720511 -160.78523984 473.79222837 -160.6875 479.3125 C-154.38402811 475.94578867 -148.42422768 472.2505775 -142.52734375 468.21875 C-141.67146667 467.63837982 -140.8155896 467.05800964 -139.93377686 466.46005249 C-138.10647761 465.22077495 -136.28021788 463.97996348 -134.45492554 462.73773193 C-129.65854418 459.47469677 -124.85273611 456.2256131 -120.046875 452.9765625 C-119.08549118 452.32640167 -118.12410736 451.67624084 -117.1335907 451.00637817 C-108.93088909 445.46470835 -100.69656078 439.97263678 -92.42459106 434.53491211 C-91.06950202 433.64406934 -89.71521864 432.7519997 -88.36178589 431.85864258 C-86.49743611 430.62871736 -84.63015406 429.40339463 -82.76171875 428.1796875 C-81.20843872 427.16044189 -81.20843872 427.16044189 -79.6237793 426.12060547 C-72.29908992 421.61019976 -67.09627671 420.16184261 -58.57383728 420.18119812 C-57.33691087 420.17802455 -57.33691087 420.17802455 -56.07499605 420.17478687 C-53.30771519 420.16878843 -50.54047994 420.16978692 -47.77319336 420.1706543 C-45.79047569 420.16762261 -43.80775872 420.16410113 -41.82504272 420.16012573 C-37.55203613 420.1522357 -33.2790379 420.14709136 -29.00602531 420.14383507 C-22.24386417 420.1379231 -15.48175561 420.12137992 -8.71961975 420.10249329 C-6.40383438 420.09615576 -4.08804897 420.08983013 -1.77226353 420.08351612 C-0.03250506 420.0787674 -0.03250506 420.0787674 1.74240005 420.07392275 C17.48540249 420.0318656 33.22838068 419.99888331 48.97143555 419.98657227 C59.58986138 419.97818335 70.2081572 419.95566038 80.826518 419.9172166 C86.44338381 419.89748143 92.06005539 419.88504293 97.67695808 419.89005089 C102.9654513 419.89470847 108.25357686 419.88083104 113.541996 419.85303879 C115.47727311 419.84614531 117.41258838 419.84598049 119.34786415 419.85323906 C135.56659671 419.90790039 147.5874089 417.35068368 160 406.1875 C168.62436066 397.12189298 171.63632532 385.0341323 171.57319641 372.87004089 C171.57702196 372.0704416 171.58084751 371.2708423 171.58478898 370.44701272 C171.59502567 367.79427755 171.59114473 365.14176262 171.5871582 362.48901367 C171.59150989 360.57614443 171.59669455 358.66327692 171.60264587 356.75041199 C171.61599891 351.57259864 171.61661859 346.39485845 171.61408257 341.21703124 C171.61301022 336.88577768 171.61790306 332.55453877 171.622688 328.22328836 C171.6337707 317.99967777 171.63422774 307.77610392 171.62817383 297.55249023 C171.62213624 287.02619108 171.63444248 276.50002568 171.6557439 265.97374934 C171.67339206 256.91782031 171.67935613 247.86193372 171.67610615 238.80598813 C171.67429807 233.40547981 171.67688185 228.00506687 171.69082069 222.6045742 C171.70344656 217.52292417 171.70141006 212.44148172 171.68840981 207.35983467 C171.68605471 205.50236311 171.68874014 203.64487672 171.69703293 201.78742218 C171.76413862 185.57909434 170.34783781 172.36392251 159.0546875 159.80078125 C150.66554838 151.64092729 140.49051573 147.2001875 128.76725006 147.17827892 C127.65759211 147.17382108 126.54793416 147.16936324 125.40465021 147.16477031 C123.58381552 147.16504685 123.58381552 147.16504685 121.72619629 147.16532898 C120.43252687 147.16165229 119.13885745 147.1579756 117.80598593 147.1541875 C114.21667862 147.14418795 110.62738803 147.140454 107.03806877 147.13779819 C103.16601733 147.13388902 99.29397905 147.12431107 95.42193604 147.11558533 C86.07126482 147.09583752 76.72059256 147.08575653 67.36990571 147.07685754 C62.96341523 147.0724716 58.55692594 147.06710556 54.1504364 147.06186867 C39.49670918 147.04485756 24.84298271 147.03036444 10.18924713 147.02311993 C6.38788349 147.02121012 2.58651986 147.01928977 -1.21484375 147.01733398 C-2.1596217 147.01685024 -3.10439966 147.0163665 -4.07780725 147.0158681 C-19.37501263 147.00757172 -34.67213972 146.9822409 -49.96931094 146.94975331 C-65.68461603 146.91665059 -81.39988137 146.89866824 -97.1152215 146.89547569 C-105.93470777 146.89331349 -114.75407988 146.88460752 -123.5735321 146.85901451 C-131.08623996 146.83726354 -138.59880592 146.82927863 -146.11154169 146.83898129 C-149.94105181 146.8435134 -153.77026944 146.84170389 -157.59973907 146.82221985 C-161.76180544 146.80122758 -165.9232286 146.81161385 -170.08532715 146.82475281 C-171.28137117 146.81378356 -172.47741519 146.80281432 -173.70970297 146.79151267 C-189.26231195 146.90956036 -201.84062355 151.94947537 -211.8984375 163.890625 Z M-257.6875 245.3125 C-261.71506375 249.76401783 -262.81033663 252.19944396 -262.86450195 258.13574219 C-262.87733719 259.3575116 -262.89017242 260.57928101 -262.90339661 261.83807373 C-262.91156524 263.17850336 -262.91900874 264.51893759 -262.92578125 265.859375 C-262.92985678 266.53407082 -262.9339323 267.20876663 -262.93813133 267.90390778 C-262.9589944 271.47695103 -262.97329346 275.04995588 -262.98266602 278.62304688 C-262.99371432 282.30628739 -263.02809415 285.98892106 -263.06783772 289.67194748 C-263.09404157 292.50991911 -263.10236126 295.3477472 -263.10594749 298.18582916 C-263.11316184 300.20102044 -263.13958771 302.21611347 -263.16633606 304.23114014 C-263.23987317 313.10781492 -263.23987317 313.10781492 -258.6875 320.3125 C-251.38641321 326.74440979 -250.63307838 325.3125 -238.6875 325.3125 C-238.6875 297.5925 -238.6875 269.8725 -238.6875 241.3125 C-249.66800399 240.70606433 -249.66800399 240.70606433 -257.6875 245.3125 Z M189.3125 241.3125 C189.3125 269.0325 189.3125 296.7525 189.3125 325.3125 C200.65916932 326.2447746 200.65916932 326.2447746 208.3125 321.3125 C212.2897413 316.79779366 213.43503975 314.45809198 213.48950195 308.48925781 C213.50233719 307.2674884 213.51517242 306.04571899 213.52839661 304.78692627 C213.53656524 303.44649664 213.54400874 302.10606241 213.55078125 300.765625 C213.55485678 300.09092918 213.5589323 299.41623337 213.56313133 298.72109222 C213.5839944 295.14804897 213.59829346 291.57504412 213.60766602 288.00195312 C213.61871432 284.31871261 213.65309415 280.63607894 213.69283772 276.95305252 C213.71904157 274.11508089 213.72736126 271.2772528 213.73094749 268.43917084 C213.73816184 266.42397956 213.76458771 264.40888653 213.79133606 262.39385986 C213.8629274 253.518419 213.8629274 253.518419 209.3125 246.3125 C202.46302374 240.13126533 199.72084686 241.3125 189.3125 241.3125 Z " fill="#ffffff" transform="translate(280.6875,6.6875)"/>
                    <path d="M0 0 C8.01586887 8.96766035 12.11674298 19.99215042 12 32 C11.27765722 43.55748448 6.73187961 55.11902282 -2 63 C-12.72410644 71.53382651 -24.00554579 74.74889761 -37.6953125 73.81640625 C-47.92175458 72.2216074 -58.15056194 66.76882572 -65 59 C-72.20229507 48.975849 -75.94477181 38.36792185 -75 26 C-72.99278251 13.86186591 -67.80272188 2.63529914 -57.609375 -4.7265625 C-38.52336687 -16.61140805 -16.99091466 -15.29975052 0 0 Z M-50 13 C-54.92754793 18.97903305 -57.54733009 25.20054619 -57 33 C-55.4582685 40.69622417 -52.16192735 46.68822182 -45.6875 51.25 C-39.4204617 55.13882996 -33.95508588 56.63858076 -26.62109375 55.65625 C-18.53606244 53.63197986 -13.32269957 49.04488116 -9.05859375 42.05078125 C-5.93409653 35.99778848 -5.76749484 28.01681733 -7.75390625 21.54296875 C-8.71298403 19.57591902 -9.7672641 17.8060012 -11 16 C-11.53625 15.113125 -12.0725 14.22625 -12.625 13.3125 C-23.68075714 2.54768384 -38.74570724 2.95004115 -50 13 Z " fill="#ffffff" transform="translate(390,234)"/>
                    <path d="M0 0 C8.3912395 7.99006471 13.0685797 18.49743062 14.1640625 30 C14.46993576 42.99961334 9.69170845 54.50474833 0.9765625 64.015625 C-2.63740311 67.33543062 -6.6828801 69.65100567 -11.0390625 71.8671875 C-11.67457031 72.1971875 -12.31007812 72.5271875 -12.96484375 72.8671875 C-22.00643597 76.89427782 -33.735812 77.04567714 -43.0390625 73.8671875 C-56.11249659 68.43254244 -64.12357014 61.54134061 -70.109375 48.54296875 C-74.0620671 37.85534983 -73.9768592 26.270799 -69.77734375 15.6875 C-64.31808586 4.19583378 -57.41386267 -3.0619801 -45.57421875 -8.203125 C-29.51263365 -13.85899941 -13.05373855 -10.98022051 0 0 Z M-49.0390625 15.8671875 C-53.62164076 22.02189114 -54.85563062 28.08715244 -54.515625 35.68359375 C-53.4822305 42.58700786 -50.71536458 47.876825 -45.2890625 52.3046875 C-38.76214389 56.60707579 -32.42952704 58.6860756 -24.59765625 57.57421875 C-17.91033874 55.72627299 -11.7420522 51.79197102 -8.0390625 45.8671875 C-7.4821875 44.9803125 -6.9253125 44.0934375 -6.3515625 43.1796875 C-3.64418937 36.34679341 -3.26061503 29.91472683 -5.97265625 23.03125 C-9.62327998 15.62302456 -14.26901368 10.87453198 -22.0390625 7.8671875 C-31.85735989 5.09394911 -42.07279163 8.57654481 -49.0390625 15.8671875 Z " fill="#ffffff" transform="translate(183.0390625,232.1328125)"/>
                    <path d="M0 0 C2.17998223 1.55214734 4.04798724 3.29148799 5.99609375 5.125 C15.61305975 14.12333076 29.54865235 16.81360307 42.4140625 16.38671875 C54.81967056 15.04885906 64.88673542 11.82351372 74.0625 3.1875 C76.88647065 0.65146361 77.81089458 0.03525694 81.6875 -0.6875 C85.63155135 0.13107669 87.34854768 0.95862822 90 4 C91.07391829 7.69159412 91.43917338 9.90206656 90 13.5 C79.79943888 26.2507014 66.94723935 31.68763109 51.125 34.75 C49.75243889 34.86681371 48.37636909 34.94362994 47 35 C45.85273438 35.0515625 44.70546875 35.103125 43.5234375 35.15625 C25.3809426 35.69912823 7.93268102 32.36318928 -6 20 C-6.61101562 19.49339844 -7.22203125 18.98679687 -7.8515625 18.46484375 C-10.69618559 15.93703887 -11.96915172 14.10343246 -13.0625 10.4375 C-12.99232214 6.57771774 -12.39017026 4.98771282 -10 2 C-6.15691236 -0.56205842 -4.5659696 -0.67366765 0 0 Z " fill="#ffffff" transform="translate(217,324)"/>
                    </svg>
                    `;
                    const tempDiv = document.createElement('div');
                    tempDiv.innerHTML = aiSvgIconString;
                    const svgElement = tempDiv.querySelector('svg'); // Changed from tempDiv.firstChild to reliably get the SVG element

                    if (svgElement) {
                        // Prepend SVG so it's visually on top of the (hidden) fallback span
                        avatarDiv.insertBefore(svgElement, fallbackSpan);
                    } else {
                        // SVG creation failed, show fallback
                        fallbackSpan.style.display = 'flex';
                    }
                }

                const messageContentDiv = document.createElement('div');
                messageContentDiv.classList.add('chat-bubble-message');
                  if (message.isLoading) {
                    messageContentDiv.classList.add('loading');
                    messageContentDiv.innerHTML = `
                        <div class="loading-indicator">
                            <div class="typing-indicator">
                                <span></span>
                                <span></span>
                                <span></span>
                            </div>
                        </div>`;
                } else {
                    messageContentDiv.textContent = message.content;
                }

                bubbleDiv.appendChild(avatarDiv);
                bubbleDiv.appendChild(messageContentDiv);
                return bubbleDiv;
            };            // Keep track of rendered message IDs to avoid duplicates
            let renderedMessageIds = new Set();
            
            const renderMessages = () => {
                // Get all currently displayed message IDs
                document.querySelectorAll('.chat-bubble[data-message-id]').forEach(el => {
                    const id = el.dataset.messageId;
                    if (id && id !== 'loading') renderedMessageIds.add(id);
                });
                
                // First, remove any loading indicators if they exist
                const existingLoadingIndicators = document.querySelectorAll('.chat-bubble[data-message-id="loading"]');
                existingLoadingIndicators.forEach(el => el.remove());
                
                // Only render messages that haven't been rendered yet
                messages.forEach((message, index) => {
                    // Skip already rendered messages
                    if (renderedMessageIds.has(message.id.toString())) return;
                    
                    const messageElement = createMessageBubbleElement(message);
                    messageElement.style.animationDelay = `0.1s`;
                    messageItemsContainer.appendChild(messageElement);
                    renderedMessageIds.add(message.id.toString());
                });

                // Add loading indicator if needed
                if (isLoading) {
                    const loadingBubble = { id: 'loading', sender: 'ai', isLoading: true };
                    const loadingElement = createMessageBubbleElement(loadingBubble);
                    loadingElement.style.animationDelay = `0.1s`;
                    messageItemsContainer.appendChild(loadingElement);
                }
                
                setTimeout(() => {
                    if (autoScrollEnabled) {
                        scrollToBottom(true);
                    }
                    updateScrollToBottomButtonVisibility();
                }, 0);
            };

            if (chatForm) chatForm.addEventListener('submit', async (e) => {
    e.preventDefault();
    const queryContent = chatInput.value.trim();
    if (!queryContent) return;

    console.log("Form submitted. Query:", queryContent);

    // Add user message to the array and render
    const userMessage = { 
        id: Date.now(), // Unique ID for the user message
        content: queryContent, 
        sender: 'user' 
    };
    messages.push(userMessage);
    renderMessages(); // Render user message immediately

    // Clear input and set loading state
    chatInput.value = '';
    chatInput.style.height = 'auto'; // Reset height after clearing
    isLoading = true;
    autoScrollEnabled = true; // Ensure auto-scroll for AI response
    updateSubmitButtonUI();
    renderMessages(); // This call will now correctly show the loading indicator

    // Determine active documents for context
    const activeDocElements = document.querySelectorAll('.documents-list .document-item.active');
    let activeDocumentNames = [];
    if (activeDocElements && activeDocElements.length > 0) {
        activeDocElements.forEach(el => {
            const docNameDiv = el.querySelector('.document-name');
            if (docNameDiv && docNameDiv.textContent) {
                activeDocumentNames.push(docNameDiv.textContent.trim());
            }
        });
        console.log("Frontend: Using actively clicked documents for context:", activeDocumentNames);
    } else {
        // Fallback to all processed documents from localStorage if no specific one is active
        const storedDocsJson = localStorage.getItem('activeDocs');
        if (storedDocsJson) {
            try {
                activeDocumentNames = JSON.parse(storedDocsJson);
                if (!Array.isArray(activeDocumentNames)) activeDocumentNames = []; // Ensure it's an array
            } catch (err) {
                console.error("Error parsing activeDocs from localStorage for query:", err);
                activeDocumentNames = []; // Default to empty if parsing fails
            }
        }
        console.log("Frontend: No active click, using documents from localStorage for context (if any):", activeDocumentNames);
    }
    // If activeDocumentNames is still empty here, the backend will query all available processed collections.

    console.log("Frontend: Sending query to /ask with model:", selectedModel, "and documents:", activeDocumentNames);

    try {
        console.log("Frontend: Attempting fetch to /ask endpoint...");
        const response = await fetch('/ask', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({
                query: queryContent,
                model: selectedModel, 
                active_documents: activeDocumentNames 
            }),
        });
        console.log("Frontend: Fetch response received, status:", response.status);

        const result = await response.json(); // Attempt to parse JSON response
        console.log("Frontend: Parsed backend response:", result);

        if (!response.ok) { // Check if response status is not 2xx
            console.error('Ask request failed from backend with status ' + response.status + ':', result);
            messages.push({
                id: Date.now() + 1, // Unique ID for the error message
                content: `Error from AI: ${result.error || "Could not get an answer. Status: " + response.status}`,
                sender: 'ai',
            });
        } else {
            // Successful response from backend
            messages.push({
                id: Date.now() + 1, 
                content: result.answer_text || "AI returned an empty answer.", // This will now be the clean text
                sender: 'ai',
                citations_data: result.parsed_citations, // Parsed citations from backend
                raw_answer_with_citations: result.raw_answer_with_citations, // Optional, if backend sends it
                summarization_chain_used: result.summarization_chain_used
            });
        }
    } catch (error) {
        // This catches network errors or if response.json() fails (e.g., if backend sent non-JSON error page)
        console.error('Frontend: Network error or other issue during /ask fetch:', error);
        messages.push({
            id: Date.now() + 1, // Unique ID for the network error message
            content: 'Network error: Could not reach the AI service. Please check your connection and try again.',
            sender: 'ai',
        });
    } finally {
        isLoading = false;
        updateSubmitButtonUI();
        renderMessages(); // Render AI response or error message
    }
});

            chatInput.addEventListener('keydown', function(e) {
                if (e.key === 'Enter' && !e.shiftKey) {
                    e.preventDefault();
                    chatForm.dispatchEvent(new Event('submit', { cancelable: true, bubbles: true }));
                }
            });

            renderMessages();
            scrollToBottom(false); 
            autoScrollEnabled = true;
            updateScrollToBottomButtonVisibility();
            updateSubmitButtonUI();

            // === START: Pixel Sparks ===
            const sparksContainer = document.getElementById('pixel-sparks-container');
            const numSparks = 70; // Adjust number of sparks as desired
            const sparkColors = ['spark-blue', 'spark-purple', 'spark-rose', '']; // Last one is default white

            if (sparksContainer) {
                // Create initial sparks
                const createSpark = () => {
                    const spark = document.createElement('div');
                    spark.classList.add('spark');

                    // More variable sizes
                    const sizeRoll = Math.random();
                    if (sizeRoll > 0.92) spark.classList.add('spark-large');
                    else if (sizeRoll > 0.7) spark.classList.add('spark-medium');

                    // Random color
                    const colorClass = sparkColors[Math.floor(Math.random() * sparkColors.length)];
                    if (colorClass) spark.classList.add(colorClass);

                    // More random position with better distribution
                    spark.style.left = `${Math.random() * 110 - 5}%`;
                    spark.style.top = `${Math.random() * 110 - 5}%`;
                    
                    // Avoid center area where chat is (but with some randomness to the boundary)
                    const leftPos = parseFloat(spark.style.left);
                    const topPos = parseFloat(spark.style.top);
                    if ((leftPos > 14.5 && leftPos < 64) && (topPos > 8 && topPos < 92)) {
                        // Randomly place at edges instead of center
                        if (Math.random() > 0.5) {
                            spark.style.left = Math.random() > 0.5 ? `${(Math.random()*100)%15}%` : '64%';
                        } else {
                            spark.style.top = Math.random() > 0.5 ? '0%' : '100%';
                        }
                    }
                    
                    // Much more variable timing
                    spark.style.setProperty('--spark-duration', `${Math.random() * 8 + 1}s`); // 1s to 9s
                    spark.style.setProperty('--spark-delay', `${Math.random() * 10}s`); // 0s to 10s delay
                    spark.style.setProperty('--spark-opacity', `${Math.random() * 0.4 + 0.05}`); // 0.05 to 0.45 opacity
                    
                    // Random starting point in animation
                    spark.style.animationDelay = `-${Math.random() * 20}s`;
                    
                    return spark;
                };

                // Create initial batch of sparks
                for (let i = 0; i < numSparks; i++) {
                    sparksContainer.appendChild(createSpark());
                }
                
                // Periodically add/remove sparks to make pattern less predictable
                setInterval(() => {
                    // Remove a random spark sometimes
                    if (Math.random() > 0.7 && sparksContainer.children.length > numSparks/2) {
                        const randomIndex = Math.floor(Math.random() * sparksContainer.children.length);
                        if (sparksContainer.children[randomIndex]) {
                            sparksContainer.children[randomIndex].remove();
                        }
                    }
                    
                    // Add a new spark sometimes
                    if (Math.random() > 0.4 && sparksContainer.children.length < numSparks*1.5) {
                        sparksContainer.appendChild(createSpark());
                    }
                }, 2000);
            }
            // === END: Pixel Sparks ===
        });    </script>    <!-- Documents functionality -->    <script src="/static/js/documents.js"></script>
    <!-- Enhanced chat functionality -->
    <script src="/static/js/chat.js"></script>
    
    <script>
        document.addEventListener('DOMContentLoaded', () => {
            // --- DOM Elements (keep existing) ---
            const messageItemsContainer = document.getElementById('messageItemsContainer');
            const chatForm = document.getElementById('chatForm');
            const chatInput = document.getElementById('chatInput');
            const messageList = document.getElementById('messageList');
            const scrollToBottomButton = document.getElementById('scrollToBottomButton');
            const modelSelect = document.getElementById('modelSelect');
            const sendButton = document.getElementById('sendButton');
            const arrowUpIcon = document.getElementById('arrowUpIcon');
            const squareIcon = document.getElementById('squareIcon');
            const submitButtonTooltipWrapper = document.getElementById('submitButtonTooltipWrapper');
            // No need for document panel elements here, they are handled by documents.js

            // --- Initial messages data (keep or modify as desired) ---
            let messages = [
                { id: Date.now() - 2, content: "Hello! I'm DocuMind AI. How can I help you with your documents today?", sender: "ai" },
                // { id: Date.now() -1, content: "I have a question about the component library.", sender: "user" },
                // { id: Date.now(), content: "Sure! I'd be happy to help. What would you like to know?", sender: "ai" },
            ];
            let isLoading = false;
            let autoScrollEnabled = true;
            let selectedModel = "gemini-1.5-flash-latest"; // Default to a model your backend supports
            const MAX_TEXTAREA_HEIGHT = 240;

            const updateSubmitButtonUI = () => { /* ... keep existing ... */ 
                if (isLoading) { arrowUpIcon.classList.add('hidden'); squareIcon.classList.remove('hidden'); sendButton.disabled = true; if (submitButtonTooltipWrapper) { submitButtonTooltipWrapper.setAttribute('data-tooltip', 'Stop generation'); submitButtonTooltipWrapper.setAttribute('title', 'Stop generation'); }
                } else { arrowUpIcon.classList.remove('hidden'); squareIcon.classList.add('hidden'); sendButton.disabled = false; if (submitButtonTooltipWrapper) { submitButtonTooltipWrapper.setAttribute('data-tooltip', 'Send message'); submitButtonTooltipWrapper.setAttribute('title', 'Send message'); } }
            };
            
            if(chatInput) chatInput.addEventListener('input', () => { /* ... keep existing for auto-resize ... */ 
                chatInput.style.height = 'auto'; const newHeight = Math.min(chatInput.scrollHeight, MAX_TEXTAREA_HEIGHT); chatInput.style.height = `${newHeight}px`;
            });            
            
            if(modelSelect) modelSelect.addEventListener('change', (event) => { /* ... keep existing ... */
                selectedModel = event.target.value; console.log(`Model changed to: ${selectedModel}`);
                // IMPORTANT: Currently, your backend Flask app is hardcoded to use GENERATION_MODEL_NAME.
                // To make this dropdown truly effective, the selectedModel would need to be passed to the
                // /ask endpoint, and the backend would need logic to instantiate/use different models.
                // For now, this dropdown is cosmetic on the frontend regarding actual model use by backend.
            });
            if(modelSelect) selectedModel = modelSelect.value; // Initialize

            const isScrolledToBottom = () => { /* ... keep existing ... */ return messageList.scrollHeight - messageList.clientHeight <= messageList.scrollTop + 20; };
            const updateScrollToBottomButtonVisibility = () => { /* ... keep existing ... */ if (!isScrolledToBottom() && messageList.scrollHeight > messageList.clientHeight) { scrollToBottomButton.classList.add('visible'); } else { scrollToBottomButton.classList.remove('visible'); }};
            const scrollToBottom = (smooth = true) => { /* ... keep existing ... */ if(messageList) messageList.scrollTo({top: messageList.scrollHeight, behavior: smooth ? 'smooth':'auto'}); autoScrollEnabled = true; updateScrollToBottomButtonVisibility();};
            if(messageList) messageList.addEventListener('scroll', () => { /* ... keep existing ... */ if (!isScrolledToBottom() && messageList.scrollTop > 0) { autoScrollEnabled = false;} else if (isScrolledToBottom()) { autoScrollEnabled = true;} updateScrollToBottomButtonVisibility();});
            const disableAutoScrollOnInteraction = () => { /* ... keep existing ... */ if (!isScrolledToBottom()) { autoScrollEnabled = false; updateScrollToBottomButtonVisibility();}};
            if(messageList) messageList.addEventListener('wheel', disableAutoScrollOnInteraction);
            if(messageList) messageList.addEventListener('touchmove', disableAutoScrollOnInteraction);
            if(scrollToBottomButton) scrollToBottomButton.addEventListener('click', () => scrollToBottom(true));
            
            const createMessageBubbleElement = (message) => { /* ... keep your existing comprehensive function ... */ 
                const bubbleDiv = document.createElement('div'); bubbleDiv.classList.add('chat-bubble', message.sender === 'user' ? 'sent' : 'received'); bubbleDiv.dataset.messageId = message.id;
                const avatarDiv = document.createElement('div'); avatarDiv.classList.add('chat-bubble-avatar'); const fallbackTextContent = message.sender === 'user' ? 'US' : 'AI';
                const fallbackSpan = document.createElement('span'); fallbackSpan.classList.add('fallback-text'); fallbackSpan.textContent = fallbackTextContent; fallbackSpan.style.display = 'none'; avatarDiv.appendChild(fallbackSpan);
                if (message.sender === 'user') { const userAvatarSrc = "https://images.unsplash.com/photo-1534528741775-53994a69daeb?w=64&h=64&q=80&crop=faces&fit=crop"; if (userAvatarSrc) { const img = document.createElement('img'); img.src = userAvatarSrc; img.alt = 'User avatar'; img.onerror = () => { img.style.display = 'none'; fallbackSpan.style.display = 'flex'; }; avatarDiv.insertBefore(img, fallbackSpan); } else { fallbackSpan.style.display = 'flex'; }
                } else { const aiSvgIconString = `<svg version="1.1" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 512 512"><path d="M0 0 C9.69242413 6.95288386 16.11995831 16.46391612 18.3125 28.3125 C19.20255131 42.86141557 16.76980464 53.93664943 7.56640625 65.5078125 C1.29219651 72.36900652 -6.61295146 76.49759029 -15.6875 78.3125 C-15.6875 95.1425 -15.6875 111.9725 -15.6875 129.3125 C-11.16675781 129.29155273 -6.64601563 129.27060547 -1.98828125 129.24902344 C12.98728159 129.18259503 27.96285042 129.13829701 42.93852234 129.10525131 C52.0137792 129.08469022 61.08889575 129.05669417 70.1640625 129.01074219 C78.08073434 128.97067559 85.99730447 128.94502039 93.9140749 128.93614602 C98.09985774 128.93096456 102.28534984 128.91892363 106.47103882 128.88957977 C110.42405159 128.86209595 114.37665844 128.85412022 118.32975769 128.86006927 C119.76791096 128.85898356 121.20608017 128.85122813 122.64414978 128.8356781 C141.27513734 128.64438222 157.31037554 133.04632072 171.09887695 146.0065918 C182.50888799 157.19497629 189.18496563 172.04000378 189.53955078 188.11962891 C189.53924116 189.7594742 189.52837038 191.39934604 189.5078125 193.0390625 C189.50356766 194.34960098 189.50356766 194.34960098 189.49923706 195.68661499 C189.48817303 198.43705814 189.46309581 201.18715774 189.4375 203.9375 C189.42745234 205.81900567 189.41832964 207.70051651 189.41015625 209.58203125 C189.38827912 214.15898357 189.35384719 218.73568385 189.3125 223.3125 C190.74468872 223.29292236 190.74468872 223.29292236 192.20581055 223.27294922 C193.46176025 223.26279785 194.71770996 223.25264648 196.01171875 223.2421875 C197.87703247 223.22260986 197.87703247 223.22260986 199.7800293 223.20263672 C208.92942354 223.4871918 216.38261053 227.84273355 223.0703125 233.90625 C231.40892456 242.78959551 231.91870649 253.21747817 231.84375 264.8125 C231.84645171 266.2988307 231.85033974 267.78515966 231.85534668 269.27148438 C231.86120996 272.37126664 231.85278259 275.47058466 231.83398438 278.5703125 C231.81119598 282.51758796 231.82433633 286.46377508 231.84827423 290.41101074 C231.86281899 293.47566738 231.8580869 296.54007154 231.84776306 299.60473633 C231.84511964 301.05900201 231.8482803 302.5132916 231.85768127 303.9675293 C231.92229808 316.76816484 229.79235563 325.67021825 220.828125 335.109375 C213.78465388 341.27788076 206.30850289 343.47926685 197.125 343.375 C196.37541016 343.37048828 195.62582031 343.36597656 194.85351562 343.36132812 C193.00647617 343.34956354 191.15947787 343.33162865 189.3125 343.3125 C189.3439209 345.04983398 189.3439209 345.04983398 189.37597656 346.82226562 C189.44942568 351.1982747 189.49467335 355.57423173 189.53222656 359.95068359 C189.55210682 361.83000914 189.57922087 363.70927483 189.61425781 365.58837891 C189.98675596 386.10864428 187.31595348 403.5897168 172.578125 419.03515625 C171.83046875 419.78667969 171.0828125 420.53820312 170.3125 421.3125 C169.70921875 421.95058594 169.1059375 422.58867187 168.484375 423.24609375 C164.42061523 427.21267666 159.95800211 429.70827385 154.875 432.125 C154.26640137 432.41528076 153.65780273 432.70556152 153.03076172 433.00463867 C145.44486995 436.41698102 137.89366341 437.6159969 129.62207031 437.55732727 C128.38274435 437.56072413 128.38274435 437.56072413 127.1183815 437.56418961 C124.37571546 437.56944711 121.6332795 437.56068588 118.890625 437.55200195 C116.91377694 437.55311589 114.93692921 437.55517534 112.96008301 437.55810547 C108.71551669 437.56281498 104.47100575 437.56172089 100.22644043 437.55632973 C93.51062045 437.54870193 86.79490651 437.55939525 80.07910156 437.5735321 C60.98632375 437.6112221 41.89359463 437.62945488 22.80078125 437.61791992 C12.24642294 437.61170072 1.69223644 437.6232956 -8.86207962 437.65436524 C-15.53416942 437.67300013 -22.20577178 437.66971995 -28.87785339 437.65019763 C-33.03236651 437.64330603 -37.18661641 437.65729205 -41.34107971 437.67714691 C-43.26269738 437.68218499 -45.1843596 437.67873313 -47.10594177 437.66601372 C-60.74764239 437.36410688 -60.74764239 437.36410688 -72.6875 443.3125 C-74.1062087 444.24315187 -75.52939193 445.16704146 -76.95825195 446.08203125 C-78.44993142 447.09266955 -79.93899327 448.10717932 -81.42578125 449.125 C-82.24884766 449.68345108 -83.07191406 450.24190216 -83.91992188 450.817276 C-85.69245915 452.02002931 -87.46396557 453.22430299 -89.23452759 454.42996216 C-93.99375952 457.66971033 -98.76193927 460.89620726 -103.53125 464.12109375 C-104.51958328 464.78942734 -105.50791656 465.45776093 -106.52619934 466.14634705 C-117.62781645 473.64119013 -128.80813969 481.01585282 -140 488.375 C-141.42706772 489.31339722 -141.42706772 489.31339722 -142.88296509 490.27075195 C-146.38709105 492.57440546 -149.89133395 494.8778749 -153.39746094 497.17848206 C-154.51229858 497.91009804 -154.51229858 497.91009804 -155.6496582 498.65649414 C-156.34173874 499.10993698 -157.03381927 499.56337982 -157.74687195 500.03056335 C-159.54131919 501.21593614 -161.31901311 502.42326369 -163.09619141 503.63427734 C-167.07771692 506.21285341 -169.01997804 505.97928885 -173.6875 505.3125 C-176.8547802 502.8140388 -178.48422626 501.32434978 -179.28877258 497.31950378 C-179.27766953 496.21141861 -179.26656647 495.10333344 -179.25512695 493.96166992 C-179.2547493 492.7007402 -179.25437164 491.43981049 -179.25398254 490.14067078 C-179.22845324 488.77475994 -179.20237333 487.40885931 -179.17578125 486.04296875 C-179.16644859 484.63765218 -179.15933461 483.23231928 -179.15434265 481.82698059 C-179.13530772 478.13911541 -179.08623971 474.45224449 -179.03082275 470.76477051 C-178.97955304 466.99761402 -178.9568123 463.23029589 -178.93164062 459.46289062 C-178.87811537 452.0789752 -178.79284782 444.69587724 -178.6875 437.3125 C-179.30183899 437.31725845 -179.91617798 437.32201691 -180.5491333 437.32691956 C-181.35695251 437.33062057 -182.16477173 437.33432159 -182.99707031 437.33813477 C-183.7970343 437.34289322 -184.59699829 437.34765167 -185.42120361 437.35255432 C-200.02114966 437.09451617 -212.80354025 428.32271471 -222.6875 418.3125 C-223.24953125 417.77238281 -223.8115625 417.23226563 -224.390625 416.67578125 C-238.55216241 401.88070138 -239.13931666 382.16545145 -238.875 363 C-238.85991845 361.08399495 -238.84623622 359.16797836 -238.83398438 357.25195312 C-238.80127091 352.60517831 -238.74968253 347.95897005 -238.6875 343.3125 C-239.7801416 343.32917725 -240.8727832 343.34585449 -241.99853516 343.36303711 C-243.44954072 343.37652665 -244.90055013 343.38960724 -246.3515625 343.40234375 C-247.42869507 343.42041077 -247.42869507 343.42041077 -248.52758789 343.43884277 C-258.70466417 343.50575894 -265.16965762 339.72749604 -272.484375 332.7578125 C-278.51872414 326.06011292 -280.78959696 319.66963039 -280.86450195 310.77490234 C-280.87107315 310.12713837 -280.87764435 309.47937439 -280.88441467 308.8119812 C-280.90407727 306.67413193 -280.91579725 304.53635284 -280.92578125 302.3984375 C-280.92985678 301.66437305 -280.9339323 300.93030861 -280.93813133 300.17399979 C-280.95896665 296.28511083 -280.97327675 292.39625685 -280.98266602 288.50732422 C-280.99369805 284.51291696 -281.02806739 280.51907173 -281.06783772 276.52486229 C-281.09413086 273.4345301 -281.10236923 270.34432898 -281.10594749 267.25389481 C-281.11078431 265.78284555 -281.12231432 264.31180099 -281.14098549 262.84086227 C-281.28605064 250.65039031 -279.62190064 241.42439506 -271.0390625 232.359375 C-263.22603024 225.55866253 -255.55821798 223.15482974 -245.375 223.25 C-244.12074219 223.25902344 -242.86648437 223.26804688 -241.57421875 223.27734375 C-240.14529297 223.29474609 -240.14529297 223.29474609 -238.6875 223.3125 C-238.70321045 222.17409668 -238.7189209 221.03569336 -238.73510742 219.86279297 C-238.79008475 215.57250256 -238.8240924 211.28224214 -238.85229492 206.99169922 C-238.86722913 205.14724463 -238.88758395 203.30282457 -238.91381836 201.45849609 C-239.1987301 180.88898562 -236.68806188 162.84723687 -221.94506836 147.37426758 C-208.70730082 134.14099733 -193.19268212 129.10278348 -174.79718018 129.06365967 C-173.39782618 129.07073472 -171.99847317 129.07800429 -170.59912109 129.08544922 C-169.10012825 129.0860144 -167.60113486 129.08569468 -166.10214233 129.08456421 C-162.06149167 129.08399387 -158.02096142 129.09569608 -153.98033857 129.10971212 C-149.74695933 129.12227309 -145.51357688 129.12339419 -141.28018188 129.12576294 C-133.27647988 129.13196201 -125.27283297 129.14835141 -117.26915514 129.16846192 C-108.15181305 129.19087005 -99.03446824 129.20182667 -89.91710544 129.21185279 C-71.17386611 129.23271914 -52.43070224 129.26937557 -33.6875 129.3125 C-33.6875 112.4825 -33.6875 95.6525 -33.6875 78.3125 C-36.9875 77.3225 -40.2875 76.3325 -43.6875 75.3125 C-46.97873068 73.65497717 -49.79189136 71.58568809 -52.6875 69.3125 C-53.30496094 68.83296875 -53.92242187 68.3534375 -54.55859375 67.859375 C-61.82763389 61.44837301 -67.26221897 50.50347625 -67.92578125 40.84375 C-68.30840151 26.98841543 -65.32576318 15.5475025 -55.6875 5.15625 C-40.32345499 -9.54933594 -17.82502146 -10.84607238 0 0 Z M-43.5625 19.0625 C-48.36226145 25.0983949 -50.37910454 30.99022683 -50.09375 38.73046875 C-48.90234047 46.3027928 -44.78354451 52.20910873 -38.84765625 56.90234375 C-32.59175535 60.98621396 -26.70462848 61.91373614 -19.328125 60.9453125 C-12.04105407 59.1990026 -6.87543848 54.35304678 -2.6875 48.3125 C0.95796947 41.8721706 1.3801024 35.81120194 -0.30859375 28.6953125 C-3.02429084 20.75340723 -7.87486716 16.58541573 -14.9375 12.375 C-25.6956105 9.32686869 -35.7676854 10.79576552 -43.5625 19.0625 Z M-211.8984375 163.890625 C-218.58798582 172.94634543 -221.00588344 182.63944449 -220.94819641 193.75495911 C-220.95202196 194.5545584 -220.95584751 195.3541577 -220.95978898 196.17798728 C-220.97002567 198.83072245 -220.96614473 201.48323738 -220.9621582 204.13598633 C-220.96650989 206.04885557 -220.97169455 207.96172308 -220.97764587 209.87458801 C-220.99099891 215.05240136 -220.99161859 220.23014155 -220.98908257 225.40796876 C-220.98801022 229.73922232 -220.99290306 234.07046123 -220.997688 238.40171164 C-221.0087707 248.62532223 -221.00922774 258.84889608 -221.00317383 269.07250977 C-220.99713624 279.59880892 -221.00944248 290.12497432 -221.0307439 300.65125066 C-221.04839206 309.70717969 -221.05435613 318.76306628 -221.05110615 327.81901187 C-221.04929807 333.21952019 -221.05188185 338.61993313 -221.06582069 344.0204258 C-221.07844656 349.10207583 -221.07641006 354.18351828 -221.06340981 359.26516533 C-221.06105471 361.12263689 -221.06374014 362.98012328 -221.07203293 364.83757782 C-221.13981382 381.20899075 -219.56771497 394.80387152 -208.1875 407.5 C-199.32893227 415.78704723 -189.76944985 418.72330915 -178.05078125 419.8671875 C-172.24095985 420.63643215 -168.33792177 422.57744122 -164.25 426.875 C-162.04516398 430.3145442 -161.55719212 432.97033412 -161.48217773 436.9675293 C-161.4522599 438.37865242 -161.4522599 438.37865242 -161.42173767 439.81828308 C-161.39666893 441.33715874 -161.39666893 441.33715874 -161.37109375 442.88671875 C-161.35031265 443.92568802 -161.32953156 444.96465729 -161.30812073 446.03511047 C-161.24269344 449.35667253 -161.18377179 452.67831416 -161.125 456 C-161.08180831 458.2506593 -161.03819381 460.50131052 -160.99414062 462.75195312 C-160.88701902 468.2720511 -160.78523984 473.79222837 -160.6875 479.3125 C-154.38402811 475.94578867 -148.42422768 472.2505775 -142.52734375 468.21875 C-141.67146667 467.63837982 -140.8155896 467.05800964 -139.93377686 466.46005249 C-138.10647761 465.22077495 -136.28021788 463.97996348 -134.45492554 462.73773193 C-129.65854418 459.47469677 -124.85273611 456.2256131 -120.046875 452.9765625 C-119.08549118 452.32640167 -118.12410736 451.67624084 -117.1335907 451.00637817 C-108.93088909 445.46470835 -100.69656078 439.97263678 -92.42459106 434.53491211 C-91.06950202 433.64406934 -89.71521864 432.7519997 -88.36178589 431.85864258 C-86.49743611 430.62871736 -84.63015406 429.40339463 -82.76171875 428.1796875 C-81.20843872 427.16044189 -81.20843872 427.16044189 -79.6237793 426.12060547 C-72.29908992 421.61019976 -67.09627671 420.16184261 -58.57383728 420.18119812 C-57.33691087 420.17802455 -57.33691087 420.17802455 -56.07499605 420.17478687 C-53.30771519 420.16878843 -50.54047994 420.16978692 -47.77319336 420.1706543 C-45.79047569 420.16762261 -43.80775872 420.16410113 -41.82504272 420.16012573 C-37.55203613 420.1522357 -33.2790379 420.14709136 -29.00602531 420.14383507 C-22.24386417 420.1379231 -15.48175561 420.12137992 -8.71961975 420.10249329 C-6.40383438 420.09615576 -4.08804897 420.08983013 -1.77226353 420.08351612 C-0.03250506 420.0787674 -0.03250506 420.0787674 1.74240005 420.07392275 C17.48540249 420.0318656 33.22838068 419.99888331 48.97143555 419.98657227 C59.58986138 419.97818335 70.2081572 419.95566038 80.826518 419.9172166 C86.44338381 419.89748143 92.06005539 419.88504293 97.67695808 419.89005089 C102.9654513 419.89470847 108.25357686 419.88083104 113.541996 419.85303879 C115.47727311 419.84614531 117.41258838 419.84598049 119.34786415 419.85323906 C135.56659671 419.90790039 147.5874089 417.35068368 160 406.1875 C168.62436066 397.12189298 171.63632532 385.0341323 171.57319641 372.87004089 C171.57702196 372.0704416 171.58084751 371.2708423 171.58478898 370.44701272 C171.59502567 367.79427755 171.59114473 365.14176262 171.5871582 362.48901367 C171.59150989 360.57614443 171.59669455 358.66327692 171.60264587 356.75041199 C171.61599891 351.57259864 171.61661859 346.39485845 171.61408257 341.21703124 C171.61301022 336.88577768 171.61790306 332.55453877 171.622688 328.22328836 C171.6337707 317.99967777 171.63422774 307.77610392 171.62817383 297.55249023 C171.62213624 287.02619108 171.63444248 276.50002568 171.6557439 265.97374934 C171.67339206 256.91782031 171.67935613 247.86193372 171.67610615 238.80598813 C171.67429807 233.40547981 171.67688185 228.00506687 171.69082069 222.6045742 C171.70344656 217.52292417 171.70141006 212.44148172 171.68840981 207.35983467 C171.68605471 205.50236311 171.68874014 203.64487672 171.69703293 201.78742218 C171.76413862 185.57909434 170.34783781 172.36392251 159.0546875 159.80078125 C150.66554838 151.64092729 140.49051573 147.2001875 128.76725006 147.17827892 C127.65759211 147.17382108 126.54793416 147.16936324 125.40465021 147.16477031 C123.58381552 147.16504685 123.58381552 147.16504685 121.72619629 147.16532898 C120.43252687 147.16165229 119.13885745 147.1579756 117.80598593 147.1541875 C114.21667862 147.14418795 110.62738803 147.140454 107.03806877 147.13779819 C103.16601733 147.13388902 99.29397905 147.12431107 95.42193604 147.11558533 C86.07126482 147.09583752 76.72059256 147.08575653 67.36990571 147.07685754 C62.96341523 147.0724716 58.55692594 147.06710556 54.1504364 147.06186867 C39.49670918 147.04485756 24.84298271 147.03036444 10.18924713 147.02311993 C6.38788349 147.02121012 2.58651986 147.01928977 -1.21484375 147.01733398 C-2.1596217 147.01685024 -3.10439966 147.0163665 -4.07780725 147.0158681 C-19.37501263 147.00757172 -34.67213972 146.9822409 -49.96931094 146.94975331 C-65.68461603 146.91665059 -81.39988137 146.89866824 -97.1152215 146.89547569 C-105.93470777 146.89331349 -114.75407988 146.88460752 -123.5735321 146.85901451 C-131.08623996 146.83726354 -138.59880592 146.82927863 -146.11154169 146.83898129 C-149.94105181 146.8435134 -153.77026944 146.84170389 -157.59973907 146.82221985 C-161.76180544 146.80122758 -165.9232286 146.81161385 -170.08532715 146.82475281 C-171.28137117 146.81378356 -172.47741519 146.80281432 -173.70970297 146.79151267 C-189.26231195 146.90956036 -201.84062355 151.94947537 -211.8984375 163.890625 Z M-257.6875 245.3125 C-261.71506375 249.76401783 -262.81033663 252.19944396 -262.86450195 258.13574219 C-262.87733719 259.3575116 -262.89017242 260.57928101 -262.90339661 261.83807373 C-262.91156524 263.17850336 -262.91900874 264.51893759 -262.92578125 265.859375 C-262.92985678 266.53407082 -262.9339323 267.20876663 -262.93813133 267.90390778 C-262.9589944 271.47695103 -262.97329346 275.04995588 -262.98266602 278.62304688 C-262.99371432 282.30628739 -263.02809415 285.98892106 -263.06783772 289.67194748 C-263.09404157 292.50991911 -263.10236126 295.3477472 -263.10594749 298.18582916 C-263.11316184 300.20102044 -263.13958771 302.21611347 -263.16633606 304.23114014 C-263.23987317 313.10781492 -263.23987317 313.10781492 -258.6875 320.3125 C-251.38641321 326.74440979 -250.63307838 325.3125 -238.6875 325.3125 C-238.6875 297.5925 -238.6875 269.8725 -238.6875 241.3125 C-249.66800399 240.70606433 -249.66800399 240.70606433 -257.6875 245.3125 Z M189.3125 241.3125 C189.3125 269.0325 189.3125 296.7525 189.3125 325.3125 C200.65916932 326.2447746 200.65916932 326.2447746 208.3125 321.3125 C212.2897413 316.79779366 213.43503975 314.45809198 213.48950195 308.48925781 C213.50233719 307.2674884 213.51517242 306.04571899 213.52839661 304.78692627 C213.53656524 303.44649664 213.54400874 302.10606241 213.55078125 300.765625 C213.55485678 300.09092918 213.5589323 299.41623337 213.56313133 298.72109222 C213.5839944 295.14804897 213.59829346 291.57504412 213.60766602 288.00195312 C213.61871432 284.31871261 213.65309415 280.63607894 213.69283772 276.95305252 C213.71904157 274.11508089 213.72736126 271.2772528 213.73094749 268.43917084 C213.73816184 266.42397956 213.76458771 264.40888653 213.79133606 262.39385986 C213.8629274 253.518419 213.8629274 253.518419 209.3125 246.3125 C202.46302374 240.13126533 199.72084686 241.3125 189.3125 241.3125 Z " fill="#ffffff" transform="translate(280.6875,6.6875)"/> <path d="M0 0 C8.01586887 8.96766035 12.11674298 19.99215042 12 32 C11.27765722 43.55748448 6.73187961 55.11902282 -2 63 C-12.72410644 71.53382651 -24.00554579 74.74889761 -37.6953125 73.81640625 C-47.92175458 72.2216074 -58.15056194 66.76882572 -65 59 C-72.20229507 48.975849 -75.94477181 38.36792185 -75 26 C-72.99278251 13.86186591 -67.80272188 2.63529914 -57.609375 -4.7265625 C-38.52336687 -16.61140805 -16.99091466 -15.29975052 0 0 Z M-50 13 C-54.92754793 18.97903305 -57.54733009 25.20054619 -57 33 C-55.4582685 40.69622417 -52.16192735 46.68822182 -45.6875 51.25 C-39.4204617 55.13882996 -33.95508588 56.63858076 -26.62109375 55.65625 C-18.53606244 53.63197986 -13.32269957 49.04488116 -9.05859375 42.05078125 C-5.93409653 35.99778848 -5.76749484 28.01681733 -7.75390625 21.54296875 C-8.71298403 19.57591902 -9.7672641 17.8060012 -11 16 C-11.53625 15.113125 -12.0725 14.22625 -12.625 13.3125 C-23.68075714 2.54768384 -38.74570724 2.95004115 -50 13 Z " fill="#ffffff" transform="translate(390,234)"/><path d="M0 0 C8.3912395 7.99006471 13.0685797 18.49743062 14.1640625 30 C14.46993576 42.99961334 9.69170845 54.50474833 0.9765625 64.015625 C-2.63740311 67.33543062 -6.6828801 69.65100567 -11.0390625 71.8671875 C-11.67457031 72.1971875 -12.31007812 72.5271875 -12.96484375 72.8671875 C-22.00643597 76.89427782 -33.735812 77.04567714 -43.0390625 73.8671875 C-56.11249659 68.43254244 -64.12357014 61.54134061 -70.109375 48.54296875 C-74.0620671 37.85534983 -73.9768592 26.270799 -69.77734375 15.6875 C-64.31808586 4.19583378 -57.41386267 -3.0619801 -45.57421875 -8.203125 C-29.51263365 -13.85899941 -13.05373855 -10.98022051 0 0 Z M-49.0390625 15.8671875 C-53.62164076 22.02189114 -54.85563062 28.08715244 -54.515625 35.68359375 C-53.4822305 42.58700786 -50.71536458 47.876825 -45.2890625 52.3046875 C-38.76214389 56.60707579 -32.42952704 58.6860756 -24.59765625 57.57421875 C-17.91033874 55.72627299 -11.7420522 51.79197102 -8.0390625 45.8671875 C-7.4821875 44.9803125 -6.9253125 44.0934375 -6.3515625 43.1796875 C-3.64418937 36.34679341 -3.26061503 29.91472683 -5.97265625 23.03125 C-9.62327998 15.62302456 -14.26901368 10.87453198 -22.0390625 7.8671875 C-31.85735989 5.09394911 -42.07279163 8.57654481 -49.0390625 15.8671875 Z " fill="#ffffff" transform="translate(183.0390625,232.1328125)"/><path d="M0 0 C2.17998223 1.55214734 4.04798724 3.29148799 5.99609375 5.125 C15.61305975 14.12333076 29.54865235 16.81360307 42.4140625 16.38671875 C54.81967056 15.04885906 64.88673542 11.82351372 74.0625 3.1875 C76.88647065 0.65146361 77.81089458 0.03525694 81.6875 -0.6875 C85.63155135 0.13107669 87.34854768 0.95862822 90 4 C91.07391829 7.69159412 91.43917338 9.90206656 90 13.5 C79.79943888 26.2507014 66.94723935 31.68763109 51.125 34.75 C49.75243889 34.86681371 48.37636909 34.94362994 47 35 C45.85273438 35.0515625 44.70546875 35.103125 43.5234375 35.15625 C25.3809426 35.69912823 7.93268102 32.36318928 -6 20 C-6.61101562 19.49339844 -7.22203125 18.98679687 -7.8515625 18.46484375 C-10.69618559 15.93703887 -11.96915172 14.10343246 -13.0625 10.4375 C-12.99232214 6.57771774 -12.39017026 4.98771282 -10 2 C-6.15691236 -0.56205842 -4.5659696 -0.67366765 0 0 Z " fill="#ffffff" transform="translate(217,324)"/></svg>`;
                    const tempDiv = document.createElement('div'); tempDiv.innerHTML = aiSvgIconString;
                    const svgElement = tempDiv.querySelector('svg');
                    if (svgElement) { avatarDiv.insertBefore(svgElement, fallbackSpan); } else { fallbackSpan.style.display = 'flex'; }
                }
                const messageContentDiv = document.createElement('div'); messageContentDiv.classList.add('chat-bubble-message');
                if (message.isLoading) {
                    messageContentDiv.classList.add('loading');
                    messageContentDiv.innerHTML = `<div class="loading-indicator"><div class="typing-indicator"><span></span><span></span><span></span></div></div>`;
                } else { messageContentDiv.textContent = message.content; }
                bubbleDiv.appendChild(avatarDiv); bubbleDiv.appendChild(messageContentDiv); return bubbleDiv;
            };
            
            let renderedMessageIds = new Set();
            const renderMessages = () => { /* ... keep existing ... */ 
                document.querySelectorAll('.chat-bubble[data-message-id]').forEach(el => { const id = el.dataset.messageId; if (id && id !== 'loading') renderedMessageIds.add(id); });
                const existingLoadingIndicators = document.querySelectorAll('.chat-bubble[data-message-id="loading"]'); existingLoadingIndicators.forEach(el => el.remove());
                messages.forEach((message, index) => { if (renderedMessageIds.has(String(message.id))) return; const messageElement = createMessageBubbleElement(message); messageElement.style.animationDelay = `0.1s`; if(messageItemsContainer) messageItemsContainer.appendChild(messageElement); renderedMessageIds.add(String(message.id)); });
                if (isLoading && messageItemsContainer) { const loadingBubble = { id: 'loading', sender: 'ai', isLoading: true }; const loadingElement = createMessageBubbleElement(loadingBubble); loadingElement.style.animationDelay = `0.1s`; messageItemsContainer.appendChild(loadingElement); }
                setTimeout(() => { if (autoScrollEnabled) { scrollToBottom(true); } updateScrollToBottomButtonVisibility(); }, 0);
            };

            // --- MODIFIED: chatForm submit listener ---
            if(chatForm) chatForm.addEventListener('submit', async (e) => {
                e.preventDefault();
                const queryContent = chatInput.value.trim();
                if (!queryContent) return;

                const userMessage = { id: Date.now(), content: queryContent, sender: 'user' };
                messages.push(userMessage);
                renderMessages(); // Render user message

                chatInput.value = '';
                chatInput.style.height = 'auto'; // Reset height
                isLoading = true;
                updateSubmitButtonUI();
                renderMessages(); // Show loading indicator for AI

                // Determine active documents (original filenames) for context
                // This relies on documents.js adding 'active' class and data-id containing original filename
                // or a more robust way to get selected document names.
                const activeDocElements = document.querySelectorAll('.documents-list .document-item.active');
                let activeDocumentNames = [];
                if (activeDocElements && activeDocElements.length > 0) {
                    activeDocElements.forEach(el => {
                        const docNameDiv = el.querySelector('.document-name');
                        if (docNameDiv && docNameDiv.textContent) {
                            activeDocumentNames.push(docNameDiv.textContent.trim());
                        }
                    });
                } else {
                    // If no specific docs selected, get all from localStorage (processed ones)
                    const storedDocsJson = localStorage.getItem('activeDocs');
                    if (storedDocsJson) {
                        try { activeDocumentNames = JSON.parse(storedDocsJson); } catch (err) { console.error("Error parsing activeDocs from localStorage for query:", err); }
                    }
                }
                console.log("Sending query with active documents:", activeDocumentNames);


                try {
                    const response = await fetch('/ask', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({
                            query: queryContent,
                            model: selectedModel, // from modelSelect dropdown
                            active_documents: activeDocumentNames // list of original filenames
                        }),
                    });

                    const result = await response.json(); // Always try to parse, even for errors

                    if (!response.ok) {
                        console.error('Ask request failed:', result);
                        messages.push({
                            id: Date.now() + 1,
                            content: `Error from AI: ${result.error || "Could not get response."}`,
                            sender: 'ai',
                        });
                    } else {
                        messages.push({
                            id: Date.now() + 1,
                            content: result.answer_text,
                            sender: 'ai',
                            citations: result.parsed_citations, // For potential display
                            summarization_chain_used: result.summarization_chain_used
                        });
                    }
                } catch (error) {
                    console.error('Network error asking question:', error);
                    messages.push({
                        id: Date.now() + 1,
                        content: 'Network error: Could not reach the AI. Please try again.',
                        sender: 'ai',
                    });
                } finally {
                    isLoading = false;
                    updateSubmitButtonUI();
                    renderMessages(); // Render AI response or error
                }
            });      
            
            if(chatInput) chatInput.addEventListener('keydown', function(e) { /* ... keep existing ... */
                if (e.key === 'Enter' && !e.shiftKey) { e.preventDefault(); if(chatForm) chatForm.dispatchEvent(new Event('submit', { cancelable: true, bubbles: true }));}
            });

            // --- Initial UI Setup Calls ---
            renderMessages(); // Render initial messages
            if (messageList) scrollToBottom(false); // Scroll to bottom on load
            autoScrollEnabled = true;
            updateScrollToBottomButtonVisibility();
            updateSubmitButtonUI();

            // === START: Pixel Sparks (keep existing) ===
            const sparksContainer = document.getElementById('pixel-sparks-container');
            if (sparksContainer) { /* ... your full sparks logic ... */ 
                const numSparks = 70; const sparkColors = ['spark-blue', 'spark-purple', 'spark-rose', ''];
                const createSpark = () => { const spark = document.createElement('div'); spark.classList.add('spark'); const sizeRoll = Math.random(); if (sizeRoll > 0.92) spark.classList.add('spark-large'); else if (sizeRoll > 0.7) spark.classList.add('spark-medium'); const colorClass = sparkColors[Math.floor(Math.random() * sparkColors.length)]; if (colorClass) spark.classList.add(colorClass); spark.style.left = `${Math.random() * 110 - 5}%`; spark.style.top = `${Math.random() * 110 - 5}%`; const leftPos = parseFloat(spark.style.left); const topPos = parseFloat(spark.style.top); if ((leftPos > 14.5 && leftPos < 64) && (topPos > 8 && topPos < 92)) { if (Math.random() > 0.5) { spark.style.left = Math.random() > 0.5 ? `${(Math.random()*100)%15}%` : '64%'; } else { spark.style.top = Math.random() > 0.5 ? '0%' : '100%'; } } spark.style.setProperty('--spark-duration', `${Math.random() * 8 + 1}s`); spark.style.setProperty('--spark-delay', `${Math.random() * 10}s`); spark.style.setProperty('--spark-opacity', `${Math.random() * 0.4 + 0.05}`); spark.style.animationDelay = `-${Math.random() * 20}s`; return spark; };
                for (let i = 0; i < numSparks; i++) { sparksContainer.appendChild(createSpark()); }
                setInterval(() => { if (Math.random() > 0.7 && sparksContainer.children.length > numSparks/2) { const randomIndex = Math.floor(Math.random() * sparksContainer.children.length); if (sparksContainer.children[randomIndex]) { sparksContainer.children[randomIndex].remove(); } } if (Math.random() > 0.4 && sparksContainer.children.length < numSparks*1.5) { sparksContainer.appendChild(createSpark()); } }, 2000);
            }
            // === END: Pixel Sparks ===
        });    
    </script>
    <!-- External JS files - paths should be correct for Flask static serving -->
    <script src="/static/js/documents.js"></script> 
    <script src="/static/js/chat.js"></script>
    
    <!-- Flickering Grid Background Script (keep existing) -->
    <script>
        function createFlickeringGrid(options) { /* ... your full flickering grid function ... */ 
            const config = { squareSize: 4, gridGap: 6, flickerChance: 0.1, lightColor: '#6B7280', darkColor: '#4B5563', maxOpacity: 0.5, ...options };
            const canvas = document.getElementById(config.canvasId); const container = canvas.parentElement; if (!canvas || !container) { console.error("Canvas or container not found!"); return null; } const ctx = canvas.getContext('2d'); if (!ctx) { console.error("No 2D context"); return null; }
            let isInView = true; let animationFrameId = null; let gridParams = null; let lastTime = 0; let currentRGBAColor = '';
            const toRGBA = (color) => { const tempCanvas = document.createElement("canvas"); tempCanvas.width = tempCanvas.height = 1; const tempCtx = tempCanvas.getContext("2d"); if (!tempCtx) return "rgba(0,0,0,"; tempCtx.fillStyle = color; tempCtx.fillRect(0,0,1,1); const [r,g,b] = Array.from(tempCtx.getImageData(0,0,1,1).data); return `rgba(${r},${g},${b},`;};
            const updateSquareColor = () => { currentRGBAColor = toRGBA(config.darkColor); if (gridParams && ctx) { drawGrid(ctx, canvas.width, canvas.height, gridParams.cols, gridParams.rows, gridParams.squares, gridParams.dpr); }};
            const setupCanvas = (width, height) => { const dpr = window.devicePixelRatio || 1; canvas.width = width * dpr; canvas.height = height * dpr; canvas.style.width = `${width}px`; canvas.style.height = `${height}px`; const cols = Math.floor(width / (config.squareSize + config.gridGap)); const rows = Math.floor(height / (config.squareSize + config.gridGap)); const squares = new Float32Array(cols * rows); for (let i = 0; i < squares.length; i++) { squares[i] = Math.random() * config.maxOpacity; } return { cols, rows, squares, dpr }; };
            const updateSquares = (squares, deltaTime) => { const chanceThisFrame = config.flickerChance * deltaTime; for (let i = 0; i < squares.length; i++) { if (Math.random() < chanceThisFrame) { squares[i] = Math.random() * config.maxOpacity; } } };
            const drawGrid = (ctx, canvasWidth, canvasHeight, cols, rows, squares, dpr) => { ctx.clearRect(0, 0, canvasWidth, canvasHeight); for (let i = 0; i < cols; i++) { for (let j = 0; j < rows; j++) { const index = i * rows + j; const opacity = squares[index]; ctx.fillStyle = `${currentRGBAColor}${opacity})`; const x = i * (config.squareSize + config.gridGap) * dpr; const y = j * (config.squareSize + config.gridGap) * dpr; const size = config.squareSize * dpr; ctx.fillRect(x, y, size, size); } } };
            const animate = (time) => { if (!isInView || !gridParams) { animationFrameId = null; return; } const deltaTime = (time - (lastTime || time)) / 1000; lastTime = time; updateSquares(gridParams.squares, deltaTime); drawGrid(ctx, canvas.width, canvas.height, gridParams.cols, gridParams.rows, gridParams.squares, gridParams.dpr); animationFrameId = requestAnimationFrame(animate); };
            const handleResize = () => { const rect = container.getBoundingClientRect(); if(rect.width === 0 || rect.height === 0) return; gridParams = setupCanvas(rect.width, rect.height); if (gridParams && ctx) { drawGrid(ctx, canvas.width, canvas.height, gridParams.cols, gridParams.rows, gridParams.squares, gridParams.dpr); }};
            window.addEventListener('resize', handleResize);
            updateSquareColor(); handleResize(); lastTime = performance.now(); if (isInView) { animationFrameId = requestAnimationFrame(animate); }
            return { cleanup: () => { if (animationFrameId) cancelAnimationFrame(animationFrameId); window.removeEventListener('resize', handleResize); console.log("Flickering Grid cleaned up."); }};
        }
        document.addEventListener('DOMContentLoaded', () => {
            if (document.getElementById('chat-grid-canvas')) { // Check if canvas exists before initializing
                window.chatGridInstance = createFlickeringGrid({ canvasId: 'chat-grid-canvas', squareSize: 3, gridGap: 6, darkColor: '#4A5568', maxOpacity: 0.2, flickerChance: 0.1 });
                window.addEventListener('beforeunload', () => { if (window.chatGridInstance) window.chatGridInstance.cleanup(); });
            }
        });
    </script>
</body>
</html>
"""

hero_html_path = os.path.join(TEMPLATES_DIR, 'hero-geometric.html')
chat_html_path = os.path.join(TEMPLATES_DIR, 'chat.html')

with open(hero_html_path, 'w') as f:
    f.write(hero_html_content)
print(f"Created '{hero_html_path}'.")

with open(chat_html_path, 'w') as f:
    f.write(chat_html_content)
print(f"Created '{chat_html_path}'.")

# 3. Write basic JavaScript files instead of uploading
documents_js_content = """
// static/js/documents.js

document.addEventListener('DOMContentLoaded', () => {
    const documentsListContainer = document.getElementById('documentsList'); // The <ul> or <div> for doc items
    const emptyDocumentsStateDiv = document.getElementById('emptyDocumentsState');
    const chatInputField = document.getElementById('chatInput'); // To add [Document: ...] to
    const documentUploadButton = document.getElementById('documentUploadBtn'); // The upload button in the doc panel
    const documentFileInputHidden = document.getElementById('documentFileInput'); // Hidden file input

    // --- Helper to create SVG icon string based on file type ---
    function getFileIconSvg(fileExtension) {
        let iconPath = '<path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path><polyline points="14 2 14 8 20 8"></polyline>'; // Default
        switch (fileExtension.toLowerCase()) {
            case 'pdf':
                iconPath = '<path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path><polyline points="14 2 14 8 20 8"></polyline><line x1="16" y1="13" x2="8" y2="13"></line><line x1="16" y1="17" x2="8" y2="17"></line><polyline points="10 9 9 9 8 9"></polyline>';
                break;
            case 'docx':
            case 'doc':
                iconPath = '<path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path><polyline points="14 2 14 8 20 8"></polyline><line x1="16" y1="13" x2="8" y2="13"></line><line x1="16" y1="17" x2="8" y2="17"></line><polyline points="10 9 9 9 8 9"></polyline>';
                break;
            case 'xlsx':
            case 'xls':
            case 'csv':
                iconPath = '<rect x="3" y="3" width="18" height="18" rx="2" ry="2"></rect><line x1="3" y1="9" x2="21" y2="9"></line><line x1="9" y1="21" x2="9" y2="9"></line>';
                break;
            case 'pptx':
            case 'ppt':
                iconPath = '<path d="M2 3h20v18H2z"></path><path d="M12 3v18"></path>';
                break;
            // Add more cases as needed
        }
        return `<svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">${iconPath}</svg>`;
    }

    // --- Function to render the document list ---
    function renderDocumentList(documentNames) {
        if (!documentsListContainer) {
            console.error("Document list container not found!");
            return;
        }

        // Clear any existing static/placeholder items, BUT NOT the empty state div
        // We select only '.document-item' children to remove.
        const itemsToRemove = documentsListContainer.querySelectorAll('.document-item');
        itemsToRemove.forEach(item => item.remove());

        if (documentNames && documentNames.length > 0) {
            if (emptyDocumentsStateDiv) emptyDocumentsStateDiv.style.display = 'none';

            documentNames.forEach((docName, index) => {
                const fileNameParts = docName.split('.');
                const fileExt = fileNameParts.length > 1 ? fileNameParts.pop().toLowerCase() : 'file';
                
                const documentItem = document.createElement('div');
                documentItem.className = 'document-item';
                // Use a more robust data-id if you have unique IDs from backend, otherwise filename is okay for display
                documentItem.setAttribute('data-id', `doc-${index}-${docName.replace(/[^a-zA-Z0-9-_]/g, '')}`);
                documentItem.setAttribute('data-doctype', fileExt);
                documentItem.setAttribute('tabindex', '0'); // For accessibility

                documentItem.innerHTML = `
                    <div class="document-icon">
                        ${getFileIconSvg(fileExt)}
                    </div>
                    <div class="document-info">
                        <div class="document-name" title="${docName}">${docName}</div>
                        <div class="document-meta">
                            <span>${fileExt.toUpperCase()}</span>
                            <!-- Meta dot and size can be added if available -->
                            <!-- <span class="document-meta-dot"></span> -->
                            <!-- <span>File Size</span> -->
                        </div>
                    </div>
                `;
                
                // Attach event listener for click
                documentItem.addEventListener('click', () => handleDocumentItemClick(documentItem, docName));
                // Attach event listener for keyboard interaction
                documentItem.addEventListener('keydown', (e) => {
                    if (e.key === 'Enter' || e.key === ' ') {
                        e.preventDefault();
                        handleDocumentItemClick(documentItem, docName);
                    }
                });

                documentsListContainer.appendChild(documentItem);
            });
        } else {
            if (emptyDocumentsStateDiv) emptyDocumentsStateDiv.style.display = 'flex'; // Show empty state
        }
    }

    // --- Function to handle click on a document item ---
    function handleDocumentItemClick(itemElement, docName) {
        if (!documentsListContainer || !chatInputField) return;

        // Toggle 'active' class
        const allItems = documentsListContainer.querySelectorAll('.document-item');
        allItems.forEach(item => {
            if (item !== itemElement) item.classList.remove('active');
        });
        itemElement.classList.toggle('active');

        // Add reference to chat input
        const documentReference = `[Doc: ${docName}] `;
        if (itemElement.classList.contains('active')) { // Only add if activating
            if (!chatInputField.value.includes(documentReference)) {
                chatInputField.value = documentReference + chatInputField.value;
            }
        } else { // If deactivating, remove the reference (optional, can be complex)
            // chatInputField.value = chatInputField.value.replace(documentReference, '').trim();
        }
        chatInputField.focus();
        // Trigger input event for textarea auto-resize if you have that in chat.js
        chatInputField.dispatchEvent(new Event('input', { bubbles: true }));
    }

    // --- Load processed document names from localStorage ---
    function loadAndDisplayProcessedDocuments() {
        const storedDocsJson = localStorage.getItem('activeDocs');
        let processedDocumentNames = [];
        if (storedDocsJson) {
            try {
                processedDocumentNames = JSON.parse(storedDocsJson);
                if (!Array.isArray(processedDocumentNames)) {
                    processedDocumentNames = []; // Ensure it's an array
                }
            } catch (e) {
                console.error("Error parsing 'activeDocs' from localStorage:", e);
                processedDocumentNames = [];
            }
        }
        
        console.log("Processed documents loaded from localStorage:", processedDocumentNames);
        renderDocumentList(processedDocumentNames);
    }

    // --- Handle the upload button within the document panel ---
    // This would ideally trigger the same upload flow as the hero page,
    // which is complex to replicate fully without refactoring into shared modules.
    // For now, it can just trigger the hidden file input. The backend processing
    // would need to be handled, and then this list refreshed.
    if (documentUploadButton && documentFileInputHidden) {
        documentUploadButton.addEventListener('click', () => {
            documentFileInputHidden.click();
        });

        documentFileInputHidden.addEventListener('change', (event) => {
            if (event.target.files && event.target.files.length > 0) {
                // This is a simplified version. In a full app, you'd send these
                // files to the backend for staging & processing, then refresh the list.
                alert(`File(s) selected: ${Array.from(event.target.files).map(f=>f.name).join(', ')}. \nFull upload from chat panel not yet implemented. Please use the main upload page.`);
                // To fully implement:
                // 1. Send these files to '/stage_file'
                // 2. Then, somehow trigger '/process_staged_files' or add to a queue
                // 3. Then call loadAndDisplayProcessedDocuments() or update list from backend response.
                event.target.value = null; // Reset file input
            }
        });
    }


    // --- Initial load ---
    loadAndDisplayProcessedDocuments();

    // Optional: Listen for storage changes if another tab modifies 'activeDocs'
    // window.addEventListener('storage', (event) => {
    //     if (event.key === 'activeDocs') {
    //         loadAndDisplayProcessedDocuments();
    //     }
    // });
});
"""
chat_js_content = """
// Chat interface animations and enhanced interactions
document.addEventListener('DOMContentLoaded', () => {
    // Element references
    const messageItemsContainer = document.getElementById('messageItemsContainer');
    const chatForm = document.getElementById('chatForm');
    const chatInput = document.getElementById('chatInput');
    const messageList = document.getElementById('messageList');    // Apply animations and interactions to message bubbles
    function enhanceMessages() {
        try {
            // Target only unenhanced bubbles for better performance
            const chatBubbles = document.querySelectorAll('.chat-bubble:not([data-enhanced="true"])');
            
            if (!chatBubbles || chatBubbles.length === 0) return false;
            
            let bubbleCount = 0;
            
            chatBubbles.forEach(bubble => {
                try {
                    const message = bubble.querySelector('.chat-bubble-message');
                    if (!message) return;
                    
                    // Skip loading messages - they're handled separately
                    if (message.classList.contains('loading')) return;
                    
                    // Add entrance animation for new messages
                    bubble.style.opacity = '0';
                    bubble.style.transform = 'translateY(10px)';
                    
                    requestAnimationFrame(() => {
                        bubble.style.transition = 'opacity 0.3s ease, transform 0.3s ease';
                        bubble.style.opacity = '1';
                        bubble.style.transform = 'translateY(0)';
                    });
                    
                    // Add ripple effect on click
                    bubble.addEventListener('click', function(e) {
                        message.classList.add('ripple');
                        
                        setTimeout(() => {
                            message.classList.remove('ripple');
                        }, 700);
                    });
                    
                    // Add hover animations
                    bubble.addEventListener('mouseenter', () => {
                        bubble.style.transition = 'all 0.3s ease';
                        if (bubble.classList.contains('sent')) {
                            message.style.transform = 'translateX(-2px)';
                        } else {
                            message.style.transform = 'translateX(2px)';
                        }
                    });
                    
                    bubble.addEventListener('mouseleave', () => {
                        message.style.transform = '';
                    });
                    
                    // Mark as enhanced
                    bubble.setAttribute('data-enhanced', 'true');
                    bubbleCount++;
                } catch (err) {
                    console.error('Error enhancing specific chat bubble:', err);
                }
            });
            
            return bubbleCount > 0;
        } catch (err) {
            console.error('Error in enhanceMessages function:', err);
            return false;
        }
    }// Update the loading indicator to use the new typing indicator animation
    function enhanceLoadingIndicator() {
        try {
            const loadingIndicators = document.querySelectorAll('.loading-indicator');
            
            if (!loadingIndicators || loadingIndicators.length === 0) return false;
            
            let anyEnhanced = false;
            
            loadingIndicators.forEach(indicator => {
                try {
                    // Check if already enhanced
                    if (indicator.getAttribute('data-enhanced') === 'true') return;
                    
                    // Create typing indicator element
                    const typingIndicator = document.createElement('div');
                    typingIndicator.className = 'typing-indicator';
                    
                    // Add three animated dots
                    for (let i = 0; i < 3; i++) {
                        const dot = document.createElement('span');
                        typingIndicator.appendChild(dot);
                    }
                    
                    // Replace existing content with the new typing indicator
                    indicator.innerHTML = '';
                    indicator.appendChild(typingIndicator);
                    
                    // Mark as enhanced
                    indicator.setAttribute('data-enhanced', 'true');
                    anyEnhanced = true;
                    
                    // Add fade-in animation
                    indicator.style.opacity = '0';
                    requestAnimationFrame(() => {
                        indicator.style.transition = 'opacity 0.3s ease';
                        indicator.style.opacity = '1';
                    });
                } catch (err) {
                    console.error('Error enhancing loading indicator:', err);
                }
            });
            
            return anyEnhanced;
        } catch (err) {
            console.error('Error in enhanceLoadingIndicator function:', err);
            return false;
        }
    }    // Apply animations to newly added messages
    const observeMessages = () => {
        try {
            if (!messageItemsContainer) {
                console.error('Message container not found, cannot observe');
                return;
            }
            
            // Create an observer instance
            const observer = new MutationObserver((mutations) => {
                try {
                    let newMessageNodes = [];
                    let newLoadingIndicator = false;
                    
                    mutations.forEach((mutation) => {
                        if (mutation.type === 'childList' && mutation.addedNodes.length) {
                            mutation.addedNodes.forEach(node => {
                                // Check if the node is a chat bubble
                                if (node.nodeType === 1 && node.classList && node.classList.contains('chat-bubble')) {
                                    if (node.querySelector('.loading-indicator')) {
                                        newLoadingIndicator = true;
                                    } else {
                                        newMessageNodes.push(node);
                                    }
                                }
                            });
                        }
                    });
                    
                    // Process in the next animation frame to ensure DOM stability
                    if (newMessageNodes.length > 0 || newLoadingIndicator) {
                        requestAnimationFrame(() => {
                            // First enhance new messages (non-loading)
                            if (newMessageNodes.length > 0) {
                                enhanceMessages();
                            }
                            
                            // Then handle loading indicators separately
                            if (newLoadingIndicator) {
                                enhanceLoadingIndicator();
                            }
                        });
                    }
                } catch (err) {
                    console.error('Error in mutation observer callback:', err);
                }
            });
            
            // Start observing the target node for configured mutations
            observer.observe(messageItemsContainer, { 
                childList: true,
                subtree: true,  // Also observe child nodes for changes
                attributes: true, // Observe attribute changes
                attributeFilter: ['class', 'data-enhanced'] // Only specific attributes
            });
            
            console.log('Message observer initialized with enhanced detection');
        } catch (err) {
            console.error('Error setting up message observer:', err);
        }
    };
    
    // Auto-resize the input as user types
    const setupInputResizing = () => {
        const MAX_HEIGHT = 240;
        
        chatInput.addEventListener('input', () => {
            chatInput.style.height = 'auto';
            const newHeight = Math.min(chatInput.scrollHeight, MAX_HEIGHT);
            chatInput.style.height = `${newHeight}px`;
        });
    };
    
    // Send button animation
    const animateSendButton = () => {
        const sendButton = document.getElementById('sendButton');
        
        if (!sendButton) return;
        
        sendButton.addEventListener('mouseenter', () => {
            // Add a slight rotation animation
            sendButton.style.transition = 'transform 0.3s ease';
            sendButton.style.transform = 'rotate(10deg) scale(1.1)';
        });
        
        sendButton.addEventListener('mouseleave', () => {
            sendButton.style.transform = '';
        });
        
        sendButton.addEventListener('click', () => {
            // Add a press animation on click
            sendButton.style.transform = 'scale(0.9)';
            setTimeout(() => {
                sendButton.style.transform = '';
            }, 150);
        });
    };    // Initialize all enhancements
    function initializeEnhancements(retryCount = 0) {
        try {
            // Make sure document is fully loaded before enhancing
            if (!messageItemsContainer || !chatForm || !chatInput || !messageList) {
                if (retryCount < 10) { // Limit retry attempts
                    console.warn(`Chat elements not found, retrying initialization (attempt ${retryCount + 1}/10)...`);
                    setTimeout(() => initializeEnhancements(retryCount + 1), 100);
                } else {
                    console.error('Failed to find chat elements after multiple attempts');
                }
                return;
            }
            
            // First observe mutations so we don't miss any new elements
            observeMessages();
            
            // Schedule enhancements in proper sequence with animation frame
            requestAnimationFrame(() => {
                // Enhance existing messages first
                enhanceMessages();
                
                // Then loading indicators
                enhanceLoadingIndicator();
                
                // Then UI components
                setupInputResizing();
                animateSendButton();
                
                console.log('Chat enhancements initialized successfully');
            });
        } catch (err) {
            console.error('Error during chat enhancement initialization:', err);
            
            // Retry initialization on error, but limit attempts
            if (retryCount < 3) {
                console.warn(`Retrying initialization due to error (attempt ${retryCount + 1}/3)...`);
                setTimeout(() => initializeEnhancements(retryCount + 1), 200);
            }
        }
    }
    
    // Use DOMContentLoaded since we're already inside that event,
    // but with a small delay to ensure all other scripts have run
    if (document.readyState === 'loading') {
        document.addEventListener('DOMContentLoaded', () => setTimeout(initializeEnhancements, 10));
    } else {
        // DOM already loaded, initialize with a slight delay
        setTimeout(initializeEnhancements, 10);
    }
});

"""

documents_js_path = os.path.join(STATIC_JS_DIR, 'documents.js')
chat_js_path = os.path.join(STATIC_JS_DIR, 'chat.js')

with open(documents_js_path, 'w') as f:
    f.write(documents_js_content)
print(f"Created '{documents_js_path}'.")

with open(chat_js_path, 'w') as f:
    f.write(chat_js_content)
print(f"Created '{chat_js_path}'.")


print("\n--- HTML and JS placeholder files created ---")

In [4]:
# Backend

# --- 1. Imports ---
import base64
import io
import json
import logging
import os
import re
import shutil
import tempfile
import time
import uuid
from typing import Any, Dict, List, Optional

import chromadb
import fitz  # PyMuPDF
import numpy as np
import pdfplumber
import pytesseract
import torch
from flask import Flask, jsonify, render_template, request, send_from_directory
from flask_cors import CORS
from google.colab import files as colab_files # Specific to Colab, handle if not in Colab
from google.colab import userdata # Specific to Colab
from huggingface_hub import InferenceClient
from huggingface_hub.utils import (GatedRepoError, HFValidationError,
                                   RepositoryNotFoundError)
from langchain.chains.summarize import \
    load_summarize_chain
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from loguru import logger
from pdf2image import convert_from_path
from PIL import Image, UnidentifiedImageError

# New Azure Imports
from azure.ai.inference import EmbeddingsClient
from azure.core.credentials import AzureKeyCredential
# New OpenAI Imports for LLM
from openai import OpenAI
from langchain_openai import ChatOpenAI # For LangChain integration with OpenAI

import cv2

from docx import Document as DocxDocument
from openpyxl import load_workbook
# Note: python-pptx doesn't have a direct save-to-PDF method, unoconv is best here.
# from pptx import Presentation as PptxPresentation
# from pptx.enum.shapes import MSO_SHAPE_TYPE
# from pptx.util import Inches

# --- 2. Loguru Configuration ---
logger.remove()
logger.add(
    lambda msg: print(msg, end=""),
    level="INFO",
    format="<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green> | <level>{level: <8}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - <level>{message}</level>",
)
LOG_FILE_PATH = "ask_your_doc_flask_merged_app.log"
logger.add(LOG_FILE_PATH, rotation="10 MB", level="DEBUG")
logger.info(f"AskYourDoc Merged Flask App Starting. Log file: {LOG_FILE_PATH}")

# --- 3. Configuration (Combined and Prioritized) ---
HF_API_TOKEN_SECRET = None

try:
    logger.info("HF_TOKEN loaded.")
except userdata.SecretNotFoundError:
    logger.warning("HF_TOKEN not in Colab secrets.")

# AZURE EMBEDDING CONFIGURATION
AZURE_EMBEDDING_ENDPOINT = "https://models.inference.ai.azure.com"
AZURE_EMBEDDING_MODEL_NAME = "text-embedding-3-large"
logger.info(f"Azure Embedding Model: {AZURE_EMBEDDING_MODEL_NAME} configured.")
logger.warning("Using a GitHub token for Azure AI Inference API Key. Please verify this is the correct type of key for your Azure endpoint.")


# AZURE LLM CONFIGURATION (using the same endpoint and key as embedding)
AZURE_LLM_ENDPOINT = AZURE_EMBEDDING_ENDPOINT # Reusing the endpoint
AZURE_LLM_MODEL_NAME = "gpt-4o"
AZURE_LLM_API_KEY_SECRET = AZURE_EMBEDDING_API_KEY_SECRET # Reusing the token for both
logger.info(f"Azure LLM Model: {AZURE_LLM_MODEL_NAME} configured.")


# From Parsing Script
VISION_MODEL_ID_CONFIG = "meta-llama/Llama-4-Scout-17B-16E-Instruct"
IMAGE_OCR_PROMPT_CONFIG = "You are an advanced OCR model. Transcribe all text from this image accurately. If it's handwritten, do your best to read it. If there is no text, output 'No text found'."
IMAGE_DESCRIPTION_PROMPT_CONFIG = "You are an expert image analyst. Describe this image in detail, focusing on elements relevant to understanding a document (e.g., charts, diagrams, important visual features). If the image primarily contains text, transcribe the text instead of describing it as a visual."


# From KB/Retrieval Script
EMBEDDING_MODEL_NAME = AZURE_EMBEDDING_MODEL_NAME # Reflect the new model for logging
EMBEDDING_MODEL_MAX_TOKENS = 8192 # Max input tokens for text-embedding-3-large
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-12-v2"
RERANKER_MAX_TOKENS = 512 # Max sequence length for reranker.

# Adjusted for larger context window of text-embedding-3-large (8192 tokens)
PRIMARY_CHUNK_SIZE_TOKENS = 1500
PRIMARY_CHUNK_OVERLAP_TOKENS = 150
MIN_CHUNK_SIZE_TOKENS_NO_SPLIT = 75
EFFECTIVE_MAX_CHUNK_TOKENS = EMBEDDING_MODEL_MAX_TOKENS - 5
ATOMIC_BLOCK_NO_SPLIT_THRESHOLD = EFFECTIVE_MAX_CHUNK_TOKENS

SEPARATORS_HIERARCHICAL = ["\n\n\n", "\n\n", "\n", ". ", "? ", "! ", "; ", ", ", " ", ""]

# From Generation Script
GENERATION_MODEL_NAME = AZURE_LLM_MODEL_NAME # Reflect the new LLM model for logging

CHROMA_DB_PATH = "./ask_your_doc_chroma_db_merged_flask"
CHROMA_COLLECTION_NAME_PREFIX = "askdocmerged"
MAX_TOKENS_FOR_DIRECT_LLM_SUMMARIZATION = 80000

# --- 4. Global Application State (Unified) ---
APP_STATE = {
    "hf_api_token": HF_API_TOKEN_SECRET,
    "hf_vision_client": None,
    "vision_client_initialized": False,
    "langchain_llm": None,
    "llm_initialized": False,
    "azure_llm_client": None,
    "tokenizer": None,
    "embedding_model": None,
    "azure_embedding_client": None,
    "reranker_model": None,
    "rag_models_loaded": False,
    "chroma_client": None,
    "db_initialized": False,
    "chroma_collection_cache": {},
    "original_doc_filename_for_chunking_context": None,
}
# Web-specific state
APP_WEB_STATE = {
    "staged_files": {},
    "processed_collections": {},
}

# --- 5. AI Persona and System Instructions (No Inline Citations) ---
AI_PERSONA_NAME = "DocuMind AI"
AI_ROLE_DESCRIPTION = (
    "a meticulous and highly accurate AI assistant, designed to interact with "
    "and answer questions about documents. You are an expert in information "
    "retrieval and text comprehension."
)
AI_CORE_DIRECTIVE = (
    "Your responses must be grounded *exclusively* in the provided context data. "
    "You do not have access to external knowledge or the internet. "
    "You must not invent, assume, or infer information beyond what is explicitly stated in the context. "
    "Accuracy and adherence to the provided context are paramount. "
    "DO NOT invent data columns, values, or statuses that are not explicitly written in the PROVIDED CONTEXT."
)
AI_ANSWERING_STYLE_REFINED = (
    "Provide answers that are factual, precise, and directly supported by the text in "
    "the 'PROVIDED CONTEXT'. Use clear and concise language. If the query asks for "
    "specific data points (e.g., numbers, dates, names), ensure they are extracted "
    "accurately. If the query is more general, synthesize the relevant information "
    "from the context into a coherent response. Avoid jargon unless it's part of the "
    "document's language and relevant to the answer. You are polite, professional, and helpful."
)
AI_CONTEXTUAL_PRIORITIZATION_POLICY = (
    "When multiple context sources are provided, synthesize information if they are "
    "complementary. If they are contradictory, point out the discrepancy if relevant "
    "to the query, or prioritize the most specific or seemingly authoritative source if "
    "a single answer is required and discernable. If context chunks seem to be out of "
    "order, try to make sense of them logically if possible, but do not assume continuity "
    "if it's not evident."
)
AI_CITATION_POLICY_TEXT = (
    "DO NOT include bracketed citations like [Doc: ..., P:..., CkID:...] in your response. "
    "You may refer to 'the document' or specific document names if it's natural for the answer, "
    "but avoid detailed source tagging in the answer text."
)
AI_NO_ANSWER_POLICY = (
    "If the context is insufficient to answer the query, you must state: 'Based on the provided document context, I could not find the information to answer this question.'"
)
AI_SUMMARIZATION_POLICY = (
    "If the query asks for a summary, provide a concise yet comprehensive overview of the main points "
    "from the 'PROVIDED CONTEXT'. The summary should be neutral, "
    "objective, and reflect the key information accurately. Follow the citation policy "
    "(i.e., no bracketed citations). Focus on the core aspects and main themes."
)

# --- 6. Initialization Functions (Combined and Refined) ---
def initialize_hf_client_from_parser():
    global APP_STATE
    if APP_STATE["vision_client_initialized"]:
        return True
    if not APP_STATE["hf_api_token"]:
        logger.warning("HF_TOKEN not found. Vision features disabled.")
        return False
    if VISION_MODEL_ID_CONFIG == "YOUR_NOVITA_SUPPORTED_VISION_MODEL_ID_PLACEHOLDER":
        logger.error(f"VISION_MODEL_ID_CONFIG is placeholder. Vision disabled.")
        return False
    try:
        APP_STATE["hf_vision_client"] = InferenceClient(
            provider="novita", api_key=HF_API_TOKEN_SECRET
        )
        logger.success(f"HF client (Novita) for {VISION_MODEL_ID_CONFIG} initialized.")
        APP_STATE["vision_client_initialized"] = True
    except Exception as e:
        logger.error(f"HF client init error: {e}", exc_info=True)
        APP_STATE["hf_vision_client"] = None
        APP_STATE["vision_client_initialized"] = False
    return APP_STATE["vision_client_initialized"]

def initialize_llm_model():
    global APP_STATE
    if APP_STATE["llm_initialized"]:
        return True
    if not AZURE_LLM_API_KEY_SECRET:
        logger.critical("AZURE_LLM_API_KEY_SECRET missing!")
        return False
    try:
        APP_STATE["azure_llm_client"] = OpenAI(
            base_url=AZURE_LLM_ENDPOINT,
            api_key=AZURE_LLM_API_KEY_SECRET,
        )
        logger.success(f"OpenAI client for '{AZURE_LLM_MODEL_NAME}' initialized.")

        APP_STATE["langchain_llm"] = ChatOpenAI(
            model=AZURE_LLM_MODEL_NAME,
            openai_api_base=AZURE_LLM_ENDPOINT,
            openai_api_key=AZURE_LLM_API_KEY_SECRET,
            temperature=0.25,
            max_tokens=1000,
        )
        logger.success(
            f"LangChain LLM wrapper for '{AZURE_LLM_MODEL_NAME}' initialized for potential summarization."
        )
        APP_STATE["llm_initialized"] = True
    except Exception as e:
        logger.critical(f"OpenAI LLM init FAILED: {e}", exc_info=True)
        APP_STATE["llm_initialized"] = False
    return APP_STATE["llm_initialized"]

def initialize_rag_models_from_kb():
    global APP_STATE
    if APP_STATE["rag_models_loaded"]:
        return True
    try:
        APP_STATE["tokenizer"] = tiktoken.get_encoding("cl100k_base")
        logger.success("Tiktoken 'cl00k_base' OK.")
    except Exception as e:
        logger.warning(f"Tiktoken fail: {e}. Using len().")
        APP_STATE["tokenizer"] = None

    device = "cuda" if torch.cuda.is_available() else "cpu"
    models_ok = True

    # Initialize Azure Embeddings Client
    if not APP_STATE.get("azure_embedding_client"):
        try:
            logger.info(f"Loading Azure Embeddings client: {AZURE_EMBEDDING_MODEL_NAME}...")
            APP_STATE["azure_embedding_client"] = EmbeddingsClient(
                endpoint=AZURE_EMBEDDING_ENDPOINT,
                credential=AzureKeyCredential(AZURE_EMBEDDING_API_KEY_SECRET)
            )
            APP_STATE["embedding_model"] = APP_STATE["azure_embedding_client"]
            logger.success(f"Azure Embeddings client for '{AZURE_EMBEDDING_MODEL_NAME}' initialized.")
        except Exception as e:
            logger.critical(f"Azure Embeddings client load FAILED: {e}", exc_info=True)
            models_ok = False

    if not APP_STATE.get("reranker_model"):
        try:
            logger.info(f"Loading reranker: {RERANKER_MODEL_NAME} ({device})...")
            APP_STATE["reranker_model"] = CrossEncoder(
                RERANKER_MODEL_NAME,
                device=device,
                trust_remote_code=True,
                max_length=RERANKER_MAX_TOKENS,
            )
            logger.success("Reranker OK.")
        except Exception as e:
            logger.warning(f"Reranker load FAIL: {e}. Reranking off.")
            APP_STATE["reranker_model"] = None
    APP_STATE["rag_models_loaded"] = models_ok and APP_STATE["embedding_model"] is not None
    if not APP_STATE["rag_models_loaded"]:
        logger.error("Essential RAG models FAILED to load.")
    return APP_STATE["rag_models_loaded"]

def initialize_chromadb_from_kb():
    global APP_STATE
    if APP_STATE["db_initialized"]:
        return True
    logger.info(f"Initializing ChromaDB: path='{CHROMA_DB_PATH}'")
    try:
        APP_STATE["chroma_client"] = chromadb.PersistentClient(path=CHROMA_DB_PATH)
        logger.success(f"ChromaDB persistent OK: '{CHROMA_DB_PATH}'.")
        APP_STATE["db_initialized"] = True
    except Exception as e:
        logger.error(f"Chroma persistent FAIL: {e}. Trying in-memory.")
        try:
            APP_STATE["chroma_client"] = chromadb.Client()
            logger.success("Chroma in-memory OK.")
            APP_STATE["db_initialized"] = True
        except Exception as e_mem:
            logger.critical(f"Chroma in-memory FAIL: {e_mem}")
            APP_STATE["db_initialized"] = False
            return False
    return True

def get_or_create_collection_cached(
    collection_name_str: str,
) -> Optional[chromadb.api.models.Collection.Collection]:
    if not APP_STATE.get("db_initialized") or not APP_STATE.get("chroma_client"):
        logger.error("ChromaDB not init.")
        return None
    if collection_name_str in APP_STATE["chroma_collection_cache"]:
        logger.debug(f"Using cached collection: {collection_name_str}")
        return APP_STATE["chroma_collection_cache"][collection_name_str]
    try:
        collection = APP_STATE["chroma_client"].get_or_create_collection(
            name=collection_name_str, metadata={"hnsw:space": "cosine"}
        )
        APP_STATE["chroma_collection_cache"][collection_name_str] = collection
        logger.info(
            f"Collection '{collection_name_str}' accessed/created. Items: {collection.count()}"
        )
        return collection
    except Exception as e:
        logger.error(
            f"FAIL get/create Chroma collection '{collection_name_str}': {e}"
        )
        return None

# --- 7. Helper Functions ---
def count_tokens(text: str) -> int:
    if APP_STATE.get("tokenizer"):
        try:
            return len(APP_STATE["tokenizer"].encode(text, disallowed_special=()))
        except Exception as e:
            logger.warning(f"Tiktoken encode error for text '{text[:30]}...': {e}. Using char count.");
            return len(text)
    return len(text)

def image_to_base64_data_uri(pil_image: Image.Image) -> str:
    buffered = io.BytesIO()
    img_format = pil_image.format if pil_image.format else 'PNG'
    try:
        pil_image.save(buffered, format=img_format)
    except Exception as e:
        logger.warning(f"Warning: Could not save image in format {img_format}, falling back to PNG. Error: {e}")
        img_format = 'PNG'
        pil_image.save(buffered, format=img_format)
    encoded_bytes = base64.b64encode(buffered.getvalue())
    encoded_string = encoded_bytes.decode('utf-8')
    mime_type = Image.MIME.get(img_format.upper(), 'image/png')
    return f"data:{mime_type};base64,{encoded_string}"

def _process_image_with_vision_model(pil_image: Image.Image, prompt_text: str) -> str:
    if not APP_STATE.get("vision_client_initialized") or not APP_STATE.get(
        "hf_vision_client"
    ):
        logger.error("Vision model client not initialized. Cannot process image.")
        return "Error: Vision model client not initialized."
    if VISION_MODEL_ID_CONFIG == "YOUR_NOVITA_SUPPORTED_VISION_MODEL_ID_PLACEHOLDER":
        logger.error(f"Vision model ID is a placeholder: {VISION_MODEL_ID_CONFIG}. Cannot process image.")
        return "Error: Vision model ID is a placeholder."

    base64_image_url = image_to_base64_data_uri(pil_image)
    try:
        completion = APP_STATE["hf_vision_client"].chat.completions.create(
            model=VISION_MODEL_ID_CONFIG,
            messages=[
                {"role": "user", "content": [{"type": "text", "text": prompt_text}, {"type": "image_url", "image_url": {"url": base64_image_url}}]}
            ],
            max_tokens=1500
        )
        if completion.choices and completion.choices[0].message and completion.choices[0].message.content:
            return completion.choices[0].message.content.strip()
        else:
            logger.error(f"Vision model ({VISION_MODEL_ID_CONFIG}) - No content in response. Prompt: {prompt_text[:50]}...")
            return f"Error: No content from vision model {VISION_MODEL_ID_CONFIG}."
    except Exception as e:
        logger.error(f"An unexpected error occurred with the vision model ({VISION_MODEL_ID_CONFIG}): {e}", exc_info=True)
        return f"Error processing image with vision model {VISION_MODEL_ID_CONFIG}: {str(e)}"

def sanitize_chromadb_collection_name(name: str) -> str:
    name = str(name)
    name = re.sub(r'[ \t\n\r\f\v.,;:!?"\'`()\[\]{}<>|/\\]+', '_', name)
    name = re.sub(r'[^a-zA-Z0-9_-]', '', name)
    name = re.sub(r'^_+|-+$', '', name)
    name = re.sub(r'^-+|_+$', '', name)
    if name and not name[0].isalnum():
        name = 'c_' + name
    if name and not name[-1].isalnum():
        name = name + '_c'
    if len(name) < 3:
        name = name + '___'
        name = name[:3]
    if len(name) > 63:
        name = name[:63]
    if name and not name[0].isalnum(): name = 'c' + name[1:]
    if name and len(name) > 1 and not name[-1].isalnum(): name = name[:-1] + 'c'
    elif name and len(name) == 1 and not name[0].isalnum(): name = 'cc'
    if not name or len(name) < 3:
        name = f"coll_{uuid.uuid4().hex[:8]}"
    return name.lower()


# --- 8. PDF Parsing Logic (The CORE parser for all converted PDFs) ---
def get_font_properties(span_dict: Dict) -> Dict:
    return {
        "size": span_dict.get("size", 0.0),
        "font": span_dict.get("font", "UnknownFont"),
        "color": span_dict.get("color", 0),
        "flags": span_dict.get("flags", 0),
        "origin_x": span_dict.get("bbox", [0, 0, 0, 0])[0],
        "origin_y": span_dict.get("bbox", [0, 0, 0, 0])[1],
    }

def is_likely_header_or_footer(
    block_text: str,
    page_rect: fitz.Rect,
    block_bbox: fitz.Rect,
    page_number: int,
    num_pages: int,
) -> bool:
    block_content = block_text.strip()
    if not block_content or len(block_content) > 150:
        return False
    page_height = page_rect.height
    is_top_zone = block_bbox.y1 < page_rect.y0 + 0.12 * page_height
    is_bottom_zone = block_bbox.y0 > page_rect.y1 - 0.12 * page_height
    if not (is_top_zone or is_bottom_zone):
        return False
    page_num_str = str(page_number)
    num_pages_str = str(num_pages)
    patterns = [
        r"^(Page\s*)?" + re.escape(page_num_str) + r"(\s*(of|-|/)\s*" + re.escape(num_pages_str) + r")?([\s.]*)$",
        r"^(\[?" + re.escape(page_num_str) + r"\]?)$",
        r"^\s*-\s*\d+\s*-\s*$",
        r"^\s*" + re.escape(page_num_str) + r"\s*$",
    ]
    for pattern in patterns:
        if re.fullmatch(pattern, block_content, re.IGNORECASE):
            return True
    if len(block_content.split()) < 7 and len(block_content) < 70:
        if not re.search(r"[.!?]$", block_content):
            if block_content.isupper() or block_content.istitle():
                return True
            if len(block_content) < 10 and len(set(block_content.replace(" ", ""))) < 4:
                return True
    return False

def parse_pdf_content_worker(pdf_path: str, original_filename: str, original_document_type: str, config_dict: dict) -> list:
    logger.info(f"  Worker: Parsing PDF content from '{os.path.basename(pdf_path)}' (originally {original_document_type}) with config: {config_dict}")
    all_content_blocks = []
    doc_block_counter = 0

    use_pdfplumber_tables = config_dict.get("use_pdfplumber_tables", True)
    use_pymupdf_text = config_dict.get("use_pymupdf_text", True)
    process_scanned_pages = config_dict.get("process_scanned_pages", False)
    use_vision_for_ocr_flag = config_dict.get("use_vision_for_ocr", False)
    process_structured_images = config_dict.get("process_structured_images", False)
    use_vision_for_description_flag = config_dict.get("use_vision_for_description", False)
    scan_detection_char_threshold = config_dict.get("scan_detection_char_threshold", 100)
    dpi_for_conversion = config_dict.get("dpi_for_conversion", 200)

    tables_by_page = {}
    if use_pdfplumber_tables:
        logger.debug("  Worker: Attempting table extraction with PdfPlumber...")
        try:
            with pdfplumber.open(pdf_path) as pdf_pl:
                for p_idx, page_pl in enumerate(pdf_pl.pages):
                    page_num_human = p_idx + 1
                    raw_tables = page_pl.find_tables()
                    extracted_tables_data = []
                    if raw_tables:
                        logger.debug(f"    P{page_num_human}: Found {len(raw_tables)} potential tables with PdfPlumber.")
                        for tbl_idx, raw_table in enumerate(raw_tables):
                            md_table = f"[Table {tbl_idx+1} on Page {page_num_human}]\n"
                            data = raw_table.extract()
                            if data:
                                try:
                                    header = data[0]
                                    if header and all(isinstance(c, (str, type(None))) for c in header):
                                        md_table += "| " + " | ".join(str(c).strip().replace("\n", " ") if c is not None else "" for c in header) + " |\n"
                                        md_table += "| " + " | ".join("---" for _ in header) + " |\n"
                                        for row in data[1:]:
                                            if row and all(isinstance(c, (str, type(None))) for c in row):
                                                md_table += f"| {' | '.join(str(c).strip().replace(chr(10),' ') if c is not None else '' for c in row)} |\n"
                                    else:
                                        for row_idx, row in enumerate(data):
                                            if row and all(isinstance(c, (str, type(None))) for c in row):
                                                md_table += ("| " if row_idx == 0 else "") + " | ".join(str(c).strip().replace("\n", " ") if c is not None else "" for c in row) + (" |\n" if row_idx == 0 and len(data) > 1 else "\n")
                                                if row_idx == 0 and len(data) > 1:
                                                    md_table += "| " + " | ".join("---" for _ in row) + " |\n"
                                except TypeError:
                                    md_table += "(Complex table structure, basic markdown conversion failed)\n"
                                    logger.debug(f"    P{page_num_human} T{tbl_idx+1}: TypeError during MD conversion.")
                            else:
                                md_table += "(Table detected, but no data could be extracted or table is empty)\n"
                            extracted_tables_data.append(
                                {"bbox": raw_table.bbox, "markdown_content": md_table, "order": raw_table.bbox[1]}
                            )
                    if extracted_tables_data:
                        tables_by_page[page_num_human] = sorted(
                            extracted_tables_data, key=lambda t: t["order"]
                        )
        except Exception as e:
            logger.error(
                f"  Worker: Pdfplumber table extraction FAILED for '{os.path.basename(pdf_path)}': {e}",
                exc_info=True,
            )

    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        logger.critical(f"  Worker: PyMuPDF open FAILED for '{os.path.basename(pdf_path)}': {e}.")
        return [{"block_id":"error_doc_open", "type":"error", "content":f"PDF open fail: {e}", "page_number":0, "document_type": original_document_type}]

    num_pages = len(doc)
    all_font_sizes_doc = []
    logger.debug(f"  Worker: Document '{os.path.basename(pdf_path)}' has {num_pages} pages. Calculating font statistics...")
    for page_fitz_temp in doc:
        try:
            textpage_temp = page_fitz_temp.get_textpage_ocr(flags=0, full=False)
            blocks_dict_temp = textpage_temp.extractDICT().get("blocks", [])
            for block_dict_temp in blocks_dict_temp:
                if block_dict_temp.get("type") == 0:
                    for line_dict_temp in block_dict_temp.get("lines", []):
                        for span_dict_temp in line_dict_temp.get("spans", []):
                            all_font_sizes_doc.append(span_dict_temp.get("size", 0))
        except Exception as e_fs:
            logger.warning(f"    Font size stats extraction error on page {page_fitz_temp.number + 1}: {e_fs}")

    avg_font_size = np.mean([s for s in all_font_sizes_doc if s > 0]) if any(s > 0 for s in all_font_sizes_doc) else 10.0
    std_font_size = np.std([s for s in all_font_sizes_doc if s > 0]) if any(s > 0 for s in all_font_sizes_doc) and len(set(s for s in all_font_sizes_doc if s > 0)) > 1 else 2.0
    if std_font_size < 1.5:
        std_font_size = 1.5
    logger.info(f"  Doc stats: Pages:{num_pages}, AvgFont:{avg_font_size:.1f}, StdFont:{std_font_size:.1f}")

    for page_idx, page_fitz in enumerate(doc):
        page_num_human = page_idx + 1
        logger.debug(f"  Worker: Processing Page {page_num_human}/{num_pages}")
        page_content_elements = []
        current_page_table_bboxes = [
            tuple(t["bbox"]) for t in tables_by_page.get(page_num_human, [])
        ]

        def check_overlap_with_tables(b_bbox_coords, t_bboxes_coords_list, threshold=0.3):
            b_x0,b_y0,b_x1,b_y1 = b_bbox_coords
            b_area=(b_x1-b_x0)*(b_y1-b_y0)
            if b_area == 0: return False
            for t_coords in t_bboxes_coords_list:
                t_x0,t_y0,t_x1,t_y1 = t_coords
                ix0,iy0 = max(b_x0,t_x0), max(b_y0,t_y0)
                ix1,iy1 = min(b_x1,t_x1), min(b_y1,t_y1)
                iarea = max(0,ix1-ix0) * max(0,iy1-iy0)
                if (iarea/b_area) > threshold: return True
            return False

        if use_pymupdf_text:
            try:
                textpage_flags = (
                    fitz.TEXTFLAGS_SEARCH | fitz.TEXTFLAGS_PRESERVE_LIGATURES
                    | fitz.TEXTFLAGS_PRESERVE_IMAGES | fitz.TEXTFLAGS_PRESERVE_WHITESPACE
                )
            except AttributeError:
                textpage_flags = fitz.TEXTFLAGS_SEARCH
                logger.warning(f"    P{page_num_human}: Older PyMuPDF version, using basic text flags.")

            try:
                textpage = page_fitz.get_textpage_ocr(flags=textpage_flags, full=False)
                page_dict = textpage.extractDICT()

                for block_dict in page_dict.get("blocks", []):
                    if block_dict.get("type") == 0:
                        block_text_content, span_font_props_list = "", []
                        for line_dict in block_dict.get("lines", []):
                            line_text_parts = []
                            for span_dict_val in line_dict.get("spans", []):
                                line_text_parts.append(span_dict_val["text"])
                                span_font_props_list.append(get_font_properties(span_dict_val))
                            block_text_content += " ".join(line_text_parts).strip() + "\n"

                        block_text_content = re.sub(r'\s*\n\s*', '\n', block_text_content).strip()
                        block_text_content = re.sub(r' +', ' ', block_text_content)
                        block_bbox_fitz = fitz.Rect(block_dict["bbox"])

                        if not block_text_content or check_overlap_with_tables(tuple(block_bbox_fitz), current_page_table_bboxes):
                            continue

                        current_block_avg_font_size = np.mean([s['size'] for s in span_font_props_list if s['size'] > 0]) if any(s['size'] > 0 for s in span_font_props_list) else avg_font_size
                        is_bold = any(s['flags'] & (1 << 4) for s in span_font_props_list)
                        num_words_first_line = len(block_text_content.split('\n')[0].split())

                        block_sem_type, is_heading_candidate = "text_paragraph", False
                        if is_likely_header_or_footer(block_text_content, page_fitz.rect, block_bbox_fitz, page_num_human, num_pages):
                            block_sem_type = "noise_header_footer"
                        elif len(block_text_content) <= 6 and not re.search(r'[a-zA-Z]{2,}', block_text_content) and \
                             not (current_block_avg_font_size > avg_font_size + 0.8 * std_font_size or is_bold):
                            block_sem_type = "noise_short_irrelevant"
                        else:
                            if page_idx == 0 and current_block_avg_font_size >= avg_font_size + 1.8 * std_font_size and \
                               num_words_first_line < 18 and block_bbox_fitz.y0 < page_fitz.rect.height * 0.3:
                                block_sem_type, is_heading_candidate = "title_document", True
                            elif current_block_avg_font_size >= avg_font_size + 1.4 * std_font_size and \
                                 (is_bold or num_words_first_line < 20 or (page_idx <=1 and num_words_first_line < 28)):
                                block_sem_type, is_heading_candidate = "h1_heading", True
                            elif current_block_avg_font_size >= avg_font_size + 0.7 * std_font_size and \
                                 (is_bold or num_words_first_line < 25):
                                block_sem_type, is_heading_candidate = "h2_heading", True
                            elif (current_block_avg_font_size >= avg_font_size + 0.3 * std_font_size and \
                                  is_bold and num_words_first_line < 30 and not block_text_content.endswith('.')):
                                block_sem_type, is_heading_candidate = "h3_heading", True

                        if is_heading_candidate:
                            logger.trace(f"    P{page_num_human}: Found {block_sem_type} (F:{current_block_avg_font_size:.1f},B:{is_bold}): '{block_text_content[:40]}...'")
                        page_content_elements.append({
                            "type": block_sem_type, "content": block_text_content, "bbox": tuple(block_bbox_fitz),
                            "order": block_bbox_fitz.y0, "is_semantic_heading": is_heading_candidate,
                            "font_size": current_block_avg_font_size, "is_bold": is_bold, "document_type": original_document_type
                        })
            except Exception as e_pymupdf_txt:
                logger.error(f"  Worker: PyMuPDF text extraction error on P{page_num_human}: {e_pymupdf_txt}", exc_info=True)

        for tbl_data in tables_by_page.get(page_num_human, []):
            page_content_elements.append({
                "type": "table_markdown", "content": tbl_data["markdown_content"],
                "bbox": tbl_data["bbox"], "order": tbl_data["order"],
                "is_semantic_heading": False, "font_size": avg_font_size, "is_bold": False, "document_type": original_document_type
            })

        initial_digital_text_len = sum(len(el["content"]) for el in page_content_elements if el["type"] not in ["noise_header_footer", "noise_short_irrelevant"])
        page_might_be_scan = initial_digital_text_len < scan_detection_char_threshold

        structured_images_on_page_meta = []
        if process_scanned_pages or process_structured_images:
            try:
                structured_images_on_page_meta = page_fitz.get_images(full=True)
            except Exception as e:
                logger.warning(f"  Worker: PyMuPDF get_images failed P{page_num_human}: {e}")

        if process_scanned_pages and page_might_be_scan and not structured_images_on_page_meta:
            logger.info(f"  P{page_num_human}: Low digital text ({initial_digital_text_len} chars), no structured images. Candidate for full page OCR.")
            try:
                pil_page_img_list = convert_from_path(
                    pdf_path, dpi=dpi_for_conversion, first_page=page_num_human,
                    last_page=page_num_human, fmt='jpeg', thread_count=1
                )
                if not pil_page_img_list:
                    logger.warning(f"    P{page_num_human}: pdf2image returned no image for full page OCR.")
                    continue
                pil_page_img = pil_page_img_list[0]
                fp_bbox, fp_ocr_text, fp_ocr_type_detail = tuple(page_fitz.rect), "", ""

                if use_vision_for_ocr_flag and APP_STATE.get("hf_vision_client"):
                    logger.debug(f"    P{page_num_human}: Attempting Vision OCR (full page).")
                    fp_ocr_text = _process_image_with_vision_model(pil_page_img, IMAGE_OCR_PROMPT_CONFIG)
                    fp_ocr_type_detail = "vision_ocr (full_page)"

                if not fp_ocr_text.strip() or "Error:" in fp_ocr_text or "No text found" in fp_ocr_text.lower():
                    logger.debug(f"    P{page_num_human}: Vision OCR failed or no text. Fallback/Attempt Tesseract (full page). Vision out: '{fp_ocr_text[:50]}...'")
                    if not use_vision_for_ocr_flag or ("Error:" in fp_ocr_text or "No text found" in fp_ocr_text.lower()):
                        fp_ocr_text = pytesseract.image_to_string(pil_page_img)
                        fp_ocr_type_detail = "tesseract_ocr (full_page)"

                if fp_ocr_text and fp_ocr_text.strip() and "Error:" not in fp_ocr_text and "No text found" not in fp_ocr_text.lower():
                    page_content_elements = [el for el in page_content_elements if el["type"] == "table_markdown"]
                    page_content_elements.append({
                        "type": "full_page_ocr_text", "content": fp_ocr_text.strip(), "bbox": fp_bbox,
                        "order": 0, "is_semantic_heading": False, "font_size": avg_font_size,
                        "is_bold": False, "ocr_source": fp_ocr_type_detail, "document_type": original_document_type
                    })
                    logger.info(f"    P{page_num_human}: Full scan OCR successful via {fp_ocr_type_detail}. Text length: {len(fp_ocr_text)}")
                else:
                    logger.warning(f"    P{page_num_human}: Full page OCR attempt ({fp_ocr_type_detail or 'N/A'}) yielded no usable text.")
            except Exception as e_fp_ocr:
                logger.error(f"    Error during P{page_num_human} full page OCR processing: {e_fp_ocr}", exc_info=True)

        if process_structured_images and structured_images_on_page_meta:
            logger.debug(f"  P{page_num_human}: Processing {len(structured_images_on_page_meta)} structured image regions.")
            for img_idx, img_meta_item in enumerate(structured_images_on_page_meta):
                xref = img_meta_item[0]
                try:
                    base_image = doc.extract_image(xref)
                    if not base_image or "image" not in base_image:
                        logger.warning(f"    P{page_num_human} ImgX{xref}: Could not extract base image data.")
                        continue

                    pil_img = Image.open(io.BytesIO(base_image["image"]))
                    img_placements_rects = page_fitz.get_image_rects(xref)

                    if not img_placements_rects:
                         if len(structured_images_on_page_meta) == 1 and page_might_be_scan:
                             img_placements_rects = [page_fitz.rect]
                             logger.debug("    Single image on scan-like page, using full page rect.")
                         else:
                             logger.warning(f"    P{page_num_human} ImgX{xref}: No placement rectangles found. Skipping image.")
                             continue

                    for rect_idx, fitz_rect_obj in enumerate(img_placements_rects):
                        img_bbox_coords = tuple(fitz_rect_obj)
                        img_filename_ref = f"p{page_num_human}_img{img_idx}_xref{xref}_r{rect_idx}.{base_image.get('ext','png')}"
                        if check_overlap_with_tables(img_bbox_coords, current_page_table_bboxes, 0.7):
                            logger.trace(f"    Skipping image XREF {xref} Rect {rect_idx} due to high table overlap.")
                            continue

                        ocr_from_img_content, desc_from_img_content, vision_ocr_attempted_for_img = "", "", False
                        if use_vision_for_ocr_flag and APP_STATE.get("hf_vision_client"):
                            vision_ocr_attempted_for_img = True
                            ocr_from_img_content = _process_image_with_vision_model(pil_img, IMAGE_OCR_PROMPT_CONFIG)
                            if ocr_from_img_content and "Error:" not in ocr_from_img_content and "No text found" not in ocr_from_img_content.lower():
                                page_content_elements.append({
                                    "type": "image_ocr_text", "content": ocr_from_img_content.strip(),
                                    "bbox": img_bbox_coords, "order": img_bbox_coords[1],
                                    "source_image_filename": img_filename_ref, "is_semantic_heading": False,
                                    "font_size": avg_font_size, "is_bold": False, "document_type": original_document_type
                                })
                            elif "No text found" in ocr_from_img_content.lower(): ocr_from_img_content = ""
                            elif "Error:" in ocr_from_img_content:
                                logger.warning(f"      P{page_num_human},ImgX{xref}R{rect_idx} Vision OCR Error: {ocr_from_img_content}")
                                ocr_from_img_content = ""

                        if process_structured_images and use_vision_for_description_flag and APP_STATE.get("hf_vision_client"):
                            should_describe_img = not (
                                vision_ocr_attempted_for_img and len(ocr_from_img_content) > 75 and
                                not any(k in ocr_from_img_content.lower() for k in ["chart","diagram","graph","figure","table"])
                            )
                            if should_describe_img:
                                desc_from_img_content = _process_image_with_vision_model(pil_img, IMAGE_DESCRIPTION_PROMPT_CONFIG)
                                if desc_from_img_content and "Error:" not in desc_from_img_content and desc_from_img_content.strip():
                                    if not(ocr_from_img_content and desc_from_img_content.strip().lower() == ocr_from_img_content.strip().lower()):
                                        page_content_elements.append({
                                            "type": "image_description", "content": desc_from_img_content.strip(),
                                            "bbox": img_bbox_coords, "order": img_bbox_coords[1] + 0.01,
                                            "source_image_filename": img_filename_ref, "is_semantic_heading": False,
                                            "font_size": avg_font_size, "is_bold": False, "document_type": original_document_type
                                        })
                except UnidentifiedImageError:
                    logger.warning(f"  P{page_num_human}, ImgX{xref}: PIL UnidentifiedImageError. Skipping this image resource.")
                except Exception as e_img_proc:
                    logger.error(f"  Error processing structured image XREF {xref} on P{page_num_human}: {e_img_proc}", exc_info=True)


        page_content_elements.sort(key=lambda item: item.get("order", float('inf')))
        for el_data in page_content_elements:
            content, el_type_str = el_data.get("content", "").strip(), el_data.get("type", "")
            if "noise" in el_type_str or content.lower() == "no text found":
                logger.trace(f"    P{page_num_human}: Final skip of noise block type '{el_type_str}' or 'No text found'.")
                continue
            if content:
                doc_block_counter += 1
                block_to_add = {k: v for k, v in el_data.items() if k not in ["order"]}
                block_to_add.update({"block_id": f"doc_block_{doc_block_counter}", "page_number": page_num_human})
                block_to_add.setdefault("is_semantic_heading", False)
                block_to_add.setdefault("font_size", avg_font_size)
                block_to_add.setdefault("is_bold", False)
                block_to_add.setdefault("document_type", original_document_type)
                all_content_blocks.append(block_to_add)
    try:
        doc.close()
    except Exception as e:
        logger.warning(f"  Worker: Error closing PDF '{os.path.basename(pdf_path)}': {e}")

    if not all_content_blocks:
        all_content_blocks.append({
            "block_id": "fallback_empty_doc", "type": "info",
            "content": f"No content could be extracted from this {original_document_type} file after PDF conversion. It might be empty, purely graphical without OCR, or a protected file.",
            "page_number": 0, "document_type": original_document_type
        })
    logger.info(
        f"  Worker: Finished parsing '{os.path.basename(pdf_path)}' (originally {original_document_type}). Total blocks extracted: {len(all_content_blocks)}"
    )
    return all_content_blocks

# --- NEW: Universal File Converter to PDF ---
def convert_to_pdf(input_path: str, temp_dir: str) -> Optional[Dict]:
    """
    Converts various document types (DOCX, XLSX, PPTX, Image) to PDF using unoconv or Pillow.
    Returns a dictionary with {pdf_path, original_filename, original_extension, original_document_type}.
    """
    original_filename = os.path.basename(input_path)
    file_extension = os.path.splitext(original_filename)[1].lower()
    base_name_no_ext = os.path.splitext(original_filename)[0]
    output_pdf_path = os.path.join(temp_dir, f"{base_name_no_ext}_{uuid.uuid4().hex[:6]}.pdf")

    original_doc_type = "unknown"
    if file_extension == ".pdf":
        original_doc_type = "pdf"
    elif file_extension == ".docx":
        original_doc_type = "docx"
    elif file_extension == ".xlsx":
        original_doc_type = "xlsx"
    elif file_extension == ".pptx":
        original_doc_type = "pptx"
    elif file_extension in [".png", ".jpg", ".jpeg", ".gif", ".bmp", ".tiff"]:
        original_doc_type = "image"
    elif file_extension == ".xls": # Old Excel format, map to xlsx for parsing
        original_doc_type = "xlsx" # Treat as xlsx for parsing context after conversion
    else:
        logger.error(f"Conversion failed: Unsupported file type for conversion: {file_extension} for '{original_filename}'")
        return {"pdf_path": None, "original_filename": original_filename, "original_extension": file_extension, "original_document_type": "unsupported", "error": f"Unsupported file type: {file_extension}"}

    if file_extension == ".pdf":
        logger.info(f"  File '{original_filename}' is already a PDF. Skipping conversion.")
        return {
            "pdf_path": input_path,
            "original_filename": original_filename,
            "original_extension": file_extension,
            "original_document_type": original_doc_type
        }
    elif original_doc_type == "image":
        try:
            img = Image.open(input_path)
            if img.mode != "RGB":
                img = img.convert("RGB")
            img.save(output_pdf_path, "PDF")
            logger.info(f"  Converted image '{original_filename}' to PDF at '{output_pdf_path}'.")
            return {
                "pdf_path": output_pdf_path,
                "original_filename": original_filename,
                "original_extension": file_extension,
                "original_document_type": original_doc_type
            }
        except Exception as e:
            logger.error(f"  Failed to convert image '{original_filename}' to PDF using Pillow: {e}", exc_info=True)
            return None
    else: # Use unoconv for DOCX, XLSX, PPTX, and now .xls
        command = f"unoconv -f pdf -o {output_pdf_path} {input_path}"
        logger.info(f"  Attempting to convert '{original_filename}' to PDF using unoconv: {command}")
        try:
            result = os.system(command)
            if result == 0 and os.path.exists(output_pdf_path) and os.path.getsize(output_pdf_path) > 0:
                logger.success(f"  Successfully converted '{original_filename}' to PDF at '{output_pdf_path}'.")
                return {
                    "pdf_path": output_pdf_path,
                    "original_filename": original_filename,
                    "original_extension": file_extension,
                    "original_document_type": original_doc_type
                }
            else:
                logger.error(f"  Unoconv failed or produced empty PDF for '{original_filename}'. Result code: {result}. PDF exists: {os.path.exists(output_pdf_path)}. Output file size: {os.path.getsize(output_pdf_path) if os.path.exists(output_pdf_path) else 'N/A'}")
                return None
        except Exception as e:
            logger.error(f"  Error during unoconv conversion of '{original_filename}': {e}", exc_info=True)
            return None

# --- Central Parsing Function (now orchestrates conversion and then PDF parsing) ---
def parse_document_via_profile(file_path: str, profile_name: str = "default_fallback") -> list:
    logger.info(f"Initiating parsing process for document '{os.path.basename(file_path)}' with profile: '{profile_name}'")

    current_config = {
        "use_pdfplumber_tables": True, "use_pymupdf_text": True, "process_scanned_pages": False,
        "process_structured_images": False, "use_vision_for_ocr": False, "use_vision_for_description": False,
        "scan_detection_char_threshold": 120, "dpi_for_conversion": 220
    }
    if profile_name == "fastest":
        current_config.update({"process_scanned_pages": False, "process_structured_images": False, "use_vision_for_ocr": False, "use_vision_for_description": False})
    elif profile_name == "digital_plus_ocr":
        current_config.update({"process_scanned_pages": True, "use_vision_for_ocr": False, "process_structured_images": True, "use_vision_for_description": False, "scan_detection_char_threshold": 150, "dpi_for_conversion": 200})
    elif profile_name == "comprehensive_vision":
        current_config.update({"process_scanned_pages": True, "use_vision_for_ocr": True, "process_structured_images": True, "use_vision_for_description": True, "scan_detection_char_threshold": 100, "dpi_for_conversion": 250})
    elif profile_name == "default_fallback":
        logger.warning(f"Using 'default_fallback' profile for '{os.path.basename(file_path)}'.")
        current_config.update({"process_scanned_pages": True, "use_vision_for_ocr": True, "process_structured_images": True, "use_vision_for_description": True})
    else:
        logger.error(f"Unexpected profile '{profile_name}'. Using minimal config.")
        current_config = {"use_pdfplumber_tables": True, "use_pymupdf_text": True}

    if not APP_STATE.get("vision_client_initialized", False):
        if current_config.get("use_vision_for_ocr"):
            logger.info(f"Profile '{profile_name}': Vision OCR globally disabled (client not ready).")
            current_config["use_vision_for_ocr"] = False
        if current_config.get("use_vision_for_description"):
            logger.info(f"Profile '{profile_name}': Vision Desc globally disabled (client not ready).")
            current_config["use_vision_for_description"] = False

    logger.info(f"Final config for parsing '{os.path.basename(file_path)}' (profile '{profile_name}'): {current_config}")
    start_time = time.time()

    converted_info = convert_to_pdf(file_path, STAGED_UPLOADS_DIR) # Use staged uploads dir for temp PDFs
    if not converted_info or not converted_info["pdf_path"]:
        error_msg = converted_info.get("error", "Unknown conversion error.")
        logger.error(f"Document conversion failed for '{os.path.basename(file_path)}': {error_msg}")
        return [{"block_id":"conversion_error", "type":"error", "content":f"File conversion to PDF failed: {error_msg}", "page_number":0, "document_type": converted_info.get("original_document_type", "unsupported")}]

    pdf_to_parse_path = converted_info["pdf_path"]
    original_filename = converted_info["original_filename"]
    original_document_type = converted_info["original_document_type"]

    try:
        # Now call the single PDF content parser
        parsed_data = parse_pdf_content_worker(pdf_to_parse_path, original_filename, original_document_type, current_config)

        # Ensure that blocks from non-PDFs have their `document_type` consistently set
        for block in parsed_data:
            block["document_type"] = original_document_type

        logger.success(f"Profile '{profile_name}' parsing for '{original_filename}' (converted from {original_document_type}) in {time.time()-start_time:.2f}s. Blocks: {len(parsed_data)}.")
        return parsed_data
    except Exception as e:
        logger.error(f"Error during PDF content parsing of converted file '{pdf_to_parse_path}' (originally {original_document_type}): {e}", exc_info=True)
        return [{"block_id":"parsing_error", "type":"error", "content":f"Error parsing PDF content after conversion: {e}", "page_number":0, "document_type": original_document_type}]
    finally:
        # Clean up the temporary PDF file if it was created
        if pdf_to_parse_path != file_path: # Only remove if it's a new temp file, not the original if it was already a PDF
            try:
                os.remove(pdf_to_parse_path)
                logger.info(f"  Removed temporary PDF file: '{pdf_to_parse_path}'.")
            except Exception as e:
                logger.warning(f"  Could not remove temporary PDF file '{pdf_to_parse_path}': {e}")


# --- 9. Knowledge Base Functions ---
def _prepend_context_to_chunk(original_chunk_text: str, block_metadata: dict) -> str:
    """
    Dynamically prepends contextual information to a chunk based on its type and metadata.
    Refined to use the actual `document_type` metadata.
    """
    orig_filename = APP_STATE.get("original_doc_filename_for_chunking_context", "this document")
    block_type = block_metadata.get("type", "text_paragraph")
    page_num = block_metadata.get("page_number", "N/A")
    doc_type = block_metadata.get("document_type", "document") # Crucial: Get original document type
    font_size = block_metadata.get("font_size", 0) # Only relevant for PDFs originally
    is_bold_str = " (Bold)" if block_metadata.get("is_bold", False) else "" # Only relevant for PDFs originally
    source_img_filename = block_metadata.get("source_image_filename")
    ocr_source = block_metadata.get("ocr_source")

    prefix = ""

    # General document type indicator for clarity in context
    doc_type_label = {
        "pdf": "PDF document",
        "docx": "Word document",
        "xlsx": "Excel document",
        "pptx": "PowerPoint presentation",
        "image": "Image file"
    }.get(doc_type, "document")

    # Specific prefixes for different block types
    if block_type == "table_markdown":
        prefix = f"Context: Data table from {doc_type_label} '{orig_filename}', section/page {page_num}.\nContent:\n"
    elif block_type == "image_ocr_text":
        img_info = f"image '{source_img_filename}'" if source_img_filename else "an image"
        prefix = f"Context: OCR transcription from {img_info} on page {page_num} of {doc_type_label} '{orig_filename}'.\nContent:\n"
    elif block_type == "image_description":
        img_info = f"image '{source_img_filename}'" if source_img_filename else "an image"
        prefix = f"Context: Description of {img_info} on page {page_num} of {doc_type_label} '{orig_filename}'.\nContent:\n"
    elif block_type == "full_page_ocr_text":
        ocr_source_info = f"via {ocr_source}" if ocr_source else ""
        prefix = f"Context: Full page OCR text from scanned page {page_num} of {doc_type_label} '{orig_filename}' {ocr_source_info}.\nContent:\n"
    elif block_type == "title_document":
        prefix = f"Context: Document Title from {doc_type_label} '{orig_filename}'(P{page_num},F{font_size:.0f}{is_bold_str}): \n"
    elif block_type in ["h1_heading", "h2_heading", "h3_heading", "heading_1", "heading_2", "heading_3", "h_candidate_uppercase", "pptx_slide_text"]:
        level_map = {
            "h1_heading": "H1", "h2_heading": "H2", "h3_heading": "H3",
            "heading_1": "Heading 1", "heading_2": "Heading 2", "heading_3": "Heading 3",
            "h_candidate_uppercase": "Potential Heading",
            "pptx_slide_text": "PowerPoint Slide Content" # Special case for PPTX converted content
        }
        level_str = level_map.get(block_type, "Heading")
        prefix = f"Context: {level_str} from {doc_type_label} '{orig_filename}', section/page {page_num} (F{font_size:.0f}{is_bold_str}).\nContent:\n"
    elif block_type == "excel_sheet_data": # For sheets that might have been detected as whole text blocks in PDF
        prefix = f"Context: Data from Excel sheet of {doc_type_label} '{orig_filename}', sheet/page {page_num}.\nContent:\n"
    elif block_type == "docx_paragraph": # For paragraphs from Word docs
        prefix = f"Context: Paragraph from Word document '{orig_filename}', page {page_num}.\nContent:\n"
    else: # Default for generic text paragraphs
        # Heuristic for detecting potential sub-headings within generic text blocks
        lines = original_chunk_text.split('\n', 1)
        first_line = lines[0].strip()
        # Check if first line is short, few words, not ending in punctuation (common for headings)
        if (1 < len(first_line.split()) < 9 and
            count_tokens(first_line) < 40 and
            not first_line.endswith(('.', '?', '!')) and
            (first_line.isupper() or first_line.istitle() or is_bold_str)):
            prefix = f"Context: From {doc_type_label} '{orig_filename}', section/page {page_num}, likely section titled '{first_line}'.\nContent:\n"
        else:
            prefix = f"Context: Text from {doc_type_label} '{orig_filename}', section/page {page_num}.\nContent:\n"

    return prefix + original_chunk_text

def _chunk_parsed_blocks(
    parsed_blocks: List[Dict], collection_name_as_doc_id: str, original_filename_for_meta: str
) -> List[Dict]:
    all_final_chunks = []
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=PRIMARY_CHUNK_SIZE_TOKENS,
        chunk_overlap=PRIMARY_CHUNK_OVERLAP_TOKENS,
        length_function=count_tokens,
        separators=SEPARATORS_HIERARCHICAL,
        keep_separator=False,
    )
    for block_idx, block in enumerate(parsed_blocks):
        original_content = block.get("content", "").strip()
        block_type = block.get("type", "unknown")
        if not original_content or "noise" in block_type or block_type == "error":
            continue

        current_block_metadata = {k:v for k,v in block.items() if k != "content"}
        current_block_metadata.update({"doc_id": collection_name_as_doc_id, "original_filename": original_filename_for_meta})
        current_block_metadata.pop("content", None)

        APP_STATE["original_doc_filename_for_chunking_context"] = original_filename_for_meta
        temp_prefixed_content = _prepend_context_to_chunk(original_content, block)
        APP_STATE["original_doc_filename_for_chunking_context"] = None

        prefixed_token_count = count_tokens(temp_prefixed_content)
        original_token_count = count_tokens(original_content)

        should_split = False
        if prefixed_token_count > EFFECTIVE_MAX_CHUNK_TOKENS:
            logger.warning(f"Block {block.get('block_id', f'b{block_idx}')} (pg:{block.get('page_number')}, type:{block_type}) prefixed form ({prefixed_token_count} tokens) exceeds max {EFFECTIVE_MAX_CHUNK_TOKENS}. Will be split.")
            should_split = True
        elif block_type in ["text_paragraph", "full_page_ocr_text", "docx_paragraph", "excel_sheet_data", "pptx_slide_text", "image_ocr_text", "image_description", "table_markdown"] and original_token_count > PRIMARY_CHUNK_SIZE_TOKENS: # Added table_markdown to types that can be split if too large
            should_split = True
        elif block_type not in ["text_paragraph", "full_page_ocr_text", "docx_paragraph", "excel_sheet_data", "pptx_slide_text", "image_ocr_text", "image_description", "table_markdown"] and original_token_count > ATOMIC_BLOCK_NO_SPLIT_THRESHOLD:
             pass

        if not should_split:
            chunk_id = f"{block.get('block_id', f'b{block_idx}')}_chunk0"
            final_meta = {**current_block_metadata, "chunk_id": chunk_id, "original_block_id": block.get('block_id')}
            all_final_chunks.append({
                "text_content_for_embedding": temp_prefixed_content,
                "original_chunk_text": original_content,
                "metadata": final_meta
            })
        else:
            sub_texts = text_splitter.split_text(original_content)
            logger.debug(f"  Splitting block {block.get('block_id')} ({block_type}, pg {block.get('page_number')}) into {len(sub_texts)} sub-chunks.")
            for i, sub_text_content in enumerate(sub_texts):
                sub_text_content = sub_text_content.strip()
                if not sub_text_content: continue

                APP_STATE["original_doc_filename_for_chunking_context"] = original_filename_for_meta
                final_sub_chunk_for_embedding = _prepend_context_to_chunk(sub_text_content, block)
                APP_STATE["original_doc_filename_for_chunking_context"] = None

                final_sub_chunk_token_count = count_tokens(final_sub_chunk_for_embedding)
                if final_sub_chunk_token_count > EFFECTIVE_MAX_CHUNK_TOKENS:
                    logger.warning(f"    Sub-chunk {i} of {block.get('block_id')} (pg:{block.get('page_number')}) still too large ({final_sub_chunk_token_count} tokens) after splitting. TRUNCATING. Text: '{final_sub_chunk_for_embedding[:100]}...'")
                    tokenizer = APP_STATE.get("tokenizer")
                    if tokenizer:
                        encoded = tokenizer.encode(final_sub_chunk_for_embedding, disallowed_special=())
                        final_sub_chunk_for_embedding = tokenizer.decode(encoded[:EFFECTIVE_MAX_CHUNK_TOKENS])
                    else:
                        final_sub_chunk_for_embedding = final_sub_chunk_for_embedding[:EFFECTIVE_MAX_CHUNK_TOKENS * 4]

                chunk_id = f"{block.get('block_id', f'b{block_idx}')}_subchunk{i}"
                final_meta = {**current_block_metadata, "chunk_id": chunk_id, "original_block_id": block.get('block_id'), "split_index": i}
                all_final_chunks.append({
                    "text_content_for_embedding": final_sub_chunk_for_embedding,
                    "original_chunk_text": sub_text_content,
                    "metadata": final_meta
                })
    logger.info(f"Chunking for '{original_filename_for_meta}': {len(all_final_chunks)} final chunks.")
    return all_final_chunks

def _embed_chunks(chunks_to_embed: List[Dict], batch_size: int = 16) -> Optional[List[Dict]]:
    if not APP_STATE.get("rag_models_loaded") or not APP_STATE.get("azure_embedding_client"):
        logger.error("Azure Embedding client not ready.")
        return None

    texts = [c["text_content_for_embedding"] for c in chunks_to_embed]
    if not texts:
        logger.warning("No texts to embed.")
        return []

    logger.info(f"Embedding {len(texts)} chunks using Azure OpenAI '{AZURE_EMBEDDING_MODEL_NAME}'...")
    try:
        response = APP_STATE["azure_embedding_client"].embed(
            input=texts,
            model=AZURE_EMBEDDING_MODEL_NAME
        )
        embeddings_list = []
        for item in response.data:
            embeddings_list.append(item.embedding)

        embeddings_np = np.array(embeddings_list, dtype=np.float32)

        for c, emb in zip(chunks_to_embed, embeddings_np):
            c["embedding_vector"] = emb.tolist()
        logger.success(f"Embedded {len(texts)} chunks using Azure OpenAI.")
        return chunks_to_embed
    except Exception as e:
        logger.error(f"Azure Embedding FAILED: {e}", exc_info=True)
        return None

def _store_chunks_in_chromadb(
    chunks_with_embeddings: List[Dict], collection: chromadb.api.models.Collection.Collection
) -> int:
    if not collection:
        logger.error("Chroma collection invalid.")
        return 0

    ids, embs, metas, docs = [], [], [], []
    valid_count = 0
    for chunk in chunks_with_embeddings:
        if "embedding_vector" not in chunk or not chunk.get("metadata", {}).get("chunk_id"):
            logger.warning(f"Skip chunk: missing embedding_vector/chunk_id. BlockID: {chunk.get('metadata',{}).get('original_block_id')}")
            continue

        meta = chunk["metadata"]
        ids.append(meta["chunk_id"])
        embs.append(chunk["embedding_vector"])
        docs.append(chunk["original_chunk_text"])

        clean_meta = {}
        for k, v in meta.items():
            if k == "bbox" and isinstance(v, (list, tuple)):
                try: clean_meta[k] = json.dumps(v)
                except TypeError: logger.warning(f"Could not serialize bbox {v}. Storing as str."); clean_meta[k] = str(v)
            elif isinstance(v, (str, int, float, bool)) or v is None:
                clean_meta[k] = v
            else:
                clean_meta[k] = str(v)
        metas.append(clean_meta)
        valid_count += 1

    if not ids:
        logger.warning(f"No valid chunks to store in '{collection.name}'.")
        return 0
    try:
        collection.upsert(ids=ids, embeddings=embs, metadatas=metas, documents=docs)
        logger.success(f"Stored {valid_count} chunks. Collection '{collection.name}' total: {collection.count()}.")
        return valid_count
    except Exception as e:
        logger.error(f"Chroma upsert FAILED for '{collection.name}': {e}", exc_info=True)
        return 0

def ingest_document_into_knowledge_base(
    parsed_blocks: List[Dict], collection_name: str, original_filename: str,
    target_collection: chromadb.api.models.Collection.Collection
) -> bool:
    start_time = time.time()
    logger.info(f"Ingesting '{original_filename}' into collection: {collection_name}")

    if not APP_STATE.get("rag_models_loaded") or not target_collection:
        logger.error(
            f"Ingest prereqs FAIL. RAG loaded: {APP_STATE.get('rag_models_loaded')}, "
            f"Collection valid: {target_collection is not None}."
        )
        return False

    chunks_for_embedding = _chunk_parsed_blocks(parsed_blocks, collection_name, original_filename)
    if not chunks_for_embedding:
        logger.error(f"Chunking for '{original_filename}' FAIL.")
        return False

    chunks_with_vectors = _embed_chunks(chunks_for_embedding)
    if not chunks_with_vectors:
        logger.error(f"Embedding for '{original_filename}' FAIL.")
        return False

    num_stored = _store_chunks_in_chromadb(chunks_with_vectors, target_collection)
    success = num_stored > 0

    logger.log(
        "SUCCESS" if success else "ERROR",
        f"Ingest '{original_filename}' ({collection_name}) in {time.time()-start_time:.2f}s. "
        f"Stored: {num_stored}."
    )
    return success

# --- 10. Retrieval Logic ---
def retrieve_relevant_context(
    query_text: str, target_collection_names: List[str],
    k_initial_retrieval: int = 20, k_final_target: int = 12,
    use_reranking: bool = True
) -> Dict:
    retrieval_start_time = time.time()
    payload = {
        "query": query_text,
        "formatted_context_for_llm": "Error: Retrieval fail.",
        "source_chunks_retrieved": [],
        "retrieval_metadata": {
            "collections_queried": target_collection_names,
            "k_initial": k_initial_retrieval,
            "k_final": k_final_target,
        },
    }

    required_keys = ["rag_models_loaded", "db_initialized", "chroma_client", "azure_embedding_client"]
    if not all(APP_STATE.get(k) for k in required_keys):
        payload["retrieval_metadata"]["error"] = "RAG/DB systems not ready (Azure Embedding client missing)."
        logger.error(payload["retrieval_metadata"]["error"])
        return payload

    try:
        query_response = APP_STATE["azure_embedding_client"].embed(
            input=[query_text],
            model=AZURE_EMBEDDING_MODEL_NAME
        )
        query_emb = query_response.data[0].embedding
    except Exception as e:
        payload["retrieval_metadata"]["error"] = f"Query embed fail with Azure: {e}"
        logger.error(f"Query embed fail with Azure: {e}")
        return payload

    candidates = []
    noise_types = ["noise_header_footer", "noise_short_irrelevant", "info"]
    is_about_query = any(p in query_text.lower() for p in ["about this document", "summarize", "overview", "main idea", "tell me about"])
    anchors = []

    for coll_name in target_collection_names:
        coll = get_or_create_collection_cached(coll_name)
        if not coll:
            logger.warning(f"Skip collection '{coll_name}' as it's invalid or couldn't be accessed.")
            continue

        current_collection_candidates_map = {}

        try:
            res = coll.query(
                query_embeddings=[query_emb],
                n_results=k_initial_retrieval,
                where={"type": {"$nin": noise_types}},
                include=['metadatas', 'documents', 'distances']
            )
            if res and res.get('ids') and res['ids'][0]:
                for i in range(len(res['ids'][0])):
                    candidate_id = res['ids'][0][i]
                    current_collection_candidates_map[candidate_id] = {
                        "id": candidate_id,
                        "text_content": res['documents'][0][i],
                        "metadata": res['metadatas'][0][i],
                        "retrieval_score_distance": res['distances'][0][i],
                        "source_stage": f"S1_Semantic_{coll_name}"
                    }
        except Exception as e:
            logger.error(f"Chroma S1 semantic query failed for '{coll_name}': {e}")

        if is_about_query and len(target_collection_names) == 1:
            try:
                res_anc = coll.query(
                    query_embeddings=[query_emb],
                    n_results=5,
                    where={"$and": [
                        {"type": {"$in": ["title_document", "h1_heading", "pptx_slide_text", "heading_1", "excel_sheet_data", "image_ocr_text", "image_description"]}},
                        {"page_number": {"$lte": 2}}
                    ]},
                    include=['metadatas', 'documents', 'distances']
                )
                if res_anc and res_anc.get('ids') and res_anc['ids'][0]:
                    page_num_key = lambda x: x["metadata"].get("page_number", 999)
                    temp_anchors_from_coll = sorted([
                        {
                            "id": res_anc['ids'][0][i],
                            "text_content": res_anc['documents'][0][i],
                            "metadata": res_anc['metadatas'][0][i],
                            "retrieval_score_distance": res_anc['distances'][0][i],
                            "source_stage": f"S2_Anchor_{coll_name}"
                        } for i in range(len(res_anc['ids'][0]))
                    ], key=lambda x: (page_num_key(x), x["retrieval_score_distance"]))

                    for anchor_cand in temp_anchors_from_coll[:min(2, len(temp_anchors_from_coll))]:
                        current_collection_candidates_map[anchor_cand['id']] = anchor_cand
                        anchors.append(anchor_cand)
            except Exception as e:
                logger.error(f"Chroma S2 anchor query failed for '{coll_name}': {e}")

        candidates.extend(list(current_collection_candidates_map.values()))

    final_anchors = list({ac['id']: ac for ac in anchors}.values())

    if not candidates:
        payload["formatted_context_for_llm"] = "No relevant context found in the document(s)."
        return payload

    selected_chunks = list(final_anchors)
    general_candidates = [c for c in candidates if c['id'] not in {fa['id'] for fa in final_anchors}]

    slots_left_for_general = k_final_target - len(selected_chunks)

    if slots_left_for_general > 0 and general_candidates:
        if use_reranking and APP_STATE.get("reranker_model"):
            try:
                rerank_pairs = [[query_text, chunk['text_content']] for chunk in general_candidates]

                scores = APP_STATE["reranker_model"].predict(rerank_pairs, show_progress_bar=False)
                for chunk, score in zip(general_candidates, scores):
                    chunk['rerank_score'] = score

                if is_about_query and len(target_collection_names) == 1:
                    for chunk in general_candidates:
                        boost = 0.0
                        meta_type = chunk.get("metadata",{}).get("type")
                        meta_page = chunk.get("metadata",{}).get("page_number", 999)
                        if meta_type == "title_document": boost = 5.0
                        elif meta_type in ["h1_heading", "heading_1", "pptx_slide_text"] and meta_page <= 2: boost = 2.5
                        elif meta_type in ["h2_heading", "heading_2"] and meta_page <= 3: boost = 1.0
                        elif meta_type in ["excel_sheet_data", "image_ocr_text", "image_description"] and meta_page <= 2: boost = 1.5
                        chunk['rerank_score'] = chunk.get('rerank_score', 0) + boost

                general_candidates.sort(key=lambda x: x.get('rerank_score', -float('inf')), reverse=True)
                payload["retrieval_metadata"]["reranked_general_candidates"] = True
            except Exception as e:
                logger.error(f"Reranking FAILED: {e}. Falling back to semantic scores for general candidates.")
                general_candidates.sort(key=lambda x: x.get('retrieval_score_distance', float('inf')))
                payload["retrieval_metadata"]["reranked_general_candidates"] = False
        else:
            general_candidates.sort(key=lambda x: x.get('retrieval_score_distance', float('inf')))
            payload["retrieval_metadata"]["reranked_general_candidates"] = False

        selected_chunks.extend(general_candidates[:slots_left_for_general])

    if not selected_chunks:
        payload["formatted_context_for_llm"] = "No relevant context found after selection and reranking."
        return payload

    def sort_key_final(chunk):
        score = chunk.get('rerank_score', -float('inf')) if use_reranking and APP_STATE.get("reranker_model") else -chunk.get('retrieval_score_distance', float('-inf'))
        page = chunk.get('metadata', {}).get('page_number', 999)
        bbox_str = chunk.get('metadata', {}).get('bbox', None)
        y_order = float('inf')
        if bbox_str:
            try:
                bbox = json.loads(bbox_str) if isinstance(bbox_str, str) else bbox_str
                if isinstance(bbox, (list, tuple)) and len(bbox) >= 2:
                    y_order = bbox[1]
            except:
                pass
        return (score, -page, -y_order)

    selected_chunks.sort(key=sort_key_final, reverse=True)

    formatted_context_parts = []
    for i, chunk in enumerate(selected_chunks):
        meta = chunk['metadata']
        orig_fname = meta.get('original_filename', 'UnknownDocument')
        doc_type = meta.get('document_type', 'document')
        bbox_val = meta.get("bbox", "N/A")

        content_for_llm = chunk.get('text_content', '')

        bbox_str_fmt = "N/A"
        if isinstance(bbox_val, str):
            try:
                parsed_bbox = json.loads(bbox_val)
                bbox_str_fmt = f"({parsed_bbox[0]:.0f},{parsed_bbox[1]:.0f},{parsed_bbox[2]:.0f},{parsed_bbox[3]:.0f})"
            except (json.JSONDecodeError, TypeError, IndexError):
                bbox_str_fmt = bbox_val
        elif isinstance(bbox_val, (list, tuple)) and len(bbox_val) == 4:
            bbox_str_fmt = f"({bbox_val[0]:.0f},{bbox_val[1]:.0f},{bbox_val[2]:.0f},{bbox_val[3]:.0f})"


        header = (
            f"--- Context Source {i+1} (Doc:'{orig_fname}', Type:{doc_type}, SrcStage:{chunk.get('source_stage','N/A')}) ---\n"
            f"Page/Section: {meta.get('page_number','?')}, Type: {meta.get('type','?')}, ChunkID: {meta.get('chunk_id','N/A')}\n"
            f"Font Size: {meta.get('font_size',0):.0f}, Bold: {meta.get('is_bold',False)}, Semantic Heading: {meta.get('is_semantic_heading',False)}\n"
            f"BBox: {bbox_str_fmt}, Distance: {chunk.get('retrieval_score_distance',-1):.4f}"
        )
        if 'rerank_score' in chunk:
            header += f", RerankScore: {chunk['rerank_score']:.4f}"

        formatted_context_parts.append(f"{header}\nContent:\n{content_for_llm}\n--- End Src {i+1} ---")
        payload["source_chunks_retrieved"].append(chunk)

    payload["formatted_context_for_llm"] = "\n\n".join(formatted_context_parts)
    payload["retrieval_metadata"]["final_chunk_count"] = len(selected_chunks)

    logger.success(
        f"Context retrieval for '{query_text[:30]}...' in {time.time()-retrieval_start_time:.2f}s. "
        f"Final chunks: {len(selected_chunks)}."
    )
    return payload

# --- 11. Response Generation ---
def _generate_llm_messages(
    user_query: str,
    context_str: str,
    document_identifier: str,
    dynamic_role_instructions: Optional[str],
    is_summary_context: bool,
) -> List[Dict]:
    system_content_parts = [
        f"You are {AI_PERSONA_NAME}, {AI_ROLE_DESCRIPTION}",
        AI_CORE_DIRECTIVE.replace("'{document_name}'", f"'{document_identifier}'"),
        AI_ANSWERING_STYLE_REFINED,
        AI_CONTEXTUAL_PRIORITIZATION_POLICY,
        AI_CITATION_POLICY_TEXT,
        AI_NO_ANSWER_POLICY,
    ]

    if is_summary_context:
        system_content_parts.append(AI_SUMMARIZATION_POLICY)
        context_header = "PROVIDED SUMMARY (Answer ONLY from this summary):"
    else:
        context_header = "PROVIDED CONTEXT (Answer ONLY from this):"

    if dynamic_role_instructions and dynamic_role_instructions.strip():
        system_content_parts.extend([
            "\n--- ADDITIONAL ROLE GUIDANCE ---",
            dynamic_role_instructions.strip(),
            "--- END GUIDANCE ---",
            "Strictly adhere to ALL guidance."
        ])

    system_message = {"role": "system", "content": "\n\n".join(system_content_parts).strip()}
    user_message = {"role": "user", "content": f"USER'S QUERY: {user_query.strip()}\n\n{context_header}\n{context_str.strip()}"}

    return [system_message, user_message]


def generate_standard_llm_response(
    user_query: str, formatted_context_from_rag: str,
    source_chunks_from_rag: List[Dict], doc_name_identifier: str,
    dynamic_role_info: Optional[str], response_payload: Dict
) -> Dict:
    messages = _generate_llm_messages(
        user_query, formatted_context_from_rag, doc_name_identifier,
        dynamic_role_info, is_summary_context=False
    )
    response_payload["llm_prompt_sent"] = messages

    try:
        logger.info(f"Sending RAG request to OpenAI ('{GENERATION_MODEL_NAME}'). Query: '{user_query[:70]}...'")
        gen_start_time = time.time()

        if not APP_STATE.get("azure_llm_client"):
            logger.error("OpenAI LLM client instance not available.")
            response_payload.update({"answer_text": "Error: AI model not configured.", "error_message": "AI model not configured."})
            return response_payload

        openai_api_response = APP_STATE["azure_llm_client"].chat.completions.create(
            model=AZURE_LLM_MODEL_NAME,
            messages=messages,
            temperature=0.25,
            max_tokens=1000,
            top_p=1.0,
        )
        response_payload["generation_time_s"] = round(time.time() - gen_start_time, 2)
        logger.success(f"OpenAI LLM response in {response_payload['generation_time_s']:.2f}s.")

        try:
            response_payload["llm_full_response_obj_str"] = str(openai_api_response)
        except Exception:
            response_payload["llm_full_response_obj_str"] = "Could not serialize full OpenAI response object."

        if openai_api_response.choices and openai_api_response.choices[0].message:
            generated_text = openai_api_response.choices[0].message.content.strip()
            response_payload["answer_text"] = generated_text

            citations_found_raw = re.findall(r'(\[Doc:[^\]]+?CkID:[^\]]+?\])', generated_text)
            if citations_found_raw:
                logger.warning(
                    f"LLM included {len(citations_found_raw)} bracketed citations despite instruction. "
                    "These should not be displayed in the final answer if the prompt was followed."
                )
            response_payload["parsed_citations"] = [{"tag": c} for c in citations_found_raw]
        else:
            logger.error("OpenAI API response did not contain expected message content.")
            response_payload.update({
                "answer_text": "Error: Could not parse text from AI response.",
                "error_message": "OpenAI response parsing error: No choices or message found."
            })

    except Exception as e:
        logger.error(f"General error during OpenAI API call: {e}", exc_info=True)
        response_payload.update({
            "answer_text": f"Error: LLM call failed ({type(e).__name__}).",
            "error_message": str(e)
        })

    response_payload.setdefault("answer_text", "Error: Processing failed to produce an answer.")
    response_payload.setdefault("parsed_citations", [])
    return response_payload

def generate_llm_response(
    user_query: str, formatted_context_from_rag: str,
    source_chunks_from_rag: List[Dict], default_doc_name: str = "the document",
    dynamic_role_info: Optional[str] = None, is_aboutness_query_flag: bool = False
) -> Dict:
    response_payload = {
        "answer_text": "Error: LLM response could not be generated.",
        "llm_prompt_sent": "",
        "llm_full_response_obj_str": None,
        "parsed_citations": [],
        "error_message": None,
        "generation_time_s": 0.0,
        "summarization_chain_used": False,
    }

    required_llm_keys = ["llm_initialized", "azure_llm_client", "langchain_llm"]
    if not all(APP_STATE.get(k) for k in required_llm_keys):
        logger.error("LLM systems not initialized (OpenAI client or LangChain LLM missing).")
        response_payload["error_message"] = "LLM systems not initialized."
        return response_payload

    doc_names = sorted(list(set(
        c['metadata'].get('original_filename', default_doc_name)
        for c in source_chunks_from_rag if c.get('metadata')
    )))
    doc_id_for_prompt = default_doc_name
    if doc_names:
        if len(doc_names) == 1:
            doc_id_for_prompt = f"document '{doc_names[0]}'"
        else:
            doc_id_for_prompt = f"documents ({', '.join(doc_names)})"

    context_total_text = "\n\n".join([
        c.get("original_chunk_text", c.get("text_content",""))
        for c in source_chunks_from_rag
    ])
    context_token_count = count_tokens(context_total_text)

    if is_aboutness_query_flag and context_token_count > MAX_TOKENS_FOR_DIRECT_LLM_SUMMARIZATION and source_chunks_from_rag:
        logger.info(
            f"Aboutness query with large context ({context_token_count} tokens > {MAX_TOKENS_FOR_DIRECT_LLM_SUMMARIZATION}). "
            "Attempting LangChain map-reduce summarization."
        )
        response_payload["summarization_chain_used"] = True

        langchain_documents = [
            Document(page_content=chunk.get("original_chunk_text", chunk.get("text_content","")), metadata=chunk.get("metadata", {}))
            for chunk in source_chunks_from_rag if chunk.get("original_chunk_text", chunk.get("text_content","")).strip()
        ]

        if not langchain_documents:
            logger.warning("No valid documents for LangChain summarization. Falling back to standard generation.")
            return generate_standard_llm_response(
                user_query, formatted_context_from_rag, source_chunks_from_rag,
                doc_id_for_prompt, dynamic_role_info, response_payload
            )
        try:
            summarize_chain = load_summarize_chain(
                llm=APP_STATE["langchain_llm"], chain_type="map_reduce", verbose=False
            )
            summarization_question = (
                f"Provide a comprehensive summary of the key information in '{doc_id_for_prompt}' "
                f"that is relevant to the user's query: '{user_query}'"
            )

            start_time_lc_summarize = time.time()
            summary_result = summarize_chain.invoke({
                "input_documents": langchain_documents,
                "question": summarization_question
            })
            response_payload["generation_time_s"] = round(time.time() - start_time_lc_summarize, 2)

            raw_summary_text = summary_result.get("output_text", "Error: Summarization failed to produce text.").strip()
            logger.success(
                f"LangChain summarization completed in {response_payload['generation_time_s']:.2f}s. "
                f"Raw summary length: {len(raw_summary_text)} characters."
            )

            if not raw_summary_text or "Error:" in raw_summary_text or "failed to produce" in raw_summary_text.lower():
                logger.warning("LangChain summary was empty or indicated failure. Falling back to standard generation.")
                return generate_standard_llm_response(
                    user_query, formatted_context_from_rag, source_chunks_from_rag,
                    doc_id_for_prompt, dynamic_role_info, response_payload
                )

            messages_refine = _generate_llm_messages(
                user_query, raw_summary_text, doc_id_for_prompt,
                dynamic_role_info, is_summary_context=True
            )
            response_payload["llm_prompt_sent"] = messages_refine

            start_time_final_refine = time.time()
            final_openai_response = APP_STATE["azure_llm_client"].chat.completions.create(
                model=AZURE_LLM_MODEL_NAME,
                messages=messages_refine,
                temperature=0.25,
                max_tokens=1000,
                top_p=1.0,
            )
            response_payload["generation_time_s"] += round(time.time() - start_time_final_refine, 2)
            response_payload["llm_full_response_obj_str"] = str(final_openai_response)

            if final_openai_response.choices and final_openai_response.choices[0].message:
                refined_answer_text = final_openai_response.choices[0].message.content.strip()
                response_payload["answer_text"] = refined_answer_text + f"\n\n(This answer is based on a summary of {doc_id_for_prompt}.)"
                logger.info("Summary refined by main LLM.")
                response_payload["parsed_citations"] = []
            else:
                logger.error("OpenAI summary refinement response did not contain expected message content.")
                response_payload.update({
                    "answer_text": "Error: Could not parse text from AI's refined summary.",
                    "error_message": "OpenAI summary refinement parsing error: No choices or message found."
                })

        except Exception as e_lc_summarize:
            logger.error(f"LangChain summarization process FAILED: {e_lc_summarize}. Falling back to standard generation.", exc_info=True)
            response_payload.update({
                "error_message": f"Summarization chain error: {str(e_lc_summarize)}. Fallback executed.",
                "summarization_chain_used": False
            })
            return generate_standard_llm_response(
                user_query, formatted_context_from_rag, source_chunks_from_rag,
                doc_id_for_prompt, dynamic_role_info, response_payload
            )
    else:
        return generate_standard_llm_response(
            user_query, formatted_context_from_rag, source_chunks_from_rag,
            doc_id_for_prompt, dynamic_role_info, response_payload
        )

    response_payload.setdefault("answer_text", "Error: Summarization path failed to produce a final answer.")
    response_payload.setdefault("parsed_citations", [])
    return response_payload

# --- One-time Initialization for AI/DB systems ---
def perform_initial_setup():
    logger.info("Flask App: Performing one-time initial setup for AI/DB systems...")
    if not AZURE_LLM_API_KEY_SECRET:
        logger.critical("AZURE_LLM_API_KEY_SECRET MISSING. APP CANNOT FUNCTION.")
        return False

    all_ok = True
    if not initialize_rag_models_from_kb():
        all_ok = False
    if not initialize_llm_model():
        all_ok = False
    if not initialize_chromadb_from_kb():
        all_ok = False
    if HF_API_TOKEN_SECRET and not initialize_hf_client_from_parser():
        logger.warning("HF Vision client (for parser) initialization failed or was skipped. Vision features might be limited.")

    if all_ok:
        logger.success("Flask App: All critical AI/DB systems initialized.")
    else:
        logger.error("Flask App: CRITICAL - One or more essential AI/DB system initializations FAILED.")
    return all_ok

APP_SYSTEMS_READY = perform_initial_setup()

# --- 12. Flask App Definition and Endpoints ---
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, 'templates')
STATIC_DIR = os.path.join(BASE_DIR, 'static')
STAGED_UPLOADS_DIR = os.path.join(BASE_DIR, 'colab_staged_uploads_final_nocite')

os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(STATIC_DIR, exist_ok=True)
os.makedirs(os.path.join(STATIC_DIR, 'js'), exist_ok=True)
os.makedirs(STAGED_UPLOADS_DIR, exist_ok=True)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
CORS(app)

# --- HTML Templates (Placeholder - create these files) ---

@app.route('/')
def hero_page_route():
    hero_html_path = os.path.join(TEMPLATES_DIR, 'hero-geometric.html')
    if not os.path.exists(hero_html_path):
        with open(hero_html_path, 'w') as f:
            f.write('<!DOCTYPE html><html><head><title>DocuMind AI</title><style>body{font-family: sans-serif; display: flex; flex-direction: column; align-items: center; justify-content: center; height: 100vh; margin: 0; background-color: #f0f2f5;} h1{color: #333;} a{text-decoration: none; padding: 10px 20px; background-color: #007bff; color: white; border-radius: 5px;}</style></head><body><h1>Welcome to DocuMind AI</h1><p>Your intelligent document assistant.</p><a href="/chatui">Start Chatting</a></body></html>')
    return render_template('hero-geometric.html')

@app.route('/chatui')
def chat_page_route():
    chat_html_path = os.path.join(TEMPLATES_DIR, 'chat.html')
    if not os.path.exists(chat_html_path):
        with open(chat_html_path, 'w') as f:
            f.write('<!DOCTYPE html><html><head><title>DocuMind AI Chat</title><style>body{font-family: sans-serif; padding: 20px; background-color: #f0f2f5;} h1{color: #333; text-align: center;}</style></head><body><h1>Chat with Your Documents</h1><div><!-- Basic chat UI elements would be added here by frontend JS --> <p>Chat interface placeholder. Real UI would be built with JavaScript.</p> <a href="/">Back to Home</a></div></body></html>')
    return render_template('chat.html')

@app.route('/stage_file', methods=['POST'])
def stage_single_file_route():
    logger.info("--- API Call: /stage_file ---")
    if 'file' not in request.files:
        return jsonify({"error": "No 'file' part in the request."}), 400

    file_obj = request.files['file']
    original_filename = request.form.get('originalFilename', file_obj.filename)

    if not file_obj or not original_filename:
        return jsonify({"error": "No file provided or filename is missing."}), 400

    safe_original_filename = re.sub(r'[^a-zA-Z0-9._-]', '_', original_filename)

    staging_id = f"staged_{uuid.uuid4().hex[:8]}_{safe_original_filename}"
    staged_file_path = os.path.join(STAGED_UPLOADS_DIR, staging_id)

    try:
        file_obj.save(staged_file_path)
        file_size = os.path.getsize(staged_file_path)
        APP_WEB_STATE["staged_files"][staging_id] = {
            "original_filename": original_filename,
            "staged_path": staged_file_path,
            "status": "staged",
            "size": file_size
        }
        logger.info(f"  Staged '{original_filename}' (ID:{staging_id}, Size:{file_size}B) to '{staged_file_path}'.")
        return jsonify({
            "message": "File staged successfully.",
            "original_filename": original_filename,
            "staging_id": staging_id,
            "status": "staged"
        }), 200
    except Exception as e:
        logger.error(f"  Error staging '{original_filename}': {e}", exc_info=True)
        return jsonify({"error": f"File staging failed: {str(e)}"}), 500

@app.route('/process_staged_files', methods=['POST'])
def process_staged_files_route():
    logger.info("--- API Call: /process_staged_files ---")
    if not APP_SYSTEMS_READY:
        logger.error("Backend AI/DB systems are not ready. Cannot process files.")
        return jsonify({"error": "Backend systems are not ready. Please try again later."}), 503

    data = request.get_json()
    if not data:
        logger.warning("/process_staged_files: No JSON data received.")
        return jsonify({"error": "Invalid JSON payload."}), 400

    staging_ids = data.get('staging_ids', [])
    options = data.get('options', {})
    logger.info(f"  Processing options: {options}. Staging IDs: {staging_ids}")

    processed_docs_summary = []
    errors_during_processing = []

    for staging_id in staging_ids:
        staged_info = APP_WEB_STATE["staged_files"].get(staging_id)
        if not staged_info or staged_info.get("status") != "staged":
            err_msg = f"File with staging ID '{staging_id}' not found or not in 'staged' state."
            errors_during_processing.append({"staging_id": staging_id, "filename": staged_info.get("original_filename", "Unknown"), "error": err_msg})
            logger.warning(f"  {err_msg}")
            continue

        path_to_staged_file = staged_info["staged_path"]
        actual_original_filename = staged_info["original_filename"]
        logger.info(f"  Processing '{actual_original_filename}' (ID: {staging_id}) from '{path_to_staged_file}'")

        profile_to_use = "fastest"
        if options.get("handwritten", False) or options.get("images", False):
            profile_to_use = "comprehensive_vision" if APP_STATE.get("vision_client_initialized") else "digital_plus_ocr"
        elif options.get("ocr", False):
            profile_to_use = "digital_plus_ocr"
        elif not options:
            profile_to_use = "default_fallback"

        logger.info(f"    Effective parsing profile for '{actual_original_filename}': '{profile_to_use}'")

        try:
            parsed_blocks = parse_document_via_profile(path_to_staged_file, profile_name=profile_to_use)

            if parsed_blocks and parsed_blocks[0].get("type") == "error":
                err_content = parsed_blocks[0].get('content', 'Unknown parsing error')
                errors_during_processing.append({"staging_id": staging_id, "filename": actual_original_filename, "error": f"Parsing/Conversion failed: {err_content}"})
                staged_info["status"] = "parse_failure"
                logger.error(f"    Parsing/Conversion failed for '{actual_original_filename}': {err_content}")
                continue

            base_collection_name = f"{CHROMA_COLLECTION_NAME_PREFIX}_{os.path.splitext(actual_original_filename)[0]}_{uuid.uuid4().hex[:6]}"
            final_collection_name = sanitize_chromadb_collection_name(base_collection_name)

            target_collection_obj = get_or_create_collection_cached(final_collection_name)
            if not target_collection_obj:
                err_msg = f"Failed to get or create ChromaDB collection '{final_collection_name}'."
                errors_during_processing.append({"staging_id": staging_id, "filename": actual_original_filename, "error": err_msg})
                staged_info["status"] = "db_collection_failure"
                logger.error(f"    {err_msg} for '{actual_original_filename}'")
                continue

            ingestion_successful = ingest_document_into_knowledge_base(
                parsed_blocks, final_collection_name, actual_original_filename, target_collection_obj
            )

            if ingestion_successful:
                APP_WEB_STATE["processed_collections"][actual_original_filename] = final_collection_name
                staged_info.update({"status": "processed", "collection_name": final_collection_name})
                processed_docs_summary.append({
                    "filename": actual_original_filename,
                    "collection_name": final_collection_name,
                    "staging_id": staging_id,
                    "status": "processed"
                })
                logger.info(f"    Successfully ingested '{actual_original_filename}' into collection '{final_collection_name}'.")
                try:
                    os.remove(path_to_staged_file)
                    logger.info(f"    Removed staged file '{path_to_staged_file}'.")
                    if staging_id in APP_WEB_STATE["staged_files"]:
                         del APP_WEB_STATE["staged_files"][staging_id]

                except Exception as e_remove:
                    logger.warning(f"    Could not remove staged file '{path_to_staged_file}': {e_remove}")
            else:
                errors_during_processing.append({"staging_id": staging_id, "filename": actual_original_filename, "error": "Ingestion into knowledge base failed."})
                staged_info["status"] = "ingestion_failure"
                logger.error(f"    Ingestion failed for '{actual_original_filename}' into '{final_collection_name}'.")

        except Exception as e_pipeline:
            error_message = f"Unhandled error in processing pipeline for '{actual_original_filename}': {str(e_pipeline)}"
            errors_during_processing.append({"staging_id": staging_id, "filename": actual_original_filename, "error": error_message})
            if staged_info: staged_info["status"] = "pipeline_error"
            logger.error(f"    {error_message}", exc_info=True)

    logger.info(f"--- /process_staged_files call ended. Successfully processed: {len(processed_docs_summary)}, Errors: {len(errors_during_processing)} ---")

    if not processed_docs_summary and errors_during_processing:
        return jsonify({"error": "All file processing attempts failed.", "details": errors_during_processing}), 500

    return jsonify({
        "message": "File processing finished.",
        "processed_documents": processed_docs_summary,
        "errors": errors_during_processing
    }), 200


@app.route('/ask', methods=['POST'])
def ask_question_route():
    logger.info("--- API Call: /ask ---")
    if not APP_SYSTEMS_READY:
        logger.error("Backend AI/DB systems not ready for /ask call.")
        return jsonify({"error": "Backend systems are not ready. Please try again later."}), 503

    data = request.get_json()
    if not data or 'query' not in data:
        logger.warning("/ask: Missing 'query' in JSON payload.")
        return jsonify({"error": "Missing 'query' in request payload."}), 400

    user_query = data['query']
    logger.info(f"  Received query for /ask: '{user_query[:100]}...'")

    active_document_filenames = data.get('active_documents', [])
    collections_to_query_names = []
    query_scope_description = "all processed documents"

    if active_document_filenames:
        logger.info(f"  Frontend specified active documents: {active_document_filenames}")
        valid_collections_for_active_docs = []
        all_active_docs_found = True
        for fname in active_document_filenames:
            coll_name = APP_WEB_STATE["processed_collections"].get(fname)
            if coll_name:
                valid_collections_for_active_docs.append(coll_name)
            else:
                logger.warning(f"    Document '{fname}' specified as active, but not found in processed collections. Query will default to all processed documents.")
                all_active_docs_found = False
                break

        if all_active_docs_found and valid_collections_for_active_docs:
            collections_to_query_names = valid_collections_for_active_docs
            if len(active_document_filenames) == 1:
                query_scope_description = f"document '{active_document_filenames[0]}'"
            else:
                query_scope_description = f"documents: {', '.join(active_document_filenames)}"
            logger.info(f"  Query will target specific collections: {collections_to_query_names}")
        else:
            collections_to_query_names = list(APP_WEB_STATE["processed_collections"].values())
            logger.info(f"  Fallback: Querying ALL {len(collections_to_query_names)} processed collections.")
    else:
        collections_to_query_names = list(APP_WEB_STATE["processed_collections"].values())
        logger.info(f"  No specific documents indicated by frontend. Querying ALL {len(collections_to_query_names)} processed collections.")

    if not collections_to_query_names:
        if not APP_WEB_STATE["processed_collections"]:
            return jsonify({"answer_text": "No documents have been processed yet. Please upload and process a document first.", "parsed_citations": []}), 400
        else:
            logger.error("Internal state error: No collections to query despite having processed documents, or collections list became empty.")
            return jsonify({"answer_text": "Error: No valid document collections available to query at this time.", "parsed_citations": []}), 500

    is_aboutness_type_query = any(
        phrase in user_query.lower() for phrase in ["about this document", "summarize", "overview", "main idea", "tell me about"]
    )
    if is_aboutness_type_query:
        logger.info("  Query detected as an 'aboutness' or summarization request.")

    retrieval_results = retrieve_relevant_context(
        user_query,
        collections_to_query_names,
        use_reranking=(APP_STATE.get("reranker_model") is not None)
    )

    formatted_context = retrieval_results.get("formatted_context_for_llm", "")
    source_chunks = retrieval_results.get("source_chunks_retrieved", [])

    if "Error:" in formatted_context or not source_chunks:
        logger.warning(f"  Context retrieval yielded no usable chunks or an error: {formatted_context[:200]}")

    llm_generation_output = generate_llm_response(
        user_query,
        formatted_context,
        source_chunks,
        default_doc_name=query_scope_description,
        is_aboutness_query_flag=is_aboutness_type_query
    )

    final_answer_text = llm_generation_output.get("answer_text", "Error: AI response generation failed or produced no text.")
    error_message_from_llm = llm_generation_output.get("error_message")

    if error_message_from_llm and "Error:" in final_answer_text:
        logger.warning(f"  LLM generation resulted in an error message displayed as answer: '{final_answer_text}'. Underlying error: {error_message_from_llm}")
    elif final_answer_text == AI_NO_ANSWER_POLICY or not final_answer_text.strip() or "Error: AI" in final_answer_text:
        final_answer_text = "I'm sorry, I couldn't find specific information to answer your question in the provided document(s)."
        logger.info("  AI indicated no answer or returned an empty/error state; providing a polite 'no information' message.")

    logger.info(f"--- /ask call finished. Answer length: {len(final_answer_text)}. Summarization used: {llm_generation_output.get('summarization_chain_used', False)} ---")

    return jsonify({
        "answer_text": final_answer_text,
        "parsed_citations": llm_generation_output.get("parsed_citations", []),
        "llm_prompt_sent": "REDACTED_IN_RESPONSE",
        "error_message": error_message_from_llm,
        "summarization_chain_used": llm_generation_output.get("summarization_chain_used", False)
    })

@app.route('/list_processed_documents_for_chat', methods=['GET'])
def list_processed_docs_for_chat_api():
    processed_filenames = list(APP_WEB_STATE["processed_collections"].keys())
    logger.info(f"API Call: /list_processed_documents_for_chat. Returning {len(processed_filenames)} document filenames.")
    return jsonify({"processed_document_filenames": processed_filenames}), 200

# --- Ngrok and App Run Function ---
public_url_cell4_final = None

def start_flask_app_cell4_final():
    global public_url_cell4_final
    if public_url_cell4_final:
        try:
            ngrok.disconnect(public_url_cell4_final)
            logger.info("Disconnected previous ngrok tunnel.")
        except Exception as e_ngrok_disc:
            logger.warning(f"Could not disconnect previous ngrok tunnel: {e_ngrok_disc}")
        public_url_cell4_final = None

    try:
        NGROK_AUTHTOKEN_SECRET = userdata.get('NGROK_AUTHTOKEN')
        if NGROK_AUTHTOKEN_SECRET:
            conf.get_default().auth_token = NGROK_AUTHTOKEN_SECRET
            logger.info("Ngrok authentication token set successfully.")
        else:
            logger.warning("NGROK_AUTHTOKEN not found in Colab secrets. Ngrok might be rate-limited or fail for non-authenticated users.")
    except userdata.SecretNotFoundError:
        logger.warning("NGROK_AUTHTOKEN secret not found. Proceeding without ngrok authentication.")
    except Exception as e_ngrok_auth:
        logger.warning(f"An error occurred while setting ngrok authtoken: {e_ngrok_auth}")

    if not APP_SYSTEMS_READY:
        logger.critical("CRITICAL: Backend AI/DB systems FAILED initialization. Flask application will not run effectively or may fail.")
        print("Flask app cannot start because critical backend systems are not ready. Check logs for errors.")
        return

    try:
        public_url_cell4_final = ngrok.connect(5000)
        logger.info(f"Flask application (Merged FINAL Version - NoCite) is accessible at: {public_url_cell4_final}")
        print(f" * Ngrok tunnel (FINAL NoCite Version) \"{public_url_cell4_final}\" -> \"http://127.0.0.1:5000\"")

        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

    except Exception as e_flask_run:
        logger.error(f"Error starting Flask application or ngrok tunnel (FINAL NoCite Version): {e_flask_run}", exc_info=True)
        if public_url_cell4_final:
            try:
                ngrok.disconnect(public_url_cell4_final)
            except Exception as e_disc_fatal:
                logger.error(f"Failed to disconnect ngrok tunnel during error handling: {e_disc_fatal}")
        try:
            ngrok.kill()
        except Exception as e_kill_fatal:
            logger.error(f"Failed to kill ngrok processes during error handling: {e_kill_fatal}")
        raise

if __name__ == "__main__":
    pass

start_flask_app_cell4_final()

2025-06-01 09:16:18.063 | INFO     | __main__:<cell line: 0>:62 - AskYourDoc Merged Flask App Starting. Log file: ask_your_doc_flask_merged_app.log
2025-06-01 09:16:18.064 | INFO     | __main__:<cell line: 0>:68 - HF_TOKEN loaded.
2025-06-01 09:16:18.064 | INFO     | __main__:<cell line: 0>:75 - Azure Embedding Model: text-embedding-3-large configured.
2025-06-01 09:16:18.064 | WARNING  | __main__:<cell line: 0>:76 - Using a GitHub token for Azure AI Inference API Key. Please verify this is the correct type of key for your Azure endpoint.
2025-06-01 09:16:18.064 | INFO     | __main__:<cell line: 0>:83 - Azure LLM Model: gpt-4o configured.
2025-06-01 09:16:18.079 | INFO     | __main__:perform_initial_setup:1598 - Flask App: Performing one-time initial setup for AI/DB systems...
2025-06-01 09:16:18.079 | WARNING  | __main__:initialize_rag_models_from_kb:244 - Tiktoken fail: name 'tiktoken' is not defined. Using len().
2025-06-01 09:16:18.080 | INFO     | __main__:initialize_rag_models_fr

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 09:18:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 09:18:26] "GET /favicon.ico HTTP/1.1" 404 -


2025-06-01 09:18:52.833 | INFO     | __main__:stage_single_file_route:1655 - --- API Call: /stage_file ---


INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 09:18:53] "POST /stage_file HTTP/1.1" 200 -


2025-06-01 09:18:53.223 | INFO     | __main__:stage_single_file_route:1679 -   Staged 'HastiVadariyaResume (1) (1).pdf' (ID:staged_81a5e222_HastiVadariyaResume__1___1_.pdf, Size:114517B) to '/content/colab_staged_uploads_final_nocite/staged_81a5e222_HastiVadariyaResume__1___1_.pdf'.


...

2025-06-01 09:19:15.558 | INFO     | __main__:ask_question_route:1799 - --- API Call: /ask ---
2025-06-01 09:19:15.559 | INFO     | __main__:ask_question_route:1810 -   Received query for /ask: 'hey whats the document abot...'
2025-06-01 09:19:15.559 | INFO     | __main__:ask_question_route:1817 -   Frontend specified active documents: ['HastiVadariyaResume (1) (1).pdf']
2025-06-01 09:19:15.559 | INFO     | __main__:ask_question_route:1835 -   Query will target specific collections: ['askdocmerged_hastivadariyaresume_1_1__80d0ed']
2025-06-01 09:19:15.896 | SUCCESS  | __main__:retrieve_relevant_context:1355 - Context retrieval for 'hey whats the document abot...' in 0.34s. Final chunks: 12.
2025-06-01 09:19:15.897 | INFO     | __main__:generate_standard_llm_response:1410 - Sending RAG request to OpenAI ('gpt-4o'). Query: 'hey whats the document abot...'


INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 09:19:19] "POST /ask HTTP/1.1" 200 -


2025-06-01 09:19:19.272 | SUCCESS  | __main__:generate_standard_llm_response:1426 - OpenAI LLM response in 3.37s.
2025-06-01 09:19:19.273 | INFO     | __main__:ask_question_route:1885 - --- /ask call finished. Answer length: 428. Summarization used: False ---
2025-06-01 09:19:45.482 | INFO     | __main__:ask_question_route:1799 - --- API Call: /ask ---
2025-06-01 09:19:45.484 | INFO     | __main__:ask_question_route:1810 -   Received query for /ask: 'can you tell me summary of her current job...'
2025-06-01 09:19:45.484 | INFO     | __main__:ask_question_route:1817 -   Frontend specified active documents: ['HastiVadariyaResume (1) (1).pdf']
2025-06-01 09:19:45.484 | INFO     | __main__:ask_question_route:1835 -   Query will target specific collections: ['askdocmerged_hastivadariyaresume_1_1__80d0ed']
2025-06-01 09:19:45.645 | SUCCESS  | __main__:retrieve_relevant_context:1355 - Context retrieval for 'can you tell me summary of her...' in 0.16s. Final chunks: 12.
2025-06-01 09:19:45.646

INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 09:19:50] "POST /ask HTTP/1.1" 200 -


2025-06-01 09:19:50.025 | SUCCESS  | __main__:generate_standard_llm_response:1426 - OpenAI LLM response in 4.38s.
2025-06-01 09:19:50.026 | INFO     | __main__:ask_question_route:1885 - --- /ask call finished. Answer length: 1374. Summarization used: False ---
